<a href="https://colab.research.google.com/github/laptop4pune/code/blob/master/gemini/getting-started/intro_gemini_2_0_flash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# _1113_6BacktesterV6.py

import pandas as pd
import numpy as np
import logging
from datetime import datetime # Import datetime for timestamp IDs
import uuid  # Keep uuid import in case it's used elsewhere, although not for generate_trade_id now
import sys # Import sys to check for handlers
# from _012_instruments import get_instrument_type # Removed unused import
# --- Logging Configuration ---
logger = logging.getLogger(__name__)



logger.setLevel(logging.DEBUG)

# Also ensure the root logger has a handler and is set to DEBUG,
# in case basicConfig was called elsewhere previously.
if not logging.root.handlers:
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
else:
    # If handlers exist, ensure at least one handler's level is DEBUG
    # and the root logger's level is DEBUG
    logging.root.setLevel(logging.DEBUG)
    handler_found = False
    for handler in logging.root.handlers:
        if isinstance(handler, logging.StreamHandler) and handler.stream in [sys.stdout, sys.stderr]:
            handler.setLevel(logging.DEBUG)
            handler_found = True
    # If no suitable handler is found (e.g., only file handlers), add a StreamHandler
    if not handler_found:
         stream_handler = logging.StreamHandler(sys.stdout)
         stream_handler.setLevel(logging.DEBUG)
         formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(message)s')
         stream_handler.setFormatter(formatter)
         logging.root.addHandler(stream_handler)


print("Logging level set to DEBUG for test.")

# Examine the __init__ method for data validation logic
# Check for the existence of the required_columns and log warnings for potential entry/exit data columns.

# required_columns check
# missing_required = [col for col in required_columns if col not in data.columns]
# if missing_required:
#      raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")

# entry_columns_to_check and exit_data_columns_to_check check
# missing_data_cols = [col for col in entry_columns_to_check + exit_data_columns_to_check if col not in data.columns]
# if missing_data_cols:
#      logger.warning(f"Input data is missing potential indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records.")

# The current code already performs these checks.
# required_columns are checked and raise a ValueError if missing.
# entry_columns_to_check and exit_data_columns_to_check are checked and log a warning if missing.

# Add comments to clarify assumptions about columns expected in input data.

class BacktesterV3:
    """
    A simple backtesting engine for evaluating trading strategies.
    Processes historical data bar by bar, generates signals, and simulates trades.
    """
    def __init__(self, data: pd.DataFrame, instrument_keys: list, active_strategies_instances: dict, initial_capital: float):
        """
        Includes the same parameters as the original __init__

        Initializes the Backtester.

        Args:
            data: A pandas DataFrame containing historical market data for all instruments,
                expected to have columns like 'timestamp', 'instrument_key',
                'open', 'high', 'low', 'close', 'volume', etc. It is also expected
                to contain pre-calculated indicator and pattern columns used by
                the strategies and for recording trade details.
            instrument_keys: A list of unique instrument keys present in the data.
            active_strategies_instances: A dictionary where keys are strategy names
                                        (strings) and values are instantiated strategy
                                        objects with a `generate_signal(data_point)` method.
            initial_capital: The starting capital for the backtest simulation.
        """
        if data is None or data.empty:
            raise ValueError("Input data DataFrame is None or empty.")
        if not isinstance(data, pd.DataFrame):
            raise TypeError("Input 'data' must be a pandas DataFrame.")
        if data.index.name is not None:
            logger.warning("Input data index is not None. Consider resetting the index before passing to Backtester.")


        # Ensure essential columns are present and sorted
        required_columns = ['timestamp', 'instrument_key', 'open', 'high', 'low', 'close']
        # Define columns expected to be in the input data for recording trade details.
        # These are typically pre-calculated indicators or pattern detection results.
        entry_exit_data_columns_expected = [
            'Trend', 'SMA20', 'RSI', 'RSIMA', 'ATR', 'ADX', 'Volatility',
            'Breakout_Detected', 'Breakdown_Detected', 'Bullish_Candlestick_Detected',
            'Bearish_Candlestick_Detected', 'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
            'name', 'interval', 'Currency',
            'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
            'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price'
        ]

        # The backtester expects these columns to be pre-calculated and provided in the input data.
        # Strategies generate signals based on these columns, and their values at the time of
        # entry and exit are recorded in the completed_trades DataFrame.


        # Perform a relaxed check: log a warning if potential entry/exit columns from data are missing
        missing_data_cols = [col for col in entry_exit_data_columns_expected if col not in data.columns]
        if missing_data_cols:
            logger.warning(f"Input data is missing expected indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records. Ensure your data preparation includes these columns if strategies or analysis depend on them.")


        # Ensure mandatory required columns are present
        missing_required = [col for col in required_columns if col not in data.columns]
        if missing_required:
            raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")


        # Ensure timestamp is datetime and sorted
        try:
            if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
                data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', utc=True) # Convert to UTC
            # Drop rows where timestamp conversion failed
            data = data.dropna(subset=['timestamp'])
            # Sort by timestamp and then instrument_key to process bars chronologically per instrument
            self.data = data.sort_values(by=['timestamp', 'instrument_key']).reset_index(drop=True)
        except Exception as e:
            raise ValueError(f"Error processing timestamp column in data: {e}")


        self.instrument_keys = instrument_keys
        self.active_strategies_instances = active_strategies_instances
        self.initial_capital = initial_capital

        # --- Backtesting State Variables ---
        self.current_capital = initial_capital
        self.positions = {}  # Dictionary to track open positions {instrument_key: {...entry details...}}
        self.completed_trades = [] # List to store completed trades
        self.trade_id_counter = 0 # Simple counter for trade IDs
        self.debug_log = [] # List to store debug information

        # Debug lists to capture values
        self._debug_timestamps = []
        self._debug_close_values = []
        self._debug_validity = []

        # Simple Slippage and Commission model (can be customized)
        self.slippage_pct = 0.001  # 0.1% slippage per trade
        self.commission_per_trade = 0.01 # $0.01 fixed commission per trade


        logger.info(f"BacktesterV2 initialized with {len(self.instrument_keys)} instruments and {len(self.active_strategies_instances)} active strategies.")
        logger.info(f"Initial Capital: {self.initial_capital}")
        logger.info(f"Data shape for backtesting: {self.data.shape}")


    def generate_trade_id(self, timestamp: datetime):
        """Generates a unique trade ID using a provided timestamp."""
        # Using microseconds to increase the chance of uniqueness
        return timestamp.strftime('%Y%m%d%H%M%S%f')


    def execute_trade(self, trade_id: str, instrument_key: str, timestamp: datetime, signal: str, strategy_name: str, price: float, data_point: pd.Series):
        """
        Simulates executing a trade based on a signal.

        Args:
            trade_id: Unique identifier for the trade.
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the trade execution (bar close time).
            signal: The trading signal ('BUY' or 'SELL').
            strategy_name: The name of the strategy generating the signal.
            price: The execution price (typically the close price of the bar).
            data_point: The pandas Series representing the data row for this bar. This Series
                        is expected to contain pre-calculated indicators and pattern data
                        used for entry/exit conditions and recording.
        """
        # Determine instrument type to handle lot size/quantity logic
        # instrument_type = get_instrument_type(instrument_key) # Removed due to import error
        instrument_type = 'Unknown' # Placeholder


        # Simple fixed quantity logic (can be replaced with dynamic position sizing)
        quantity_to_trade = 1 # Example: trade 1 unit/lot

        if signal == 'BUY':
            # Check if we already have a position in this instrument (optional, depending on strategy)
            if instrument_key not in self.positions:
                # Simulate buying
                cost = quantity_to_trade * price
                # Check if we have enough capital
                if self.current_capital >= cost:
                    self.current_capital -= cost

                    # Calculate entry costs (slippage and commission on entry)
                    entry_slippage = cost * self.slippage_pct
                    entry_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (entry_slippage + entry_commission) # Deduct costs from capital

                    # Capture entry-specific details from the data_point and other variables
                    self.positions[instrument_key] = {
                        'quantity': quantity_to_trade,
                        'entry_price': price, # This is the execution price for this simple model
                        'entry_time': timestamp,
                        'strategy': strategy_name,
                        'trade_id': trade_id,
                        'instrument_type': instrument_type, # Store instrument type (now placeholder)
                        'side': 'BUY', # Store trade side

                        # --- Entry-Specific Columns (Populated from data_point at Entry) ---
                        'Strategy_name': strategy_name,
                        'instrument_key': instrument_key,
                        'name': data_point.get('name'), # Use .get() to avoid errors if column is missing
                        'interval': data_point.get('interval'),
                        'Position_type': 'Long', # Assuming BUY means Long position
                        'Entry_order_type': 'Market', # Assuming market order execution on close
                        'Entry_timestamp': timestamp,
                        'Entry_price_trigger': None, # Not explicitly handled in this simple model
                        'Entry_price_execution': price,
                        'Entry_shares': quantity_to_trade, # Using quantity_to_trade as shares
                        'Entry_cost': cost, # Gross cost before fees
                        'Entry_signal_type': signal,
                        'Entry_Trend': data_point.get('Trend'), # Capture Trend at Entry
                        'Entry_SMA20': data_point.get('SMA20'), # Capture SMA20 at Entry
                        'Entry_RSI': data_point.get('RSI'), # Capture RSI at Entry
                        'Entry_RSI_MA': data_point.get('RSI_MA'), # Capture RSI_MA at Entry
                        'Entry_ATR': data_point.get('ATR'), # Capture ATR at Entry
                        'Entry_ADX': data_point.get('ADX'), # Capture ADX at Entry
                        'Entry_Volatility': data_point.get('Volatility'), # Capture Volatility at Entry
                        'Entry_Breakout_Detected': data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Entry
                        'Entry_Breakdown_Detected': data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Entry
                        'Entry_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Entry
                        'Entry_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Entry
                        'Entry_Bullish_Chart_Pattern_Name': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Entry
                        'Entry_Bearish_Chart_Pattern_Name': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Entry
                        'Instrument_Type': instrument_type, # Store instrument type (now placeholder)
                        'Currency': data_point.get('Currency'),
                        'Slippage_Entry': entry_slippage, # Store entry slippage
                        'Commission_Fees_Entry': entry_commission, # Store entry commission

                        # Placeholder for other entry-specific details that might be calculated by strategy (e.g., initial stop/target)
                        'Initial_Stop_Loss_Distance (%)': data_point.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': data_point.get('Risk_Amount'),
                        'Reward_Amount': data_point.get('Reward_Amount'),


                        # Placeholders for exit/other info that will be filled on close
                        # These fields are included here so the structure is consistent for retrieval on exit,
                        # even though their values are None at the time of entry.
                         'Max_Favorable_Excursion_MFE': None, # Will be calculated on exit
                         'Max_Adverse_Excursion_MAE': None, # Will be calculated on exit
                        'Current_trailing_stop': None, # Need logic for trailing stops
                        'Trailing_stop_method': None,
                        'Trailing_stop_value': None,
                        'Stop_loss_price': None,


                        'Exit_Trend': None, 'Exit_signal_type': None, 'Exit_SMA20': None,
                        'Exit_RSI': None, 'Exit_RSI_MA': None, 'Exit_ATR': None, 'Exit_ADX': None,
                        'Exit_Volatility': None, 'Exit_Breakout_Detected': None,
                        'Exit_Breakdown_Detected': None, 'Exit_Bullish_Candlestick_Name': None,
                        'Exit_Bearish_Candlestick_Name': None, 'Exit_Bullish_Chart_Pattern_Detected': None,
                        'Exit_Bearish_Chart_Pattern_Detected': None, 'Exit_shares': None,
                        'Exit_cost': None, 'Exit_revenue': None, 'PnL_trade': None,
                        'Trade_type': None, 'Profit_loss': None, 'Exit_reason': None,
                        'Slippage': None, 'Commission_Fees': None, 'Trade_Duration': None,
                        'Exit_Order_Type': None
                    }

                    # --- Add debug logging for Entry columns here ---
                    logger.debug(f"DEBUG Entry Data Point for {instrument_key} at {timestamp}:")
                    debug_cols_to_check = [
                        'Trend', 'SMA20', 'RSI', 'RSI_MA', 'ATR', 'ADX', 'Volatility',
                        'Breakout_Detected', 'Breakdown_Detected', 'Bullish_Candlestick_Detected',
                        'Bearish_Candlestick_Detected', 'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
                        'Currency', 'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
                        'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE', 'Current_trailing_stop',
                        'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price', 'Exit_Trend',
                        'Exit_signal_type', 'Exit_SMA20', 'Exit_RSI', 'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX',
                        'Exit_Volatility', 'Exit_Breakout_Detected', 'Exit_Breakdown_Detected',
                        'Exit_Bullish_Candlestick_Name', 'Exit_Bearish_Candlestick_Name',
                        'Exit_Bullish_Chart_Pattern_Detected', 'Exit_Bearish_Chart_Pattern_Detected',
                        'Exit_cost'
                    ]
                    for col in debug_cols_to_check:
                         logger.debug(f"  {col}: {data_point.get(col, 'Column Not Found or None')}")
                    # --- End Debug Logging ---


                    logger.info(f"Executed BUY trade {trade_id} for {instrument_key} at {timestamp} @ {price} (Qty: {quantity_to_trade}). Costs: Slippage={entry_slippage:.4f}, Commission={entry_commission:.2f}. Capital left: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'BUY', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'slippage': entry_slippage, 'commission': entry_commission})
                else:
                    logger.warning(f"Insufficient capital ({self.current_capital:.2f}) to BUY {instrument_key} at {price} (Cost: {cost:.2f}). Skipping trade {trade_id}.")
                    self.debug_log.append({'type': 'SKIP_BUY_CAPITAL', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Insufficient Capital'})

            else:
                # Already in a position, maybe add to it or skip depending on strategy rules
                logger.debug(f"Skipping BUY signal for {instrument_key} at {timestamp}. Already in a position.")
                self.debug_log.append({'type': 'SKIP_BUY_POSITION', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Already in Position'})


        elif signal == 'SELL':
            # For backtesting, a 'SELL' signal usually means closing a long position or opening a short position
            # Let's assume 'SELL' means closing a long position if one exists for simplicity in this example.
            # For a shorting strategy, you'd need different logic.
            if instrument_key in self.positions and self.positions[instrument_key]['side'] == 'BUY':
                # Simulate selling to close a long position
                position = self.positions[instrument_key]
                quantity_to_sell = position['quantity']
                entry_price = position['entry_price']
                entry_time = position['entry_time']
                strategy_opened = position['strategy']
                open_trade_id = position['trade_id']


                revenue = quantity_to_sell * price
                self.current_capital += revenue

                # Calculate Profit/Loss (Gross PnL)
                gross_pnl = (price - entry_price) * quantity_to_sell # For long position

                # Calculate exit costs (slippage and commission on exit)
                exit_slippage = revenue * self.slippage_pct
                exit_commission = self.commission_per_trade # Fixed commission per trade


                self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

                # Calculate Net PnL
                total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
                total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
                pnl_trade = gross_pnl - total_slippage - total_commission

                # Calculate Trade Duration
                trade_duration = (timestamp - entry_time).total_seconds() if pd.notnull(timestamp) and pd.notnull(entry_time) else None

                # Record completed trade - Populate all desired columns
                trade_record = {
                    'open_trade_id': open_trade_id,
                    'close_trade_id': trade_id,
                    'instrument_key': instrument_key,
                    'instrument_type': position.get('instrument_type'), # Assuming this key is correct in position
                    'side': position.get('side'), # Side of the position being closed (BUY/LONG)
                    'quantity': quantity_to_sell, # Quantity closed
                    'entry_price': entry_price,
                    'entry_time': entry_time,
                    'exit_price': price,
                    'exit_time': timestamp,
                    'pnl': gross_pnl, # Keep gross PnL for reference
                    'strategy_opened': strategy_opened,
                    'strategy_closed': strategy_name, # Record which strategy/signal closed it

                    # --- Transfer Entry Details from Position ---
                    'Strategy_name': position.get('Strategy_name'),
                    'instrument_key': position.get('instrument_key'),
                    'name': position.get('name'),
                    'interval': position.get('interval'),
                    'Position_type': position.get('Position_type'),
                    'Entry_order_type': position.get('Entry_order_type'),
                    'Entry_timestamp': position.get('Entry_timestamp'),
                    'Entry_price_trigger': position.get('Entry_price_trigger'),
                    'Entry_price_execution': position.get('Entry_price_execution'),
                    'Entry_shares': position.get('Entry_shares'),
                    'Entry_cost': position.get('Entry_cost'),
                    'Entry_signal_type': position.get('Entry_signal_type'),
                    'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                    'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                    'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                    'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                    'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                    'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                    'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                    'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                    'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                    'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Detected'), # Get Bullish Candlestick Name/Flag at Entry from Position
                    'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Detected'), # Get Bearish Candlestick Name/Flag at Entry from Position
                    'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Detected_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                    'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Detected_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                    'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                    'Currency': position.get('Currency'), # Get Currency at Entry from Position
                    'Slippage_Entry': position.get('Slippage_Entry'), # Transfer entry slippage
                    'Commission_Fees_Entry': position.get('Commission_Fees_Entry'), # Transfer entry commission
                    'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                    'Risk_Amount': position.get('Risk_Amount'),
                    'Reward_Amount': position.get('Reward_Amount'),
                    'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                    'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),


                    # --- Exit-Specific Columns (Populated from data_point at Exit) ---
                    'Exit_Trend': data_point.get('Exit_Trend'), # Capture Trend at Exit
                    'Exit_signal_type': signal, # Signal that triggered the exit
                    'Exit_SMA20': data_point.get('Exit_SMA20'), # Capture SMA20 at Exit
                    'Exit_RSI': data_point.get('Exit_RSI'), # Capture RSI at Exit
                    'Exit_RSI_MA': data_point.get('Exit_RSI_MA'), # Capture RSI_MA at Exit
                    'Exit_ATR': data_point.get('Exit_ATR'), # Capture ATR at Exit
                    'Exit_ADX': data_point.get('Exit_ADX'), # Capture ADX at Exit
                    'Exit_Volatility': data_point.get('Exit_Volatility'), # Capture Volatility at Exit
                    'Exit_Breakout_Detected': data_point.get('Exit_Breakout_Detected'), # Capture Breakout_Detected at Exit
                    'Exit_Breakdown_Detected': data_point.get('Exit_Breakdown_Detected'), # Capture Breakdown_Detected at Exit
                    'Exit_Bullish_Candlestick_Name': data_point.get('Exit_Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Exit
                    'Exit_Bearish_Candlestick_Name': data_point.get('Exit_Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Exit
                    'Exit_Bullish_Chart_Pattern_Detected': data_point.get('Exit_Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Exit
                    'Exit_Bearish_Chart_Pattern_Detected': data_point.get('Exit_Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Exit
                    'Exit_shares': quantity_to_sell, # Shares exited
                    'Exit_cost': 0, # Assuming no cost to exit a long position
                    'Exit_revenue': revenue, # Gross revenue before fees
                    'PnL_trade': pnl_trade, # Net PnL after costs
                    'Trade_type': 'Long Close (Signal)', # Or 'Long' if trade refers to the full cycle
                    'Profit_loss': pnl_trade, # Update Profit_loss to net PnL
                    'Exit_reason': f'Signal_{signal}', # Reason for exit
                    'Exit_Order_Type': 'Market', # Set Exit_Order_Type to 'Market'

                    # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                    'Current_trailing_stop': data_point.get('Current_trailing_stop'), # Capture if calculated and available
                    'Trailing_stop_method': data_point.get('Trailing_stop_method'), # Capture if calculated and available
                    'Trailing_stop_value': data_point.get('Trailing_stop_value'), # Capture if calculated and available
                    'Stop_loss_price': data_point.get('Stop_loss_price'), # Capture if calculated and available

                    'Slippage': total_slippage, # Total slippage for the round trip
                    'Commission_Fees': total_commission, # Total commission for the round trip
                    'Trade_Duration': trade_duration,

                }
                self.completed_trades.append(trade_record)


                # Remove position
                del self.positions[instrument_key]

                logger.info(f"Executed SELL trade {trade_id} for {instrument_key} at {timestamp} @ {price} to CLOSE long position {open_trade_id}. Gross PnL: {gross_pnl:.2f}. Costs: Slippage={total_slippage:.4f}, Commission={total_commission:.2f}. Net PnL: {pnl_trade:.2f}. Capital left: {self.current_capital:.2f}")
                self.debug_log.append({'type': 'SELL_CLOSE_LONG', 'open_trade_id': open_trade_id, 'close_trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_sell, 'gross_pnl': gross_pnl, 'net_pnl': pnl_trade, 'strategy_closed': strategy_name})


            else:
                # No matching long position to close, or maybe a shorting signal
                # For this simple backtester, we'll just log and skip if no long position
                logger.debug(f"Skipping SELL signal for {instrument_key} at {timestamp}. No matching long position to close.")
                self.debug_log.append({'type': 'SKIP_SELL_NO_LONG', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'strategy': strategy_name, 'reason': 'No Long Position'})


    def run_backtest(self):
        """
        Runs the backtesting simulation bar by bar through the data.
        """
        logger.info("Starting backtest simulation...")

        # Group data by timestamp first, then iterate through timestamps
        # This processes all instruments available at a given time before moving to the next time.
        grouped_by_time = self.data.groupby('timestamp')

        for timestamp, time_slice_df in grouped_by_time:
            # Process data for all instruments available at this timestamp
            for index, data_point in time_slice_df.iterrows():
                instrument_key = data_point['instrument_key']
                current_price = data_point['close'] # Assume close price for execution

                # Debug capture
                self._debug_timestamps.append(timestamp)
                self._debug_close_values.append(current_price)
                self._debug_validity.append(pd.notna(current_price))

                # Ensure current_price is valid for trading
                if pd.isna(current_price):
                    logger.debug(f"Skipping signal generation for {instrument_key} at {timestamp} due to invalid close price ({current_price}).")
                    self.debug_log.append({'type': 'SKIP_SIGNAL_PRICE_NAN', 'instrument': instrument_key, 'time': timestamp, 'reason': 'Invalid Price'})
                    continue # Skip this data point if price is invalid

                # Check for signals from all active strategies for this data point
                # Pass the single data_point (as a Series) to the strategy
                # The strategy is expected to handle a single row/Series or convert it internally
                # For the backtester's execute_trade, we need the Series directly.

                for strategy_name, strategy_instance in self.active_strategies_instances.items():
                    try:
                        # Pass the single data_point Series to the strategy's generate_signal
                        # Ensure data_point is passed as a DataFrame slice if strategy expects DataFrame
                        signal = strategy_instance.generate_signal(pd.DataFrame([data_point]))
                        # Ensure signal is a string, handle potential None returns gracefully
                        signal = str(signal).upper() if signal is not None else 'HOLD'

                        if signal in ['BUY', 'SELL']:
                            # Generate a unique trade ID for this potential trade using the bar's timestamp
                            trade_id = self.generate_trade_id(timestamp) # Pass the historical timestamp
                            # Pass the original data_point Series to execute_trade
                            self.execute_trade(trade_id, instrument_key, timestamp, signal, strategy_name, current_price, data_point)

                    except Exception as e:
                        logger.error(f"Error generating signal for {instrument_key} at {timestamp} using strategy '{strategy_name}': {e}", exc_info=True)
                        self.debug_log.append({'type': 'STRATEGY_ERROR', 'instrument': instrument_key, 'time': timestamp, 'strategy': strategy_name, 'error': str(e)})


        # After iterating through all data, close any remaining open positions
        self.close_all_positions(self.data['timestamp'].max()) # Use the timestamp of the last data point as exit time

        logger.info("Backtest simulation completed.")
        logger.info(f"Final Capital: {self.current_capital:.2f}")
        logger.info(f"Number of completed trades: {len(self.completed_trades)}")
        logger.info(f"Number of open positions remaining: {len(self.positions)}")

        # Return completed trades as a DataFrame for analysis
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

            # Ensure numeric columns are numeric
            numeric_cols = [
                'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
            ]
            for col in numeric_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')


            return completed_trades_df
        else:
            logger.warning("No completed trades recorded. Returning empty DataFrame.")
            return pd.DataFrame() # Return empty DataFrame if no trades


    def close_all_positions(self, exit_timestamp: datetime):
        """
        Closes all remaining open positions at the specified exit timestamp.
        Assumes closing at the price of the last available bar for each instrument.
        """
        logger.info(f"Closing all remaining {len(self.positions)} open positions at {exit_timestamp}...")

        # Get the last known price and data point for each instrument with an open position
        last_data_points = self.data.groupby('instrument_key').tail(1).set_index('instrument_key')
        last_prices = last_data_points['close'].to_dict()


        positions_to_close = list(self.positions.keys()) # Iterate over a copy

        for instrument_key in positions_to_close:
            if instrument_key in self.positions: # Check if position still exists (wasn't closed by a signal just before the end)
                position = self.positions[instrument_key]
                closing_price = last_prices.get(instrument_key, np.nan) # Get last price, default to NaN if instrument not found

                # Get the last data point for the instrument to capture exit conditions
                last_data_point = last_data_points.get(instrument_key, pd.Series({})) # Use empty Series if no data found


                if pd.notna(closing_price):
                    trade_id = self.generate_trade_id(exit_timestamp) # Generate a new trade ID for the closing trade using exit_timestamp
                    # Simulate selling to close a long position
                    quantity_to_sell = position['quantity']
                    entry_price = position['entry_price']
                    entry_time = position['entry_time']
                    strategy_opened = position['strategy']
                    open_trade_id = position['trade_id']


                    revenue = quantity_to_sell * closing_price
                    self.current_capital += revenue

                    # Calculate Profit/Loss (Gross PnL)
                    gross_pnl = (closing_price - entry_price) * quantity_to_sell # For long position

                    # Calculate exit costs (slippage and commission on exit)
                    exit_slippage = revenue * self.slippage_pct
                    exit_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

                    # Calculate Net PnL
                    total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
                    total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
                    pnl_trade = gross_pnl - total_slippage - total_commission


                    # Calculate Trade Duration
                    trade_duration = (exit_timestamp - entry_time).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(entry_time) else None


                    # Record completed trade - Populate all desired columns
                    trade_record = {
                        'open_trade_id': open_trade_id,
                        'close_trade_id': trade_id, # New ID for closing
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'), # Use .get() for safety
                        'side': position.get('side'), # Side of the position being closed (BUY/LONG)
                        'quantity': quantity_to_sell, # Quantity closed
                        'entry_price': entry_price,
                        'entry_time': entry_time,
                        'exit_price': closing_price,
                        'exit_time': exit_timestamp, # Use the provided exit timestamp
                        'pnl': gross_pnl, # Keep gross PnL for reference
                        'strategy_opened': strategy_opened,
                        'strategy_closed': 'Backtester_Forced_Close', # Indicate it was closed by the backtester

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': position.get('Slippage_Entry'), # Transfer entry slippage
                        'Commission_Fees_Entry': position.get('Commission_Fees_Entry'), # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),


                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Exit_Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close', # Indicate forced close
                        'Exit_SMA20': last_data_point.get('Exit_SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('Exit_RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('Exit_RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('Exit_ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('Exit_ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Exit_Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Exit_Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Exit_Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Exit_Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Exit_Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Exit_Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Exit_Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': quantity_to_sell, # Shares exited
                        'Exit_cost': 0, # Assuming no cost to exit a long position
                        'Exit_revenue': revenue, # Gross revenue before fees
                        'PnL_trade': pnl_trade, # Net PnL after costs
                        'Trade_type': 'Long Close (Forced)', # Or 'Long' if trade refers to the full cycle
                        'Profit_loss': pnl_trade, # Update Profit_loss to net PnL
                        'Exit_reason': 'Backtester_Forced_Close', # Reason for exit
                        'Exit_Order_Type': 'Market', # Set Exit_Order_Type to 'Market' or 'Forced'

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': last_data_point.get('Current_trailing_stop'), # Capture if calculated and available
                        'Trailing_stop_method': last_data_point.get('Trailing_stop_method'), # Capture if calculated and available
                        'Trailing_stop_value': last_data_point.get('Trailing_stop_value'), # Capture if calculated and available
                        'Stop_loss_price': last_data_point.get('Stop_loss_price'), # Capture if calculated and available


                        'Slippage': total_slippage, # Total slippage for the round trip
                        'Commission_Fees': total_commission, # Total commission for the round trip
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(trade_record)

                    # Remove position
                    del self.positions[instrument_key]

                    logger.info(f"Closed remaining position {open_trade_id} for {instrument_key} at {exit_timestamp} @ {closing_price}. Gross PnL: {gross_pnl:.2f}. Costs: Slippage={total_slippage:.4f}, Commission={total_commission:.2f}. Net PnL: {pnl_trade:.2f}. Capital: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'FORCED_CLOSE', 'open_trade_id': open_trade_id, 'close_trade_id': trade_id, 'instrument': instrument_key, 'time': exit_timestamp, 'price': closing_price, 'quantity': quantity_to_sell, 'gross_pnl': gross_pnl, 'net_pnl': pnl_trade, 'strategy_closed': 'Backtester_Forced_Close'})

                else:
                    logger.warning(f"Could not find last price for {instrument_key}. Cannot close position {position['trade_id']}. Logging as unresolved.")
                    # Log as an unresolved position or assume zero PnL

                    # Calculate Trade Duration even if closing price is NaN
                    trade_duration = (exit_timestamp - position.get('entry_time')).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(position.get('entry_time')) else None

                    # Transfer known costs
                    entry_slippage = position.get('Slippage_Entry', 0)
                    entry_commission = position.get('Commission_Fees_Entry', 0)


                    unresolved_trade_record = {
                        'open_trade_id': position.get('trade_id'),
                        'close_trade_id': None, # No closing trade ID
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'),
                        'side': position.get('side'),
                        'quantity': position.get('quantity'),
                        'entry_price': position.get('entry_price'),
                        'entry_time': position.get('entry_time'),
                        'exit_price': None, # No exit price
                        'exit_time': exit_timestamp, # Use the requested exit timestamp
                        'pnl': 0, # Assume zero Gross PnL if cannot close
                        'strategy_opened': position.get('strategy'),
                        'strategy_closed': 'Backtester_Forced_Close_Error', # Indicate error

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': entry_slippage, # Transfer entry slippage
                        'Commission_Fees_Entry': entry_commission, # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),


                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close_Error', # Indicate forced close error
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': position.get('quantity'), # Shares that were supposed to be exited
                        'Exit_cost': 0,
                        'Exit_revenue': 0, # Assuming zero revenue if cannot close
                        'PnL_trade': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Profit_loss': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Exit_reason': 'Backtester_Forced_Close_Error: No_Last_Price', # Reason for exit
                        'Exit_Order_Type': None, # Could not execute exit order

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': last_data_point.get('Current_trailing_stop'), # Capture if calculated and available
                        'Trailing_stop_method': last_data_point.get('Trailing_stop_method'), # Capture if calculated and available
                        'Trailing_stop_value': last_data_point.get('Trailing_stop_value'), # Capture if calculated and available
                        'Stop_loss_price': last_data_point.get('Stop_loss_price'), # Capture if calculated and available

                        'Slippage': entry_slippage, # Only entry slippage is known
                        'Commission_Fees': entry_commission, # Only entry commission is known
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(unresolved_trade_record)

                    # Remove position even if it couldn't be closed properly to prevent it from being processed again
                    del self.positions[instrument_key]


                    self.debug_log.append({'type': 'FORCED_CLOSE_ERROR', 'open_trade_id': position['trade_id'], 'instrument': instrument_key, 'time': exit_timestamp, 'reason': 'Last price not available'})


        logger.info("All remaining positions closed.")


    def analyze_backtest_results(self):
        """
        Analyzes the completed trades and provides performance metrics.
        Returns a DataFrame summarizing the analysis.
        """
        logger.info("Analyzing backtest results...")

        if not self.completed_trades:
            logger.warning("No completed trades to analyze.")
            return pd.DataFrame({'Message': ['No completed trades to analyze.']})

        # 1. Access the self.completed_trades list and Create a pandas DataFrame
        trades_df = pd.DataFrame(self.completed_trades)

        # 3. Ensure that relevant columns are converted to appropriate numeric types
        numeric_cols = [
            'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
            'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
            'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
            'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
            'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
            'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
            'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
            'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Include other potentially numeric cols
        ]
        for col in numeric_cols:
            if col in trades_df.columns:
                trades_df[col] = pd.to_numeric(trades_df[col], errors='coerce')

        # Handle potential NaN values during conversion - drop rows where PnL_trade (or pnl) is NaN
        pnl_col_for_analysis = 'PnL_trade' if 'PnL_trade' in trades_df.columns else 'pnl'
        if pnl_col_for_analysis in trades_df.columns:
            # Only consider trades with a valid PnL for core analysis metrics
            trades_df_analysis = trades_df.dropna(subset=[pnl_col_for_analysis]).copy() # Use .copy() to avoid SettingWithCopyWarning
        else:
            logger.warning("Neither 'PnL_trade' nor 'pnl' column found for analysis.")
            return pd.DataFrame({'Message': ['No PnL column found for analysis.']})


        if trades_df_analysis.empty:
            logger.warning("No valid trades after numeric conversion/dropna for analysis. Analysis stopped.")
            return pd.DataFrame({'Message': ['No valid trades after numeric conversion/dropna for analysis.']})


        # 4. Update the calculation of basic performance metrics using 'PnL_trade'
        total_trades = len(trades_df_analysis)
        total_pnl = trades_df_analysis[pnl_col_for_analysis].sum()

        winning_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] > 0]
        losing_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] < 0]
        breakeven_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] == 0]

        num_winning = len(winning_trades)
        num_losing = len(losing_trades)
        num_breakeven = len(breakeven_trades)

        win_rate = (num_winning / total_trades) * 100 if total_trades > 0 else 0
        avg_win = winning_trades[pnl_col_for_analysis].mean() if num_winning > 0 else 0
        avg_loss = losing_trades[pnl_col_for_analysis].mean() if num_losing > 0 else 0
        expectancy = (win_rate / 100) * avg_win + ((100 - win_rate) / 100) * avg_loss if total_trades > 0 else 0

        # 5. Update Max Drawdown calculation to use 'PnL_trade' and sort by exit time
        # Calculate cumulative PnL and then cumulative capital
        trades_df_analysis = trades_df_analysis.sort_values(by='exit_time') # Sort by exit time for cumulative calculation

        trades_df_analysis['cumulative_pnl'] = trades_df_analysis[pnl_col_for_analysis].cumsum()

        # Add initial capital to cumulative PnL
        trades_df_analysis['cumulative_capital'] = self.initial_capital + trades_df_analysis['cumulative_pnl']

        # Calculate peak capital up to each point
        trades_df_analysis['peak_capital'] = trades_df_analysis['cumulative_capital'].cummax()

        # Calculate drawdown at each point
        trades_df_analysis['drawdown'] = trades_df_analysis['peak_capital'] - trades_df_analysis['cumulative_capital']

        # Calculate percentage drawdown
        # Avoid division by zero if peak_capital is 0 or None
        trades_df_analysis['pct_drawdown'] = trades_df_analysis.apply(
            lambda row: (row['drawdown'] / row['peak_capital']) * 100 if row['peak_capital'] > 0 and pd.notna(row['peak_capital']) else 0,
            axis=1
        )


        max_drawdown_amount = trades_df_analysis['drawdown'].max() if not trades_df_analysis.empty else 0
        max_drawdown_pct = trades_df_analysis['pct_drawdown'].max() if not trades_df_analysis.empty else 0


        # 6. Update analysis summary metric names
        analysis_summary = {
            'Metric': [
                'Initial Capital',
                'Final Capital',
                'Total PnL (Net)', # Indicate Net PnL
                'Total Trades',
                'Winning Trades (Net)', # Indicate Net PnL
                'Losing Trades (Net)', # Indicate Net PnL
                'Breakeven Trades (Net)', # Indicate Net PnL
                'Win Rate (%) (Net PnL)', # Indicate Net PnL
                'Average Win (Net)', # Indicate Net PnL
                'Average Loss (Net)', # Indicate Net PnL
                'Expectancy per Trade (Net)', # Indicate Net PnL
                'Max Drawdown (Amount)',
                'Max Drawdown (%)',
            ],
            'Value': [
                self.initial_capital,
                self.current_capital,
                round(total_pnl, 2), # Format to 2 decimal places
                total_trades,
                num_winning,
                num_losing,
                num_breakeven,
                round(win_rate, 2), # Format to 2 decimal places
                round(avg_win, 2),
                round(avg_loss, 2),
                round(expectancy, 2),
                round(max_drawdown_amount, 2),
                round(max_drawdown_pct, 2),
            ]
        }

        analysis_df = pd.DataFrame(analysis_summary)

        logger.info("Backtest analysis completed.")
        # You can print the analysis_df here or return it
        # print("\n--- Backtest Analysis Summary ---")
        # display(analysis_df) # Use display for notebooks

        # 8. Ensure the method returns the updated analysis summary DataFrame
        return analysis_df

    def get_completed_trades(self):
        """Returns a DataFrame of completed trades."""
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

                    # Ensure numeric columns are numeric
                    numeric_cols = [
                        'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                        'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                        'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                        'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                        'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                        'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                        'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                        'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                        'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
                    ]
                    for col in numeric_cols:
                        if col in completed_trades_df.columns:
                            completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')

                    return completed_trades_df
                else:
                    return pd.DataFrame() # Return empty DataFrame if no trades

In [1]:
#_1113_mainV2.py

import asyncio
import nest_asyncio
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from pprint import pprint
import logging
from IPython.display import display

import websockets
import os
import pytz # Import the pytz library for timezone handling
import datetime # Import datetime for timestamp conversions
import json # Import json for potentially logging raw message content if not protobuf
import json # Import the json library
import talib

from _1111_1_FnoINELV1 import header_df
from _1111_1_market_utils import is_market_open, get_exchange_status  # Import functions
from _1111_historical_data_retrievalV1 import get_historical_data
from _1111_intraday_data_retrievalV1 import fetch_intraday_data
from _1111_web_1clean_and_transform import clean_and_transform, clean_data
from _1111_web_4combine_dataV2 import combine_data
import MarketDataFeedV3_pb2 as pb2 # Import your generated protobuf file
from _1111_websocket_connection3 import connectwebsocket
from _1111_web_5socket_connectionresample import RealTimeResampler # Assuming RealTimeResampler is in this file for modification
from _1111_web_3clean_and_transform_incremental import IncrementalDataTransformer
from _1113_5Trendline import TrendlineDetector
from _1113_1BullishCandlestickPatternv1 import BullishCandlestickPatterns
from _1113_2BearishCandlestickPatternv1 import BearishCandlestickPatterns
from _1113_3BullishChartPatternsV1 import BullishChartPatterns
from _1113_4BearishChartPatternsV1 import BearishChartPatterns
from _1113_6BacktesterV6 import BacktesterV3
from _1113_7strategy_CombinedSignal import CombinedSignalStrategy
from _1113_6strategy_TrendFollowing import TrendFollowingStrategy
from _1113_6strategy_TrendFollowingV1 import TrendFollowingStrategyV1


# from _1113_5Trendline import TrendlineDetector
# from _1113_1BullishCandlestickPattern import BullishCandlestickPatterns
# from _1113_2BearishCandlestickPattern import BearishCandlestickPatterns
# from _1113_3BullishChartPatterns import BullishChartPatterns
# from _1113_4BearishChartPatterns import BearishChartPatterns
# from _1113_6Backtester import Backtester


# Import technical indicator functions - ensure this module exists and is accessible
try:
    from _1112_technical_indicatorsV2 import (calculate_RSI_MA,
        calculate_SMA20, calculate_RSI, calculate_RSI_MA, calculate_EMA,
        calculate_pivot_points, calculate_MACD, calculate_ATR, calculate_ADX,
        calculate_BollingerBands, calculate_MA, calculate_volatility,
        calculate_VWAP, calculate_WMA, calculate_FibonacciRetracement, calculate_MACD_pandas_ta

    )
    print("Successfully imported technical indicator functions.")
except ImportError:
    print("Warning: Could not import technical indicator functions from _1112_technical_indicators.")
    # Define placeholder functions or handle this error appropriately if needed
    def calculate_SMA20(df, window=20): return pd.Series(index=df.index)
    def calculate_RSI(series, period=14): return pd.Series(index=series.index)
    def calculate_RSI_MA(series, rsi_length=14, ma_length=14): return pd.Series(index=series.index)
    def calculate_EMA(df, window=20): return pd.Series(index=df.index)
    def calculate_pivot_points(df): return {'Pivot': pd.Series(index=df.index), 'S1': pd.Series(index=df.index), 'S2': pd.Series(index=df.index),'S3': pd.Series(index=df.index), 'R1': pd.Series(index=df.index), 'R2': pd.Series(index=df.index), 'R3': pd.Series(index=df.index)}
    def calculate_MACD(df, fastperiod=12, slowperiod=26, signalperiod=9): return (pd.Series(index=df.index), pd.Series(index=df.index), pd.Series(index=df.index))
    def calculate_ATR(df, timeperiod=14): return pd.Series(index=df.index)
    def calculate_ADX(df, timeperiod=14): return pd.Series(index=df.index)
    def calculate_BollingerBands(df, window=20, num_std_dev=2): return (pd.Series(index=df.index), pd.Series(index=df.index), pd.Series(index=df.index))
    def calculate_MA(df, window=20): return pd.Series(index=df.index)
    def calculate_volatility(df, window=20): return pd.Series(index=df.index)
    def calculate_VWAP(df): return pd.Series(index=df.index)
    def calculate_WMA(df, window=20): return pd.Series(index=df.index)
    def calculate_FibonacciRetracement(high, low): return {} # Return empty dict or similar placeholder






# Set up logging
logging.basicConfig(level=logging.INFO) # Configure logging format and handlers
logging.basicConfig(
    level=logging.INFO,  # Set the minimum logging level
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Define the log message format
    handlers=[
        logging.StreamHandler(),  # Log to the console
        # logging.FileHandler('app.log')  # Log to a file (optional)
    ])

logger = logging.getLogger(__name__)

nest_asyncio.apply()
loop = asyncio.get_event_loop() # Get the event loop globally

# Create a global asyncio.Event object for signaling shutdown
shutdown_event = asyncio.Event()

print("Shutdown event created.")


# Extract instrument keys from header_df
instrument_keys = header_df['instrument_key'].unique().tolist()


# Global variables
merged_data = {}
paper_trades = []
completed_trades = [] # Initialize globally
combined_df = None  # Define combined_df outside the function
# Create an instance of TrendlineDetector
trendline_detector = TrendlineDetector(combined_df)




# *** PLACE THE HELPER FUNCTION DEFINITIONS HERE ***
# These functions should be defined *before* the async def main(): block

def get_instrument_type(instrument_key):
    """Determines instrument type based on key."""
    # Example implementation - Adapt based on your instrument key format or header_df
    if pd.isna(instrument_key): return 'Unknown'
    if 'NSE_EQ' in str(instrument_key): return 'Equity'
    if 'NSE_FUT' in str(instrument_key): return 'Futures'
    if 'NSE_OPT' in str(instrument_key): return 'Options'
    if 'NSE_INDEX' in str(instrument_key): return 'Index'
    return 'Unknown' # Default


def get_instrument_currency(instrument_key):
    """Determines currency based on key."""
    if pd.isna(instrument_key):
        return 'Unknown'
    instrument_key_upper = str(instrument_key).upper()

    if 'NSE_CUR' in instrument_key_upper or 'BSE_CUR' in instrument_key_upper:
        # Currency derivatives might specify currency pairs, but generally traded in INR
        # You might need more complex logic here if dealing with specific currency pairs
        return 'INR' # Currency derivatives are settled in INR in India
    elif 'MCX' in instrument_key_upper or 'NSE_COM' in instrument_key_upper:
        return 'INR' # Commodities in India are traded in INR
    elif 'NSE_' in instrument_key_upper or 'BSE_' in instrument_key_upper:
        # Assuming all other NSE/BSE instruments (Equity, F&O, Index) are INR
        return 'INR'
    # Add logic for other currencies if you trade international instruments

    return 'Unknown' # Default

def determine_entry_order_type(signal):
    """Determines the intended order type based on the signal."""
    # Example implementation - Adapt based on your strategy's execution logic
    if signal in ['BUY', 'SELL']:
        return 'Market' # Assume market execution for entry signals
    # Add other conditions if your strategy uses limit or stop orders for entry
    return 'N/A_Signal' # Or 'HOLD_Signal' etc.










async def main():

    global merged_data, paper_trades, combined_df, completed_trades # Add completed_trades here
    combined_df = pd.DataFrame()  # Initialize as an empty DataFrame




    # Read access token
    with open("accessToken.txt", "r") as file:
        access_token = file.read().strip()

    # Get instrument details
    # Get instrument details
    input_data = header_df[['instrument_key', 'name', 'expiry', 'lot_size', 'max_lot_size']]
    instrument_keys = input_data['instrument_key'].unique().tolist()  # Use unique() to get unique keys

    # Time intervals for historical data
    intervals = ["1minute", "day"]

    # Trading Mode (choose one: 'backtest', 'paper', 'live')
    # trading_mode =  'paper', 'live' # <--- This is a tuple, should be a single string
    trading_mode = 'backtest' # 'paper'   # 'backtest' # 'paper'  Or 'paper' or 'live' or 'backtest'

    print("Current Trading Mode:", trading_mode)


    # 1. Fetch Historical Data
    # Fetch historical data only if needed (e.g., for initial combined_df or backtesting)
    historical_data = await get_historical_data(instrument_keys)

    # 2. Clean and Transform Historical Data
    try:
        historical_data_future = loop.run_in_executor(None, clean_and_transform, historical_data, True)
        cleaned_historical_data = await historical_data_future
    except Exception as e:
        logger.error(f"Error during clean_and_transform for historical data: {e}")
        cleaned_historical_data = pd.DataFrame()  # Assign empty DataFrame if error


    if isinstance(cleaned_historical_data, pd.DataFrame):
        logger.info(f"main: Shape of cleaned_historical_data after clean_and_transform: {cleaned_historical_data.shape}")
        if not cleaned_historical_data.empty:
            logger.info(f"main: Columns in cleaned_historical_data: {cleaned_historical_data.columns.tolist()}")
            if 'close' in cleaned_historical_data.columns:
                logger.info(f"main: 'close' column dtype in cleaned_historical_data: {cleaned_historical_data['close'].dtype}")
                logger.info(f"main: Count of non-NaN 'close' values in cleaned_historical_data: {cleaned_historical_data['close'].count()}")
            else:
                logger.info(f"main: 'close' column NOT PRESENT in cleaned_historical_data.")
    else:
        logger.warning(f"main: cleaned_historical_data is not a DataFrame after clean_and_transform. Type: {type(cleaned_historical_data)}")

    # 3. Fetch Intraday Data (Needed for backtesting on recent data or initial paper/live context)
    intraday_data = await fetch_intraday_data(instrument_keys)

    # 4. Clean and Transform Intraday Data (if available)
    if intraday_data is not None and not intraday_data.empty:
        try:
            intraday_data_future = loop.run_in_executor(None, clean_and_transform, intraday_data)
            cleaned_intraday_data = await intraday_data_future
        except Exception as e:
            logger.error(f"Error during clean_and_transform for intraday data: {e}")
            cleaned_intraday_data = pd.DataFrame()  # Assign empty DataFrame if error
    else:
        cleaned_intraday_data = pd.DataFrame(columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'oi', 'instrument_key', 'name', 'interval'])


    # 5. Combine Data (Needed for backtesting or providing initial context for paper/live)
    print("Before combine_data")
    logger.info("Before combine_data execution")
    combine_future = loop.run_in_executor(None, combine_data, cleaned_historical_data, cleaned_intraday_data if not cleaned_intraday_data.empty else None, instrument_keys)
    try:
        merged_data = await combine_future
        combined_df = merged_data
        if combined_df is not None and isinstance(combined_df, pd.DataFrame) and combined_df.index.name == 'timestamp': # Check if timestamp is the index and combined_df is valid
            combined_df = combined_df.reset_index() # Make it a column

#####*************************************************************

        # Now 'timestamp' will be a column if it was the index and combined_df is valid

        # Replaced prints with logger.debug/info
        logger.debug(f"Columns in combined_df after combine_data and potential reset_index: {combined_df.columns.tolist() if isinstance(combined_df, pd.DataFrame) else 'combined_df is None'}")
        if combined_df is not None:
            if 'timestamp' in combined_df.columns:
                logger.debug("'timestamp' column now exists in combined_df")
            else:
                logger.debug("'timestamp' column still does NOT exist in combined_df")

            logger.info(f"After combine_data, combined_df shape: {combined_df.shape}")

            # Display head (using display for notebooks) - Removed redundant print labels
            print("\nCombined DataFrame Head (first 10 rows):") # Optional print to label the display
            display(combined_df.head(10)) # Using display for Jupyter, showing 10 rows

            if 'interval' in combined_df.columns:
                # Replaced print with logger.debug
                logger.debug(f"Unique intervals in combined_df: {combined_df['interval'].unique()}")
            else:
                # Replaced print with logger.warning
                logger.warning("'interval' column not found in combined_df.")
        else:
             # Replaced print with logger.warning
             logger.warning("combined_df is None after combine_data.")


    except Exception as e:
        # Replaced print with logger.error
        logger.error(f"Error during combine_data: {e}", exc_info=True)

        combined_df = pd.DataFrame() # Ensure combined_df is defined on error


    #####****>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***
    #####****>>>>>>>>>>>>>>>>> ADD CODE HERE: Filter combined_df to create daily_data_df <<<<<<<<<<<***
    #####****>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***


    daily_data_df = pd.DataFrame() # Initialize daily_data_df
    if combined_df is not None and not combined_df.empty and 'interval' in combined_df.columns:
        try:
            daily_data_df = combined_df[combined_df['interval'] == 'day'].copy() # Filter for 'day' interval and make a copy
            if 'timestamp' in daily_data_df.columns:
                 daily_data_df['timestamp'] = pd.to_datetime(daily_data_df['timestamp'], errors='coerce')
                 daily_data_df = daily_data_df.dropna(subset=['timestamp'])
            print(f"Successfully filtered daily data from combined_df. Shape: {daily_data_df.shape}")
        except Exception as e:
            print(f"Error filtering daily data from combined_df: {e}")
            daily_data_df = pd.DataFrame()
    else:
        print("Warning: combined_df is not available, is empty, or missing 'interval' column. Cannot filter daily data.")
        daily_data_df = pd.DataFrame()

    # *** ADD CODE HERE: Calculate actual_backtest_duration_years from the FULL combined_df ***
    first_data_timestamp = None
    last_data_timestamp = None
    actual_backtest_duration_years = 0

    if combined_df is not None and not combined_df.empty:
        try:
            if isinstance(combined_df.index, pd.DatetimeIndex):
                first_data_timestamp = combined_df.index.min()
                last_data_timestamp = combined_df.index.max()
            elif 'timestamp' in combined_df.columns and pd.api.types.is_datetime64_any_dtype(combined_df['timestamp']):
                 first_data_timestamp = combined_df['timestamp'].min()
                 last_data_timestamp = combined_df['timestamp'].max()

            if pd.notna(first_data_timestamp) and pd.notna(last_data_timestamp) and last_data_timestamp > first_data_timestamp:
                 time_difference = last_data_timestamp - first_data_timestamp
                 actual_backtest_duration_years = time_difference.days / 365.25
                 print(f"Debug: Calculated actual backtest duration from combined_df: {time_difference} ({actual_backtest_duration_years:.2f} years)")
            else:
                 print("Warning: Calculated data duration from combined_df is zero or negative.")

        except Exception as e:
             print(f"Error determining actual backtest duration from combined_df: {e}")

    else:
        print("Warning: combined_df is not available or is empty, cannot calculate actual backtest duration.")


#####*****************************************************************************************#####
    # 6.  ADD THE PATTERN DETECTION AND PROCESSING SNIPPET HERE ---
    if isinstance(combined_df, pd.DataFrame) and not combined_df.empty: # Use isinstance check as well
        print("\nProceeding with pattern detection on combined_df...")
        # Assuming your data for pattern detection is now combined_df
        data_for_pattern_detection = combined_df.copy() # Make a copy to be safe

        processed_data_list = [] # List to collect DataFrames for each instrument after adding pattern columns

        # Group data by instrument
        for (instrument_key, name, interval), group_data in data_for_pattern_detection.groupby(['instrument_key', 'name', 'interval']):
            # print(f"Processing patterns for: {instrument_key}, {name}, {interval}")

            # --- Ensure group_data is a copy to avoid SettingWithCopyWarning ---
            group_data = group_data.copy()

            # --- Calculate Technical Indicators (including Trend) ---

            # Recalculate MA, HH, LL for group_data as these are needed for the trend logic
            window_size = 20 # Or your chosen window size for trend calculation

            # Ensure 'close' column exists before calculations
            # Add this print BEFORE the if statement
            # print(f"Debug Check: About to check for 'close' column in group_data for {instrument_key},{name}, {interval}. Columns available: {group_data.columns.tolist()}")

            if 'close' in group_data.columns:
                # This is the original line, we can keep it or add more specific prints
                # print(f"Calculating MA, HH, LL, and Trend for {instrument_key},{name} with window {window_size}")

                # Add more prints inside the if block if needed
                # print(f"Debug Check: 'close' column FOUND for {instrument_key},{name}, {interval}. Proceeding with MA, HH, LL, Trend calculation.")
                # You could also print the head of the close column here if you want to see values

                # Calculate rolling indicators
                group_data['MA'] = group_data['close'].rolling(window=window_size).mean()
                group_data['HH'] = group_data['close'].rolling(window=window_size).max()
                group_data['LL'] = group_data['close'].rolling(window=window_size).min()
                # --- Add debug prints after rolling calculations ---
                logger.debug(f"Debug Rolling: After MA, HH, LL calculation for {group_data.name}. Shape: {group_data.shape}")

                # Ensure 'Trend' column exists with a default value before applying the function
                group_data['Trend'] = 'Sideways'

                # Define the get_trend_for_row function (keep this inside the loop or define it outside)
                # Let's define it inside the loop for now for simplicity, it captures the loop's 'window_size'
                def get_trend_for_row(row, df_full_slice, current_window):
                    # Use the index within the current slice for relative positioning
                    try:
                        pos_idx_list = df_full_slice.index.get_indexer([row.name])
                        if len(pos_idx_list) == 0 or pos_idx_list[0] == -1:
                            # Should not happen if row is from df_full_slice, but safety check
                            return "Calculation Error"
                        pos_idx = pos_idx_list[0]


                        if pos_idx < current_window - 1:
                            return 'Sideways'

                        # Use iloc for positional indexing relative to the start of the slice
                        start_window_pos_idx = pos_idx - current_window + 1
                        # No need to check < 0 here due to pos_idx < current_window - 1 check above

                        # Get the start row of the window using positional index on the slice
                        start_window_row = df_full_slice.iloc[start_window_pos_idx]

                        # Safely access calculated columns from both the current row and the start-of-window row
                        # Access the already calculated MA, HH, LL for the current row and the start_window_row
                        current_ma = row.get('MA', np.nan)
                        current_hh = row.get('HH', np.nan)
                        current_ll = row.get('LL', np.nan)

                        start_ma = start_window_row.get('MA', np.nan)
                        start_hh = start_window_row.get('HH', np.nan)
                        start_ll = start_window_row.get('LL', np.nan)


                        # Check if calculated values are valid (not NaN or None) for comparison
                        if pd.isna(current_ma) or pd.isna(current_hh) or pd.isna(current_ll) or \
                        pd.isna(start_ma) or pd.isna(start_hh) or pd.isna(start_ll):
                            return "Sideways" # Cannot determine trend if any value is missing


                        # Determine trend based on conditions
                        # Compare current MA, HH, LL with those at the start of the rolling window
                        if current_ma > start_ma and \
                        current_hh > start_hh and \
                        current_ll > start_ll:
                            return 'Uptrend'
                        elif current_ma < start_ma and \
                            current_hh < start_hh and \
                            current_ll < start_ll:
                            return 'Downtrend'
                        else:
                            return 'Sideways'

                    except Exception as e:
                        # This catch is inside the apply function now
                        print(f"Error in get_trend_for_row at index {row.name} for {instrument_key},{name}, {interval}: {e}")
                        return "Calculation Error" # Return an error status for this specific row


                # Apply the trend logic only to rows where the window is fully available within this slice
                try: # Add a try block around the apply for specific error handling
                    if len(group_data) >= window_size: # Check for sufficient data length in the slice
                            group_data.loc[group_data.index[window_size - 1]:, 'Trend'] = group_data.iloc[window_size - 1:].apply(
                            lambda row: get_trend_for_row(row, group_data.iloc[window_size - 1:], window_size), axis=1 # Apply to the sliced part and pass the sliced part
                        )

                        # The rows before window_size - 1 remain 'Sideways' as initialized
                    else:
                        # If the total length of group_data is less than the window, all trends are 'Sideways' (already the default)
                        pass # No change needed, 'Trend' is already 'Sideways'
                    logger.debug(f"Debug Pattern Check: Successfully added 'Trend' column for {instrument_key},{name}, {interval} using rolling window logic.") # This logger.debug is good to keep for debugging
                    # print(f"Debug Pattern Check: Successfully added 'Trend' column for {instrument_key},{name}, {interval} using rolling window logic.")

                except Exception as apply_e:
                    print(f"Error applying trend calculation for {instrument_key},{name}, {interval}: {apply_e}")
                    # Add a default 'Trend' column with 'Calculation Error' if apply fails
                    group_data['Trend'] = 'Calculation Error' # Overwrite the entire column or just the remaining rows? Let's overwrite for clarity.
                    print(f"Debug Pattern Check: Added 'Trend' column with 'Calculation Error' due to error for {instrument_key},{name}, {interval}.")


            else:
                print(f"Warning: 'close' column not found for {instrument_key},{name}, skipping trend calculation.")
                # Ensure columns exist even if 'close' is missing
                if 'MA' not in group_data.columns: group_data['MA'] = None
                if 'HH' not in group_data.columns: group_data['HH'] = None
                if 'LL' not in group_data.columns: group_data['LL'] = None


            # Ensure the 'Trend' column exists after calculation attempts
            if 'Trend' not in group_data.columns:
                group_data['Trend'] = 'Sideways'
                logger.warning(f"Debug: 'Trend' column was missing after calculation attempts, added as 'Sideways' for {instrument_key},{name}.")
            else:
                logger.debug(f"Debug Trend: 'Trend' column found after calculation attempt for {group_data.name}.")





            # --- Calculate Support and Resistance Levels ---
            try:
                # Instantiate TrendlineDetector with group_data
                # ENSURE THIS LINE IS PRESENT AND EXECUTED BEFORE CALLING METHODS ON trend_detector
                trend_detector = TrendlineDetector(group_data.copy()) # Create a new instance here


                # Call the find_support_resistance method on the instance
                # Use the same window_size or a different one as appropriate
                support, resistance = trend_detector.find_support_resistance(window=window_size) # Using window_size here

                # Store these levels in new columns
                # Storing lists directly in DataFrame columns can sometimes be less performant
                # but for backtesting summary or occasional access it might be acceptable.
                # Ensure the column names are consistent with how you'll use them later.
                group_data['Support_Levels'] = [support] * len(group_data) # Store the same list for all rows
                group_data['Resistance_Levels'] = [resistance] * len(group_data) # Store the same list for all rows


                # print(f"Debug: Found Support: {support}, Resistance: {resistance} for {instrument_key},{name}, {interval}")

            except Exception as e:
                print(f"Error finding Support/Resistance for {instrument_key},{name}, {interval}: {e}")
                # Add default columns with empty lists on error
                group_data['Support_Levels'] = [[]] * len(group_data)
                group_data['Resistance_Levels'] = [[]] * len(group_data)


            # --- Detect Breakouts and Breakdowns ---
            try:
                # Reuse the 'trend_detector' instance initialized just above in the Support/Resistance block
                # trend_detector = TrendlineDetector(group_data.copy()) # Only create a new instance if not created above

                # Detect potential breakouts (price crossing above trendline in an assumed Uptrend context)
                # Note: The detect_breakout method calculates its own MA trendline.
                # You might want to refine this later to use the 'Trend' column or a different trendline.
                breakout_points = trend_detector.detect_breakout(trendline_type='Uptrend', window=window_size) # Use window_size or another window

                # Detect potential breakdowns (price crossing below trendline in an assumed Downtrend context)
                breakdown_points = trend_detector.detect_breakout(trendline_type='Downtrend', window=window_size) # Use window_size or another window

                # Store the breakout/breakdown information in boolean columns
                # Initialize boolean columns
                group_data['Breakout_Detected'] = False
                group_data['Breakdown_Detected'] = False

                # Mark the indices where breakouts/breakdowns were detected
                # Ensure breakout_points and breakdown_points are lists of indices compatible with .loc
                if breakout_points: # Check if list is not empty
                    group_data.loc[breakout_points, 'Breakout_Detected'] = True
                if breakdown_points: # Check if list is not empty
                     group_data.loc[breakdown_points, 'Breakdown_Detected'] = True


                print(f"Debug: Found {len(breakout_points)} Breakouts and {len(breakdown_points)} Breakdowns for {instrument_key},{name}, {interval}")

            except Exception as e:
                print(f"Error detecting Breakouts/Breakdowns for {instrument_key},{name}, {interval}: {e}")
                # Add default boolean columns on error
                group_data['Breakout_Detected'] = False
                group_data['Breakdown_Detected'] = False


            # --- Detect Triangle Pattern ---
            try:
                # Reuse the 'trend_detector' instance initialized earlier in the loop
                # trend_detector = TrendlineDetector(group_data.copy()) # Only create a new instance if not created above

                # Call the detect_triangle_pattern method on the instance
                # Use the same window_size or a different one as appropriate
                triangle_pattern = trend_detector.detect_triangle_pattern(window=window_size) # Using window_size here

                # Store the detected triangle pattern type in a new column
                # Ensure the column name is consistent with how you'll use it later.
                group_data['Triangle_Pattern'] = triangle_pattern

                print(f"Debug: Detected Triangle Pattern: '{triangle_pattern}' for {instrument_key},{name}, {interval}")

            except Exception as e:
                print(f"Error detecting Triangle Pattern for {instrument_key},{name}, {interval}: {e}")
                # Add a default column with 'Unknown' or None on error
                group_data['Triangle_Pattern'] = 'Unknown' # Add default string value
                print(f"Debug: Added 'Triangle_Pattern' column with 'Unknown' due to error for {instrument_key},{name}, {interval}.")




            # --- Chart Pattern Detectors - Instantiate here with group_data ---
            # Re-instantiate with the updated group_data that now has Trend, MA, HH, LL
            try:
                bullish_chart_pattern_detector = BullishChartPatterns(group_data)
                bearish_chart_pattern_detector = BearishChartPatterns(group_data)
                print(f"Debug Pattern Check: Chart Pattern Detectors re-instantiated with updated group_data for {instrument_key},{name}, {interval}.")
            except Exception as e:
                print(f"Error re-instantiating Chart Pattern Detectors for {instrument_key},{name}, {interval}: {e}")
                bullish_chart_pattern_detector = None # Set to None if re-instantiation fails
                bearish_chart_pattern_detector = None # Set to None if re-instantiation fails

            # --- CALL THE DETECT METHODS AND ADD THE RESULTS AS NEW COLUMNS ---
            # Chart Pattern Detection
            # Assuming these methods return a Series or list of pattern names, or None/NaN
            # Ensure the method names and return types match your actual Chart Pattern classes
            if bullish_chart_pattern_detector is not None and bearish_chart_pattern_detector is not None:
                try: # <--- Start of inner try block for detection calls
                    # Call the detect methods for chart patterns - Corrected method name call
                    # Ensure these methods operate on self.df (the group_data passed during instantiation)
                    # If they need specific data slices or arguments, adjust here
                    bullish_chart_names_series = bullish_chart_pattern_detector.detect_bullish_chart_patterns() # Assuming this returns a Series of names or None
                    bearish_chart_names_series = bearish_chart_pattern_detector.detect_bearish_chart_patterns() # Assuming this returns a Series of names or None


                    # Add Chart Pattern results as columns to group_data
                    # Ensure column names match exactly what your strategy expects ('Detected_Bullish_Chart_Pattern_Name' etc.)
                    # Check if the results are Series or lists and handle assignment appropriately
                    # Assign the Series directly. If a row has no pattern, the Series should have None/NaN for that index.
                    group_data['Detected_Bullish_Chart_Pattern_Name'] = bullish_chart_names_series
                    group_data['Detected_Bearish_Chart_Pattern_Name'] = bearish_chart_names_series
                    logger.debug(f"Debug Pattern Check: Successfully added Chart Pattern columns for {instrument_key},{name}, {interval}.")

                except AttributeError: # <--- except block aligned with try
                    print(f"Error: Chart Pattern Detector method not found for {instrument_key},{name}, {interval}. Check method name.") # More generic error message
                    # Add dummy columns if method call failed to prevent errors later
                    # Ensure column names match what the strategy expects
                    if 'Detected_Bullish_Chart_Pattern_Name' not in group_data.columns:
                        group_data['Detected_Bullish_Chart_Pattern_Name'] = None # Or False/'' depending on expected type
                    if 'Detected_Bearish_Chart_Pattern_Name' not in group_data.columns:
                        group_data['Detected_Bearish_Chart_Pattern_Name'] = None # Or False/''
                    logger.debug(f"Debug Pattern Check: Added dummy Chart Pattern columns due to AttributeError for {instrument_key},{name}, {interval}.")

                except Exception as e: # <--- except block aligned with try
                    print(f"Error detecting and adding chart patterns for {instrument_key},{name}, {interval}: {e}")
                    # Add dummy columns if detection failed to prevent errors later
                    # Ensure column names match what the strategy expects
                    if 'Detected_Bullish_Chart_Pattern_Name' not in group_data.columns:
                        group_data['Detected_Bullish_Chart_Pattern_Name'] = None # Or False/''
                    if 'Detected_Bearish_Chart_Pattern_Name' not in group_data.columns:
                        group_data['Detected_Bearish_Chart_Pattern_Name'] = None # Or False/''
                    logger.debug(f"Debug Pattern Check: Added dummy Chart Pattern columns due to error for {instrument_key},{name}, {interval}.")

            else: # <--- else block aligned with if
                logging.warning(f"Warning: Chart Pattern Detectors not instantiated or re-instantiated for {instrument_key},{name}, {interval}. Skipping chart pattern detection.")
                # Ensure dummy columns exist even if detectors weren't instantiated/re-instantiated
                if 'Detected_Bullish_Chart_Pattern_Name' not in group_data.columns:
                    group_data['Detected_Bullish_Chart_Pattern_Name'] = None # Or False/''
                if 'Detected_Bearish_Chart_Pattern_Name' not in group_data.columns:
                    group_data['Detected_Bearish_Chart_Pattern_Name'] = None # Or False/''
                logger.debug(f"Debug Pattern Check: Added dummy Chart Pattern columns due to not instantiated for {instrument_key},{name}, {interval}.")


            # Chart Pattern Detectors - Instantiate here with group_data
            try:
                # Re-instantiate if needed, or ensure the existing instances are updated with the latest group_data
                # If the detectors were instantiated with the initial group_data.copy() before trend calculation,
                # they might not have the 'Trend' column. It's better to re-instantiate or pass the updated group_data.
                bullish_chart_pattern_detector = BullishChartPatterns(group_data.copy()) # Pass updated group_data
                bearish_chart_pattern_detector = BearishChartPatterns(group_data.copy()) # Pass updated group_data
                logger.debug(f"Debug Pattern Check: Chart Pattern Detectors re-instantiated with updated group_data for {instrument_key},{name}, {interval}.")

            except Exception as e:
                logger.error(f"Error re-instantiating Chart Pattern Detectors for {instrument_key},{name}, {interval}: {e}")
                bullish_chart_pattern_detector = None # Set to None if instantiation fails
                bearish_chart_pattern_detector = None # Set to None if instantiation fails

            # --- CALL THE DETECT METHODS AND ADD THE RESULTS AS NEW COLUMNS ---
            # This part remains the same, operating on the potentially updated group_data
            if bullish_chart_pattern_detector is not None and bearish_chart_pattern_detector is not None:
                try: # <--- Start of inner try block
                    # Call the detect methods for chart patterns
                    bullish_chart_names_series = bullish_chart_pattern_detector.detect_bullish_chart_patterns() # Assuming this returns a Series of names or None
                    bearish_chart_names_series = bearish_chart_pattern_detector.detect_bearish_chart_patterns() # Assuming this returns a Series of names or None

                    # Add Chart Pattern results as columns to group_data
                    # Assign the Series directly. If a row has no pattern, the Series should have None/NaN for that index.
                    group_data['Detected_Bullish_Chart_Pattern_Name'] = bullish_chart_names_series
                    group_data['Detected_Bearish_Chart_Pattern_Name'] = bearish_chart_names_series
                    logger.debug(f"Debug Pattern Check: Successfully added Chart Pattern columns for {instrument_key},{name}, {interval}.")

                except AttributeError:
                    logger.error(f"Error: Chart Pattern Detector method not found for {instrument_key},{name}, {interval}. Check method name.")
                    # Add dummy columns if method call failed
                    if 'Detected_Bullish_Chart_Pattern_Name' not in group_data.columns:
                        group_data['Detected_Bullish_Chart_Pattern_Name'] = None
                    if 'Detected_Bearish_Chart_Pattern_Name' not in group_data.columns:
                        group_data['Detected_Bearish_Chart_Pattern_Name'] = None
                    logger.debug(f"Debug Pattern Check: Added dummy Chart Pattern columns due to AttributeError.")

                except Exception as e:
                     logger.error(f"Error detecting and adding chart patterns for {instrument_key},{name}, {interval}: {e}")
                     # Add dummy columns if detection failed
                     if 'Detected_Bullish_Chart_Pattern_Name' not in group_data.columns:
                         group_data['Detected_Bullish_Chart_Pattern_Name'] = None
                     if 'Detected_Bearish_Chart_Pattern_Name' not in group_data.columns:
                         group_data['Detected_Bearish_Chart_Pattern_Name'] = None
                     logger.debug(f"Debug Pattern Check: Added dummy Chart Pattern columns due to error.")

            else:
                 logger.warning(f"Warning: Chart Pattern Detectors not instantiated for {instrument_key},{name}, {interval}. Skipping chart pattern detection.")
                 # Ensure dummy columns exist even if detectors weren't instantiated
                 if 'Detected_Bullish_Chart_Pattern_Name' not in group_data.columns:
                     group_data['Detected_Bullish_Chart_Pattern_Name'] = None
                 if 'Detected_Bearish_Chart_Pattern_Name' not in group_data.columns:
                     group_data['Detected_Bearish_Chart_Pattern_Name'] = None
                 logger.debug(f"Debug Pattern Check: Added dummy Chart Pattern columns due to not instantiated.")




            # --- Instantiate Pattern Classes with the current instrument's data ---
            # Candlestick Pattern Detectors
            try:
                # Re-instantiate or pass updated group_data
                bullish_Candlestick_detector = BullishCandlestickPatterns(group_data.copy()) # Pass updated group_data
                bearish_Candlestick_detector = BearishCandlestickPatterns(group_data.copy()) # Pass updated group_data
                logger.debug(f"Debug Pattern Check: Candlestick Pattern Detectors re-instantiated with updated group_data for {instrument_key},{name}, {interval}.")

            except Exception as e:
                logger.error(f"Error re-instantiating Candlestick Pattern Detectors for {instrument_key},{name}, {interval}: {e}")
                bullish_Candlestick_detector = None # Set to None if instantiation fails
                bearish_Candlestick_detector = None # Set to None if instantiation fails



            # Candlestick Pattern Detection
            # Assuming these methods return a Series with True/False or 1/0 flags
            if bullish_Candlestick_detector is not None:
                try:
                    # Call detection methods - Ensure they accept the DataFrame as the first argument if needed
                    # Or ensure the detector instance was created with the correct DataFrame
                    bullish_detected_series = bullish_Candlestick_detector.detect_bullish_candlestick_patterns(instrument_key, name, interval) # Assuming methods use the df passed during instantiation
                    group_data['Bullish_Candlestick_Detected'] = bullish_detected_series
                    logger.debug(f"Debug Pattern Check: Successfully added Bullish Candlestick column for {instrument_key},{name}, {interval}.")
                except Exception as e:
                    logger.error(f"Error detecting bullish candlestick patterns for {instrument_key},{name}, {interval}: {e}")
                    group_data['Bullish_Candlestick_Detected'] = False # Add default column on error
                    logger.debug(f"Debug Pattern Check: Added dummy Bullish Candlestick column due to error.")

            else:
                logger.warning(f"Warning: Bullish Candlestick Detector not instantiated for {instrument_key},{name}, {interval}. Skipping bullish candlestick detection.")
                group_data['Bullish_Candlestick_Detected'] = False # Ensure column exists
                logger.debug(f"Debug Pattern Check: Ensured Bullish Candlestick column exists.")


            if bearish_Candlestick_detector is not None:
                try:
                    # Call detection methods
                    bearish_detected_series = bearish_Candlestick_detector.detect_Bearish_candlestick_patterns(instrument_key, name, interval) # Assuming methods use the df passed during instantiation
                    group_data['Bearish_Candlestick_Detected'] = bearish_detected_series
                    logger.debug(f"Debug Pattern Check: Successfully added Bearish Candlestick column for {instrument_key},{name}, {interval}.")
                except Exception as e:
                    logger.error(f"Error detecting bearish candlestick patterns for {instrument_key},{name}, {interval}: {e}")
                    group_data['Bearish_Candlestick_Detected'] = False # Add default column on error
                    logger.debug(f"Debug Pattern Check: Added dummy Bearish Candlestick column due to error.")
            else:
                logger.warning(f"Warning: Bearish Candlestick Detector not instantiated for {instrument_key},{name}, {interval}. Skipping bearish candlestick detection.")
                group_data['Bearish_Candlestick_Detected'] = False # Ensure column exists
                logger.debug(f"Debug Pattern Check: Ensured Bearish Candlestick column exists.")

            # Append the processed group_data to the list after adding all patterns/indicators
            processed_data_list.append(group_data)

        # After the loop, concatenate the processed data for all instruments
        if processed_data_list:
            # Concatenate all the processed DataFrames into df_backtest
            df_backtest = pd.concat(processed_data_list, ignore_index=True) # Use ignore_index=True to avoid duplicate indices
            print("\nFinal processed DataFrame for backtesting created.")
            print("Final processed DataFrame shape:", df_backtest.shape)
            print("Final processed DataFrame columns:", df_backtest.columns.tolist())
            print("Final processed DataFrame head:\n", df_backtest.head(10))

                    # Ensure timestamp column is datetime and sorted for the backtester
            if 'timestamp' in df_backtest.columns:
                df_backtest['timestamp'] = pd.to_datetime(df_backtest['timestamp'])
                df_backtest = df_backtest.sort_values('timestamp').reset_index(drop=True)
                logger.debug("df_backtest sorted by timestamp and index reset for backtester.")
            else:
                 logger.error("Timestamp column not found in df_backtest. Backtesting might fail.")


            print("\nProcessed data for backtesting head:")
            display(df_backtest.head())

        else:
            logger.warning("No data processed for backtesting.")
            df_backtest = pd.DataFrame() # Ensure it's an empty DataFrame if no data processed


    else:
        logger.warning("combined_df is not available or is empty. Skipping pattern detection and backtesting preparation.")
        df_backtest = pd.DataFrame() # Ensure df_backtest is empty if combined_df is not ready




#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####

    # --- Define Strategies and Instantiate Them (All Potential Strategies) ---
    # Initial capital for backtesting
    initial_capital = 200000.0 # Example initial capital

    # Create a pool of ALL potential strategy instances
    # Instantiating them here means they are available to be activated later
    all_strategy = {}

#####*****>>>>>>>>>>>>>> 1. Instantiate TrendFollowingStrategy and add to pool
    try:
        # This line might be failing
        trend_following_strategy = TrendFollowingStrategy(window_size=20) # Example parameter
        # If the above line fails, the code jumps to the except blocks
        # and the next line (adding to the dictionary) is never reached.

        all_strategy['TrendFollowing'] = trend_following_strategy # Use a consistent key name
        logger.info("TrendFollowingStrategy instantiated and added to pool.") # This log might not be appearing

    # >>> Check these except blocks closely <<<
    except NameError:
        # This catches if the TrendFollowingStrategy class itself isn't found/imported
        logger.warning("TrendFollowingStrategy class not found. Skipping instantiation.")
        # If this happens, the strategy is not added to the pool.
    except Exception as e:
        # This catches any other error during the __init__ method of the strategy
        logger.error(f"Error instantiating TrendFollowingStrategy for pool: {e}")
        # If this happens, the strategy is not added to the pool.

#####*****>>>>>>>>>>>>>> 2. Instantiate TrendFollowingStrategyV1 and add to pool
    try:
        # This line might be failing
        trend_following_strategyV1 = TrendFollowingStrategyV1(window_size=20) # Example parameter
        # If the above line fails, the code jumps to the except blocks
        # and the next line (adding to the dictionary) is never reached.

        all_strategy['TrendFollowingV1'] = trend_following_strategyV1 # Use a consistent key name
        logger.info("TrendFollowingStrategyV1 instantiated and added to pool.") # This log might not be appearing

    # >>> Check these except blocks closely <<<
    except NameError:
        # This catches if the TrendFollowingStrategyV1 class itself isn't found/imported
        logger.warning("TrendFollowingStrategyV1 class not found. Skipping instantiation.")
        # If this happens, the strategy is not added to the pool.
    except Exception as e:
        # This catches any other error during the __init__ method of the strategy
        logger.error(f"Error instantiating TrendFollowingStrategyV1 for pool: {e}")
        # If this happens, the strategy is not added to the pool.



#####*****>>>>>>>>>>>>>> 3.  Instantiate CombinedSignalStrategy and add to pool
    try:
        combined_signal_strategy = CombinedSignalStrategy() # Example
        all_strategy['CombinedSignal'] = combined_signal_strategy # Use a consistent key name
        logger.info("CombinedSignalStrategy instantiated and added to pool.")
    except NameError:
        logger.warning("CombinedSignalStrategy class not found. Skipping instantiation.")
    except Exception as e:
        logger.error(f"Error instantiating CombinedSignalStrategy for pool: {e}")

    # Add instantiations for any other strategy classes here, adding them to all_strategy_instances_pool


    # --- Backtesting Mode Execution ---
    if trading_mode == 'backtest':
        print("\nExecuting in Backtest Mode...")
        logger.info("Entering Backtest Mode.")
        print(">>> DEBUG MARKER 1: Entering backtesting mode <<<")

        # >>> DEFINE ACTIVE STRATEGIES FOR THIS RUN HERE <<<
        # List the NAMES (keys from all_strategy_instances_pool) of the strategies you want to run
        active_strategies_names_to_run = ['TrendFollowing'] # Example: Run only TrendFollowing
        # active_strategies_names_to_run = ['TrendFollowingV1'] # Example: Run only TrendFollowing
        # active_strategies_names_to_run = ['CombinedSignal'] # Example: Run only CombinedSignal
        # active_strategies_names_to_run = ['TrendFollowing', 'CombinedSignal'] # Example: Run both
        # active_strategies_names_to_run = [] # Example: Run none (useful for testing data loading)


        # >>> BUILD THE DICTIONARY OF ACTIVE STRATEGY INSTANCES FOR THE BACKTESTER <<<
        # Filter the pool based on the names in active_strategies_names_to_run
        active_strategies_instances_for_backtester = {}
        for strategy_name in active_strategies_names_to_run:
            if strategy_name in all_strategy:
                # Only add strategies whose names are in the list and are in the pool
                active_strategies_instances_for_backtester[strategy_name] = all_strategy[strategy_name]
            else:
                logger.warning(f"Strategy '{strategy_name}' listed in active_strategies_names_to_run not found in the instantiated pool. Skipping.")

        # Check if any strategies were successfully activated for the backtester
        if not active_strategies_instances_for_backtester:
            print("\nWarning: No valid active strategies found to run backtest based on your list. Check 'active_strategies_names_to_run'.")
            logger.warning("No valid active strategies found for backtest.")
            # You might want to add a 'continue' or 'pass' here if you want to skip the backtest execution
            pass # Skip the backtest execution block below if no strategies are active
        else:
            # --- Proceed with Backtester Instantiation and Execution ONLY if there are active strategies ---
            print(f"Active strategies for this backtest run: {list(active_strategies_instances_for_backtester.keys())}")

            # Ensure df_backtest is defined and not empty before proceeding
            # Assuming df_backtest is the combined_df filtered or prepared for backtesting
            if 'df_backtest' in locals() and not df_backtest.empty:
                try:
                    print("Starting Backtester instantiation...")
                    # Instantiate the Backtester with the prepared data and the FILTERED strategy instances
                    backtester = BacktesterV3(
                        data=df_backtest,
                        # It's better to get instrument keys from the data actually being used for backtesting
                        instrument_keys=df_backtest['instrument_key'].unique().tolist(),
                        active_strategies_instances=active_strategies_instances_for_backtester, # Pass the FILTERED dictionary
                        initial_capital=initial_capital
                        # header_df=header_df # Pass the header_df here
                    )
                    print("Backtester instantiated successfully.")

                    # Run the backtest simulation
                    print("Running backtest simulation...")
                    completed_trades_df = backtester.run_backtest()
                    print("Backtest simulation finished.")

                    # Get the completed trades (as a DataFrame)
                    # This is already returned by run_backtest(), so no need for get_completed_trades() here
                    # completed_trades_df = completed_trades_df # This line is redundant


                    # Analyze the backtest results
                    if completed_trades_df is not None and not completed_trades_df.empty:
                        print("\n--- Backtest Completed Trades Sample ---")
                        display(completed_trades_df.head()) # Display head of trades

                        # Perform analysis using the Backtester's method
                        print("\nAnalyzing backtest performance...")
                        analysis_results_df = backtester.analyze_backtest_results()

                        # Get the names of the active strategies that were run
                        strategy_names = list(active_strategies_instances_for_backtester.keys())

                        # Create a filename by joining the strategy names
                        # Replace potentially problematic characters like spaces and hyphens
                        filename_strategy_part = "_".join(strategy_names).replace(" ", "_").replace("-", "_")

                        # Add a timestamp to the filename for uniqueness
                        from datetime import datetime
                        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")


                        # Define the folder name and output filename
                        folder_name = 'Excel_backtest_result'

                        # Construct the full output filename
                        output_filename = f"backtest_results_{filename_strategy_part}_{timestamp_str}.csv"

                        # Construct the full output path
                        output_path = os.path.join(folder_name, output_filename)

                        # Create the directory if it doesn't exist
                        try:
                            os.makedirs(folder_name, exist_ok=True)
                            print(f"Ensured directory '{folder_name}' exists.")
                            logger.info(f"Ensured directory '{folder_name}' exists.")
                        except Exception as e:
                            logger.error(f"Error creating directory '{folder_name}': {e}")
                            print(f"Error creating directory '{folder_name}': {e}")

                        # Save the DataFrame to a CSV file
                        try:
                            completed_trades_df.to_csv(output_path, index=False) # Corrected: Pass only output_path
                            print(f"\nBacktest results saved to {output_path}") # Corrected print message
                            logger.info(f"Backtest results saved to {output_path}") # Corrected log message
                        except Exception as e:
                            logger.error(f"Error saving backtest results to CSV: {e}")
                            print(f"\nError saving backtest results to {output_path}: {e}") # Corrected print message

                        # --- End of code to SAVE results to CSV ---
                        # >>> Add the debug display code here <<<
                        print("--- Debug Data Captured ---")
                        if hasattr(backtester, '_debug_close_values'):
                            print(f"Captured {len(backtester._debug_close_values)} debug points.")
                            # Display the last few captured values to see the issue
                            display_count = min(20, len(backtester._debug_close_values)) # Increased display count slightly
                            print(f"Last {display_count} captured Close values and their validity:")
                            for j in range(len(backtester._debug_close_values) - display_count, len(backtester._debug_close_values)):
                                ts = backtester._debug_timestamps[j]
                                close_val = backtester._debug_close_values[j]
                                validity = backtester._debug_validity[j]
                                print(f"Timestamp: {ts}, Close: {close_val}, is_close_valid: {validity}, Type of Close: {type(close_val)}")
                        else:
                            print("Debug capture lists not found on backtester instance. Please ensure they are added to __init__ and populated in the loop.")

                    else:
                        print("\nNo completed trades recorded during backtesting.")
                        logger.info("No completed trades recorded during backtesting.")

                except NameError:
                    logger.error("df_backtest DataFrame is not defined. Cannot run backtest.")
                    print("\nError: Backtesting data (df_backtest) is not defined. Cannot run backtest.")

                except Exception as e:
                    logger.error(f"An error occurred during backtest execution: {e}", exc_info=True)
                    print(f"\nAn error occurred during backtest execution: {e}")

            elif not 'df_backtest' in locals() or df_backtest.empty:
                print("\nWarning: Backtesting data (df_backtest) is not available or is empty. Cannot run backtest.")
                logger.warning("Backtesting data (df_backtest) is not available or is empty. Cannot run backtest.")

         # --- End of backtesting mode execution block ---
#####*****************************************************************************************#####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####***>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<***####
#####*****************************************************************************************#####


    # --- Paper/Live Trading Mode (This part remains separate from backtesting) ---
    elif trading_mode in ['paper', 'live']:
        print(f"\nExecuting in {trading_mode.capitalize()} Mode...")
        logger.info(f"Entering {trading_mode.capitalize()} Mode.")

        # --- WebSocket Connection ---
        try:
             # Assuming connectwebsocket handles the connection and keeps it open
             # You might need to pass the instantiated strategies to the WebSocket handler
             # or a message processing function that can access and use them.
             # This part needs significant modification to integrate with strategies and execution.

             # Example (Conceptual - Needs Implementation in connectwebsocket or a handler):
             # await connectwebsocket(access_token, instrument_keys, data_handler_function, trading_mode, active_strategies_instances, initial_capital)

            print("WebSocket connection logic would go here...")
            print(f"In {trading_mode} mode, you would connect to the data feed, process real-time ticks/bars,")
            print("generate signals using active_strategies_instances, and execute paper/live trades.")
            print("This requires a separate real-time processing loop.")

             # Example of a real-time loop structure (highly simplified)
            async def real_time_loop():
                 resampler = RealTimeResampler(instrument_keys) # Instantiate Resampler
                 # Assume connectwebsocket somehow feeds raw data to resampler
                 # Assume resampler yields resampled bars (e.g., 1-minute bars)

                 # Needs a way to receive data, process it incrementally,
                 # add indicators/patterns to the latest bar, and then run strategies
                 # on that latest bar.

                 # Example structure within the real-time loop:
                 # async for resampled_bar in resampler.get_bars():
                 #      processed_bar = incremental_transformer.transform(resampled_bar) # Add latest indicators/patterns
                 #      for strategy_name, strategy_instance in active_strategies_instances.items():
                 #           signal = strategy_instance.generate_signal(processed_bar)
                 #           # Execute trade based on signal and trading_mode ('paper'/'live')
                 #           # Update paper_trades or send live order

                 print("Real-time processing loop placeholder.")


            # You would typically run the real_time_loop here or have connectwebsocket manage it
            # await real_time_loop()

            # Keep the application running until shutdown is signaled
            # await shutdown_event.wait() # Wait for the shutdown signal


        except Exception as e:
            logger.error(f"Error during {trading_mode.capitalize()} mode execution: {e}", exc_info=True)
            print(f"\nAn error occurred during {trading_mode.capitalize()} mode: {e}")


    #####*****************************************************************************************#####
    # --- Program End ---
    print("\nMain execution finished.")
    logger.info("Main execution finished.")

    # Display the final completed trades (if any) again for clarity in notebooks
    if 'completed_trades_df' in locals() and not completed_trades_df.empty:
         print("\nFinal Completed Trades (from Backtester):")
         display(completed_trades_df)
    elif trading_mode == 'backtest':
         print("\nBacktest ran, but no completed_trades_df available or it is empty.")
    else:
         print("\nNot in backtest mode, no completed trades to display from backtester.")

    # You might also display paper_trades or live trade logs here

# --- Run the main async function ---
if __name__ == "__main__":
    # Initialize a flag to indicate if we are in a Jupyter/IPython environment
    is_jupyter_env = False

    # Check if get_ipython is available (means we are in an IPython-like environment)
    try:
        # Attempt to get the IPython interactive shell
        # Add # type: ignore to suppress Pylance warning if desired
        ipython_shell = get_ipython() # type: ignore

        # If we successfully got the shell, we are in an IPython environment
        # Further check if it's specifically the kernel used by Jupyter notebooks
        if ipython_shell is not None and 'ipykernel' in ipython_shell.config:
             print("Running in Jupyter notebook environment.")
             is_jupyter_env = True
             # Apply nest_asyncio here specifically for the notebook environment
             # if you want to avoid applying it when running as a standard script
             # nest_asyncio.apply() # You can move this inside here if preferred
        # else:
             # Running in a different IPython environment (e.g., terminal IPython)
             # is_jupyter_env remains False

    except NameError:
        # get_ipython is not defined, which means we are in a standard Python environment
        print("Running in standard Python environment.")
        is_jupyter_env = False
        # If nest_asyncio.apply() was outside the try block, it will still run.
        # If you move it inside the try block's 'ipykernel' check,
        # it won't run in a standard script. Choose based on whether you need
        # nested asyncio loops in the standard script as well.
        # Given your original imports, it seems nest_asyncio.apply() is
        # already being called outside this block, which is fine if needed
        # for the standard script as well.

    except Exception as e:
        # Catch any other potential errors during the check
        print(f"An unexpected error occurred while checking environment: {e}")
        is_jupyter_env = False


    # --- Now, call your main async function 'main()' ---
    # Make sure the async def main(): function is defined elsewhere in your script

    # Get the event loop
    try:
        # Using asyncio.run() is often simpler for top-level entry points in Python 3.7+
        # if you don't need the custom loop returned by get_event_loop().
        # However, since you applied nest_asyncio, get_event_loop().run_until_complete()
        # is also appropriate and consistent.
        # Let's use get_event_loop() as per your previous pattern:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main()) # Ensure your 'main' function is async and defined

    except Exception as e:
         # Handle exceptions that might occur during the execution of main()
         print(f"An error occurred during script execution: {e}")
         logger.error(f"An error occurred during script execution: {e}", exc_info=True)

ModuleNotFoundError: No module named 'talib'

# Task
Modify the provided Python code for the `BacktesterV3` class to capture and store the names of detected bullish and bearish candlestick and chart patterns at the time of trade entry and exit. Ensure these pattern names are included in the output CSV file. The relevant columns in the CSV are `Entry_Bullish_Candlestick_Name`, `Entry_Bearish_Candlestick_Name`, `Entry_Bullish_Chart_Pattern_Name`, `Entry_Bearish_Chart_Pattern_Name`, `Exit_Bullish_Candlestick_Name`, `Exit_Bearish_Candlestick_Name`, `Exit_Bullish_Chart_Pattern_Detected`, and `Exit_Bearish_Chart_Pattern_Detected`. The debug output indicates that pattern detection is occurring, but the names are not being stored. The goal is to store the actual pattern names as strings in the CSV output.

## Refine data capture in `execute trade`

### Subtask:
Ensure all relevant entry data points (indicators, patterns, etc.) are correctly captured and stored when a position is opened.


**Reasoning**:
Review and confirm that the entry-specific data points are correctly captured from the `data_point` Series within the `execute_trade` method, ensuring the use of `.get()` for safety and verifying the assignment of `Entry_signal_type`, `Entry_order_type`, and `Position_type`. Also, check and update the debug logging for entry columns.



In [2]:
# _1113_6BacktesterV6.py

import pandas as pd
import numpy as np
import logging
from datetime import datetime # Import datetime for timestamp IDs
import uuid  # Keep uuid import in case it's used elsewhere, although not for generate_trade_id now
import sys # Import sys to check for handlers
# from _012_instruments import get_instrument_type # Removed unused import
# --- Logging Configuration ---
logger = logging.getLogger(__name__)



logger.setLevel(logging.DEBUG)

# Also ensure the root logger has a handler and is set to DEBUG,
# in case basicConfig was called elsewhere previously.
if not logging.root.handlers:
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
else:
    # If handlers exist, ensure at least one handler's level is DEBUG
    # and the root logger's level is DEBUG
    logging.root.setLevel(logging.DEBUG)
    handler_found = False
    for handler in logging.root.handlers:
        if isinstance(handler, logging.StreamHandler) and handler.stream in [sys.stdout, sys.stderr]:
            handler.setLevel(logging.DEBUG)
            handler_found = True
    # If no suitable handler is found (e.g., only file handlers), add a StreamHandler
    if not handler_found:
         stream_handler = logging.StreamHandler(sys.stdout)
         stream_handler.setLevel(logging.DEBUG)
         formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(message)s')
         stream_handler.setFormatter(formatter)
         logging.root.addHandler(stream_handler)


print("Logging level set to DEBUG for test.")

# Examine the __init__ method for data validation logic
# Check for the existence of the required_columns and log warnings for potential entry/exit data columns.

# required_columns check
# missing_required = [col for col in required_columns if col not in data.columns]
# if missing_required:
#      raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")

# entry_columns_to_check and exit_data_columns_to_check check
# missing_data_cols = [col for col in entry_columns_to_check + exit_data_columns_to_check if col not in data.columns]
# if missing_data_cols:
#      logger.warning(f"Input data is missing potential indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records.")

# The current code already performs these checks.
# required_columns are checked and raise a ValueError if missing.
# entry_columns_to_check and exit_data_columns_to_check are checked and log a warning if missing.

# Add comments to clarify assumptions about columns expected in input data.

class BacktesterV3:
    """
    A simple backtesting engine for evaluating trading strategies.
    Processes historical data bar by bar, generates signals, and simulates trades.
    """
    def __init__(self, data: pd.DataFrame, instrument_keys: list, active_strategies_instances: dict, initial_capital: float):
        """
        Includes the same parameters as the original __init__

        Initializes the Backtester.

        Args:
            data: A pandas DataFrame containing historical market data for all instruments,
                expected to have columns like 'timestamp', 'instrument_key',
                'open', 'high', 'low', 'close', 'volume', etc. It is also expected
                to contain pre-calculated indicator and pattern columns used by
                the strategies and for recording trade details.
            instrument_keys: A list of unique instrument keys present in the data.
            active_strategies_instances: A dictionary where keys are strategy names
                                        (strings) and values are instantiated strategy
                                        objects with a `generate_signal(data_point)` method.
            initial_capital: The starting capital for the backtest simulation.
        """
        if data is None or data.empty:
            raise ValueError("Input data DataFrame is None or empty.")
        if not isinstance(data, pd.DataFrame):
            raise TypeError("Input 'data' must be a pandas DataFrame.")
        if data.index.name is not None:
            logger.warning("Input data index is not None. Consider resetting the index before passing to Backtester.")


        # Ensure essential columns are present and sorted
        required_columns = ['timestamp', 'instrument_key', 'open', 'high', 'low', 'close']
        # Define columns expected to be in the input data for recording trade details.
        # These are typically pre-calculated indicators or pattern detection results.
        entry_exit_data_columns_expected = [
            'Trend', 'SMA20', 'RSI', 'RSIMA', 'ATR', 'ADX', 'Volatility',
            'Breakout_Detected', 'Breakdown_Detected', 'Bullish_Candlestick_Detected',
            'Bearish_Candlestick_Detected', 'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
            'name', 'interval', 'Currency',
            'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
            'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price'
        ]

        # The backtester expects these columns to be pre-calculated and provided in the input data.
        # Strategies generate signals based on these columns, and their values at the time of
        # entry and exit are recorded in the completed_trades DataFrame.


        # Perform a relaxed check: log a warning if potential entry/exit columns from data are missing
        missing_data_cols = [col for col in entry_exit_data_columns_expected if col not in data.columns]
        if missing_data_cols:
            logger.warning(f"Input data is missing expected indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records. Ensure your data preparation includes these columns if strategies or analysis depend on them.")


        # Ensure mandatory required columns are present
        missing_required = [col for col in required_columns if col not in data.columns]
        if missing_required:
            raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")


        # Ensure timestamp is datetime and sorted
        try:
            if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
                data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', utc=True) # Convert to UTC
            # Drop rows where timestamp conversion failed
            data = data.dropna(subset=['timestamp'])
            # Sort by timestamp and then instrument_key to process bars chronologically per instrument
            self.data = data.sort_values(by=['timestamp', 'instrument_key']).reset_index(drop=True)
        except Exception as e:
            raise ValueError(f"Error processing timestamp column in data: {e}")


        self.instrument_keys = instrument_keys
        self.active_strategies_instances = active_strategies_instances
        self.initial_capital = initial_capital

        # --- Backtesting State Variables ---
        self.current_capital = initial_capital
        self.positions = {}  # Dictionary to track open positions {instrument_key: {...entry details...}}
        self.completed_trades = [] # List to store completed trades
        self.trade_id_counter = 0 # Simple counter for trade IDs
        self.debug_log = [] # List to store debug information

        # Debug lists to capture values
        self._debug_timestamps = []
        self._debug_close_values = []
        self._debug_validity = []

        # Simple Slippage and Commission model (can be customized)
        self.slippage_pct = 0.001  # 0.1% slippage per trade
        self.commission_per_trade = 0.01 # $0.01 fixed commission per trade


        logger.info(f"BacktesterV2 initialized with {len(self.instrument_keys)} instruments and {len(self.active_strategies_instances)} active strategies.")
        logger.info(f"Initial Capital: {self.initial_capital}")
        logger.info(f"Data shape for backtesting: {self.data.shape}")


    def generate_trade_id(self, timestamp: datetime):
        """Generates a unique trade ID using a provided timestamp."""
        # Using microseconds to increase the chance of uniqueness
        return timestamp.strftime('%Y%m%d%H%M%S%f')


    def execute_trade(self, trade_id: str, instrument_key: str, timestamp: datetime, signal: str, strategy_name: str, price: float, data_point: pd.Series):
        """
        Simulates executing a trade based on a signal.

        Args:
            trade_id: Unique identifier for the trade.
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the trade execution (bar close time).
            signal: The trading signal ('BUY' or 'SELL').
            strategy_name: The name of the strategy generating the signal.
            price: The execution price (typically the close price of the bar).
            data_point: The pandas Series representing the data row for this bar. This Series
                        is expected to contain pre-calculated indicators and pattern data
                        used for entry/exit conditions and recording.
        """
        # Determine instrument type to handle lot size/quantity logic
        # instrument_type = get_instrument_type(instrument_key) # Removed due to import error
        instrument_type = 'Unknown' # Placeholder


        # Simple fixed quantity logic (can be replaced with dynamic position sizing)
        quantity_to_trade = 1 # Example: trade 1 unit/lot

        if signal == 'BUY':
            # Check if we already have a position in this instrument (optional, depending on strategy)
            if instrument_key not in self.positions:
                # Simulate buying
                cost = quantity_to_trade * price
                # Check if we have enough capital
                if self.current_capital >= cost:
                    self.current_capital -= cost

                    # Calculate entry costs (slippage and commission on entry)
                    entry_slippage = cost * self.slippage_pct
                    entry_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (entry_slippage + entry_commission) # Deduct costs from capital

                    # Capture entry-specific details from the data_point and other variables
                    self.positions[instrument_key] = {
                        'quantity': quantity_to_trade,
                        'entry_price': price, # This is the execution price for this simple model
                        'entry_time': timestamp,
                        'strategy': strategy_name,
                        'trade_id': trade_id,
                        'instrument_type': instrument_type, # Store instrument type (now placeholder)
                        'side': 'BUY', # Store trade side

                        # --- Entry-Specific Columns (Populated from data_point at Entry) ---
                        'Strategy_name': strategy_name,
                        'instrument_key': instrument_key,
                        'name': data_point.get('name'), # Use .get() to avoid errors if column is missing
                        'interval': data_point.get('interval'),
                        'Position_type': 'Long', # Assuming BUY means Long position
                        'Entry_order_type': 'Market', # Assuming market order execution on close
                        'Entry_timestamp': timestamp,
                        'Entry_price_trigger': None, # Not explicitly handled in this simple model
                        'Entry_price_execution': price,
                        'Entry_shares': quantity_to_trade, # Using quantity_to_trade as shares
                        'Entry_cost': cost, # Gross cost before fees
                        'Entry_signal_type': signal, # Ensure signal is captured
                        'Entry_Trend': data_point.get('Trend'), # Capture Trend at Entry
                        'Entry_SMA20': data_point.get('SMA20'), # Capture SMA20 at Entry
                        'Entry_RSI': data_point.get('RSI'), # Capture RSI at Entry
                        'Entry_RSI_MA': data_point.get('RSI_MA'), # Capture RSI_MA at Entry
                        'Entry_ATR': data_point.get('ATR'), # Capture ATR at Entry
                        'Entry_ADX': data_point.get('ADX'), # Capture ADX at Entry
                        'Entry_Volatility': data_point.get('Volatility'), # Capture Volatility at Entry
                        'Entry_Breakout_Detected': data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Entry
                        'Entry_Breakdown_Detected': data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Entry
                        # Corrected column names to match expected input data
                        'Entry_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Entry
                        'Entry_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Entry
                        'Entry_Bullish_Chart_Pattern_Name': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Entry
                        'Entry_Bearish_Chart_Pattern_Name': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Entry
                        'Instrument_Type': instrument_type, # Store instrument type (now placeholder)
                        'Currency': data_point.get('Currency'),
                        'Slippage_Entry': entry_slippage, # Store entry slippage
                        'Commission_Fees_Entry': entry_commission, # Store entry commission

                        # Placeholder for other entry-specific details that might be calculated by strategy (e.g., initial stop/target)
                        'Initial_Stop_Loss_Distance (%)': data_point.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': data_point.get('Risk_Amount'),
                        'Reward_Amount': data_point.get('Reward_Amount'),


                        # Placeholders for exit/other info that will be filled on close
                        # These fields are included here so the structure is consistent for retrieval on exit,
                        # even though their values are None at the time of entry.
                         'Max_Favorable_Excursion_MFE': None, # Will be calculated on exit
                         'Max_Adverse_Excursion_MAE': None, # Will be calculated on exit
                        'Current_trailing_stop': None, # Need logic for trailing stops
                        'Trailing_stop_method': None,
                        'Trailing_stop_value': None,
                        'Stop_loss_price': None,


                        'Exit_Trend': None, 'Exit_signal_type': None, 'Exit_SMA20': None,
                        'Exit_RSI': None, 'Exit_RSI_MA': None, 'Exit_ATR': None, 'Exit_ADX': None,
                        'Exit_Volatility': None, 'Exit_Breakout_Detected': None,
                        'Exit_Breakdown_Detected': None, 'Exit_Bullish_Candlestick_Name': None,
                        'Exit_Bearish_Candlestick_Name': None, 'Exit_Bullish_Chart_Pattern_Detected': None,
                        'Exit_Bearish_Chart_Pattern_Detected': None, 'Exit_shares': None,
                        'Exit_cost': None, 'Exit_revenue': None, 'PnL_trade': None,
                        'Trade_type': None, 'Profit_loss': None, 'Exit_reason': None,
                        'Slippage': None, 'Commission_Fees': None, 'Trade_Duration': None,
                        'Exit_Order_Type': None
                    }

                    # --- Add debug logging for Entry columns here ---
                    logger.debug(f"DEBUG Entry Data Point for {instrument_key} at {timestamp}:")
                    debug_cols_to_check = [
                        'Trend', 'SMA20', 'RSI', 'RSI_MA', 'ATR', 'ADX', 'Volatility',
                        'Breakout_Detected', 'Breakdown_Detected',
                        # Corrected debug column names to match expected input data
                        'Bullish_Candlestick_Detected', 'Bearish_Candlestick_Detected',
                        'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
                        'Currency', 'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
                        'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE', 'Current_trailing_stop',
                        'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price', 'Exit_Trend',
                        'Exit_signal_type', 'Exit_SMA20', 'Exit_RSI', 'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX',
                        'Exit_Volatility', 'Exit_Breakout_Detected', 'Exit_Breakdown_Detected',
                        'Exit_Bullish_Candlestick_Name', 'Exit_Bearish_Candlestick_Name',
                        'Exit_Bullish_Chart_Pattern_Detected', 'Exit_Bearish_Chart_Pattern_Detected',
                        'Exit_cost'
                    ]
                    for col in debug_cols_to_check:
                         logger.debug(f"  {col}: {data_point.get(col, 'Column Not Found or None')}")
                    # --- End Debug Logging ---


                    logger.info(f"Executed BUY trade {trade_id} for {instrument_key} at {timestamp} @ {price} (Qty: {quantity_to_trade}). Costs: Slippage={entry_slippage:.4f}, Commission={entry_commission:.2f}. Capital left: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'BUY', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'slippage': entry_slippage, 'commission': entry_commission})
                else:
                    logger.warning(f"Insufficient capital ({self.current_capital:.2f}) to BUY {instrument_key} at {price} (Cost: {cost:.2f}). Skipping trade {trade_id}.")
                    self.debug_log.append({'type': 'SKIP_BUY_CAPITAL', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Insufficient Capital'})

            else:
                # Already in a position, maybe add to it or skip depending on strategy rules
                logger.debug(f"Skipping BUY signal for {instrument_key} at {timestamp}. Already in a position.")
                self.debug_log.append({'type': 'SKIP_BUY_POSITION', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Already in Position'})


        elif signal == 'SELL':
            # For backtesting, a 'SELL' signal usually means closing a long position or opening a short position
            # Let's assume 'SELL' means closing a long position if one exists for simplicity in this example.
            # For a shorting strategy, you'd need different logic.
            if instrument_key in self.positions and self.positions[instrument_key]['side'] == 'BUY':
                # Simulate selling to close a long position
                position = self.positions[instrument_key]
                quantity_to_sell = position['quantity']
                entry_price = position['entry_price']
                entry_time = position['entry_time']
                strategy_opened = position['strategy']
                open_trade_id = position['trade_id']


                revenue = quantity_to_sell * price
                self.current_capital += revenue

                # Calculate Profit/Loss (Gross PnL)
                gross_pnl = (price - entry_price) * quantity_to_sell # For long position

                # Calculate exit costs (slippage and commission on exit)
                exit_slippage = revenue * self.slippage_pct
                exit_commission = self.commission_per_trade # Fixed commission per trade


                self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

                # Calculate Net PnL
                total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
                total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
                pnl_trade = gross_pnl - total_slippage - total_commission

                # Calculate Trade Duration
                trade_duration = (timestamp - entry_time).total_seconds() if pd.notnull(timestamp) and pd.notnull(entry_time) else None

                # Record completed trade - Populate all desired columns
                trade_record = {
                    'open_trade_id': open_trade_id,
                    'close_trade_id': trade_id,
                    'instrument_key': instrument_key,
                    'instrument_type': position.get('instrument_type'), # Assuming this key is correct in position
                    'side': position.get('side'), # Side of the position being closed (BUY/LONG)
                    'quantity': quantity_to_sell, # Quantity closed
                    'entry_price': entry_price,
                    'entry_time': entry_time,
                    'exit_price': price,
                    'exit_time': timestamp,
                    'pnl': gross_pnl, # Keep gross PnL for reference
                    'strategy_opened': strategy_opened,
                    'strategy_closed': strategy_name, # Record which strategy/signal closed it

                    # --- Transfer Entry Details from Position ---
                    'Strategy_name': position.get('Strategy_name'),
                    'instrument_key': position.get('instrument_key'),
                    'name': position.get('name'),
                    'interval': position.get('interval'),
                    'Position_type': position.get('Position_type'),
                    'Entry_order_type': position.get('Entry_order_type'),
                    'Entry_timestamp': position.get('Entry_timestamp'),
                    'Entry_price_trigger': position.get('Entry_price_trigger'),
                    'Entry_price_execution': position.get('Entry_price_execution'),
                    'Entry_shares': position.get('Entry_shares'),
                    'Entry_cost': position.get('Entry_cost'),
                    'Entry_signal_type': position.get('Entry_signal_type'),
                    'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                    'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                    'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                    'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                    'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                    'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                    'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                    'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                    'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                    # Corrected keys to match the expected format
                    'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                    'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                    'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                    'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                    'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                    'Currency': position.get('Currency'), # Get Currency at Entry from Position
                    'Slippage_Entry': position.get('Slippage_Entry'), # Transfer entry slippage
                    'Commission_Fees_Entry': position.get('Commission_Fees_Entry'), # Transfer entry commission
                    'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                    'Risk_Amount': position.get('Risk_Amount'),
                    'Reward_Amount': position.get('Reward_Amount'),
                    'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                    'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),


                    # --- Exit-Specific Columns (Populated from data_point at Exit) ---
                    'Exit_Trend': data_point.get('Exit_Trend'), # Capture Trend at Exit
                    'Exit_signal_type': signal, # Signal that triggered the exit
                    'Exit_SMA20': data_point.get('Exit_SMA20'), # Capture SMA20 at Exit
                    'Exit_RSI': data_point.get('Exit_RSI'), # Capture RSI at Exit
                    'Exit_RSI_MA': data_point.get('Exit_RSI_MA'), # Capture RSI_MA at Exit
                    'Exit_ATR': data_point.get('Exit_ATR'), # Capture ATR at Exit
                    'Exit_ADX': data_point.get('Exit_ADX'), # Capture ADX at Exit
                    'Exit_Volatility': data_point.get('Exit_Volatility'), # Capture Volatility at Exit
                    'Exit_Breakout_Detected': data_point.get('Exit_Breakout_Detected'), # Capture Breakout_Detected at Exit
                    'Exit_Breakdown_Detected': data_point.get('Exit_Breakdown_Detected'), # Capture Breakdown_Detected at Exit
                    # Corrected keys to match the expected format
                    'Exit_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Exit
                    'Exit_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Exit
                    'Exit_Bullish_Chart_Pattern_Detected': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Exit
                    'Exit_Bearish_Chart_Pattern_Detected': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Exit
                    'Exit_shares': quantity_to_sell, # Shares exited
                    'Exit_cost': 0, # Assuming no cost to exit a long position
                    'Exit_revenue': revenue, # Gross revenue before fees
                    'PnL_trade': pnl_trade, # Net PnL after costs
                    'Trade_type': 'Long Close (Signal)', # Or 'Long' if trade refers to the full cycle
                    'Profit_loss': pnl_trade, # Update Profit_loss to net PnL
                    'Exit_reason': f'Signal_{signal}', # Reason for exit
                    'Exit_Order_Type': 'Market', # Set Exit_Order_Type to 'Market'

                    # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                    'Current_trailing_stop': data_point.get('Current_trailing_stop'), # Capture if calculated and available
                    'Trailing_stop_method': data_point.get('Trailing_stop_method'), # Capture if calculated and available
                    'Trailing_stop_value': data_point.get('Trailing_stop_value'), # Capture if calculated and available
                    'Stop_loss_price': data_point.get('Stop_loss_price'), # Capture if calculated and available


                    'Slippage': total_slippage, # Total slippage for the round trip
                    'Commission_Fees': total_commission, # Total commission for the round trip
                    'Trade_Duration': trade_duration,

                }
                self.completed_trades.append(trade_record)


                # Remove position
                del self.positions[instrument_key]

                logger.info(f"Executed SELL trade {trade_id} for {instrument_key} at {timestamp} @ {price} to CLOSE long position {open_trade_id}. Gross PnL: {gross_pnl:.2f}. Costs: Slippage={total_slippage:.4f}, Commission={total_commission:.2f}. Net PnL: {pnl_trade:.2f}. Capital left: {self.current_capital:.2f}")
                self.debug_log.append({'type': 'SELL_CLOSE_LONG', 'open_trade_id': open_trade_id, 'close_trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_sell, 'gross_pnl': gross_pnl, 'net_pnl': pnl_trade, 'strategy_closed': strategy_name})


            else:
                # No matching long position to close, or maybe a shorting signal
                # For this simple backtester, we'll just log and skip if no long position
                logger.debug(f"Skipping SELL signal for {instrument_key} at {timestamp}. No matching long position to close.")
                self.debug_log.append({'type': 'SKIP_SELL_NO_LONG', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'strategy': strategy_name, 'reason': 'No Long Position'})


    def run_backtest(self):
        """
        Runs the backtesting simulation bar by bar through the data.
        """
        logger.info("Starting backtest simulation...")

        # Group data by timestamp first, then iterate through timestamps
        # This processes all instruments available at a given time before moving to the next time.
        grouped_by_time = self.data.groupby('timestamp')

        for timestamp, time_slice_df in grouped_by_time:
            # Process data for all instruments available at this timestamp
            for index, data_point in time_slice_df.iterrows():
                instrument_key = data_point['instrument_key']
                current_price = data_point['close'] # Assume close price for execution

                # Debug capture
                self._debug_timestamps.append(timestamp)
                self._debug_close_values.append(current_price)
                self._debug_validity.append(pd.notna(current_price))

                # Ensure current_price is valid for trading
                if pd.isna(current_price):
                    logger.debug(f"Skipping signal generation for {instrument_key} at {timestamp} due to invalid close price ({current_price}).")
                    self.debug_log.append({'type': 'SKIP_SIGNAL_PRICE_NAN', 'instrument': instrument_key, 'time': timestamp, 'reason': 'Invalid Price'})
                    continue # Skip this data point if price is invalid

                # Check for signals from all active strategies for this data point
                # Pass the single data_point (as a Series) to the strategy
                # The strategy is expected to handle a single row/Series or convert it internally
                # For the backtester's execute_trade, we need the Series directly.

                for strategy_name, strategy_instance in self.active_strategies_instances.items():
                    try:
                        # Pass the single data_point Series to the strategy's generate_signal
                        # Ensure data_point is passed as a DataFrame slice if strategy expects DataFrame
                        signal = strategy_instance.generate_signal(pd.DataFrame([data_point]))
                        # Ensure signal is a string, handle potential None returns gracefully
                        signal = str(signal).upper() if signal is not None else 'HOLD'

                        if signal in ['BUY', 'SELL']:
                            # Generate a unique trade ID for this potential trade using the bar's timestamp
                            trade_id = self.generate_trade_id(timestamp) # Pass the historical timestamp
                            # Pass the original data_point Series to execute_trade
                            self.execute_trade(trade_id, instrument_key, timestamp, signal, strategy_name, current_price, data_point)

                    except Exception as e:
                        logger.error(f"Error generating signal for {instrument_key} at {timestamp} using strategy '{strategy_name}': {e}", exc_info=True)
                        self.debug_log.append({'type': 'STRATEGY_ERROR', 'instrument': instrument_key, 'time': timestamp, 'strategy': strategy_name, 'error': str(e)})


        # After iterating through all data, close any remaining open positions
        self.close_all_positions(self.data['timestamp'].max()) # Use the timestamp of the last data point as exit time

        logger.info("Backtest simulation completed.")
        logger.info(f"Final Capital: {self.current_capital:.2f}")
        logger.info(f"Number of completed trades: {len(self.completed_trades)}")
        logger.info(f"Number of open positions remaining: {len(self.positions)}")

        # Return completed trades as a DataFrame for analysis
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

            # Ensure numeric columns are numeric
            numeric_cols = [
                'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
            ]
            for col in numeric_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')


            return completed_trades_df
        else:
            logger.warning("No completed trades recorded. Returning empty DataFrame.")
            return pd.DataFrame() # Return empty DataFrame if no trades


    def close_all_positions(self, exit_timestamp: datetime):
        """
        Closes all remaining open positions at the specified exit timestamp.
        Assumes closing at the price of the last available bar for each instrument.
        """
        logger.info(f"Closing all remaining {len(self.positions)} open positions at {exit_timestamp}...")

        # Get the last known price and data point for each instrument with an open position
        last_data_points = self.data.groupby('instrument_key').tail(1).set_index('instrument_key')
        last_prices = last_data_points['close'].to_dict()


        positions_to_close = list(self.positions.keys()) # Iterate over a copy

        for instrument_key in positions_to_close:
            if instrument_key in self.positions: # Check if position still exists (wasn't closed by a signal just before the end)
                position = self.positions[instrument_key]
                closing_price = last_prices.get(instrument_key, np.nan) # Get last price, default to NaN if instrument not found

                # Get the last data point for the instrument to capture exit conditions
                last_data_point = last_data_points.get(instrument_key, pd.Series({})) # Use empty Series if no data found


                if pd.notna(closing_price):
                    trade_id = self.generate_trade_id(exit_timestamp) # Generate a new trade ID for the closing trade using exit_timestamp
                    # Simulate selling to close a long position
                    quantity_to_sell = position['quantity']
                    entry_price = position['entry_price']
                    entry_time = position['entry_time']
                    strategy_opened = position['strategy']
                    open_trade_id = position['trade_id']


                    revenue = quantity_to_sell * closing_price
                    self.current_capital += revenue

                    # Calculate Profit/Loss (Gross PnL)
                    gross_pnl = (closing_price - entry_price) * quantity_to_sell # For long position

                    # Calculate exit costs (slippage and commission on exit)
                    exit_slippage = revenue * self.slippage_pct
                    exit_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

                    # Calculate Net PnL
                    total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
                    total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
                    pnl_trade = gross_pnl - total_slippage - total_commission


                    # Calculate Trade Duration
                    trade_duration = (exit_timestamp - entry_time).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(entry_time) else None


                    # Record completed trade - Populate all desired columns
                    trade_record = {
                        'open_trade_id': open_trade_id,
                        'close_trade_id': trade_id, # New ID for closing
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'), # Use .get() for safety
                        'side': position.get('side'), # Side of the position being closed (BUY/LONG)
                        'quantity': quantity_to_sell, # Quantity closed
                        'entry_price': entry_price,
                        'entry_time': entry_time,
                        'exit_price': closing_price,
                        'exit_time': exit_timestamp, # Use the provided exit timestamp
                        'pnl': gross_pnl, # Keep gross PnL for reference
                        'strategy_opened': strategy_opened,
                        'strategy_closed': 'Backtester_Forced_Close', # Indicate it was closed by the backtester

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        # Corrected keys to match the expected format
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': position.get('Slippage_Entry'), # Transfer entry slippage
                        'Commission_Fees_Entry': position.get('Commission_Fees_Entry'), # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Adverse_Excursion_MAE'),


                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close', # Indicate forced close
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        # Corrected keys to match the expected format
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': quantity_to_sell, # Shares exited
                        'Exit_cost': 0, # Assuming no cost to exit a long position
                        'Exit_revenue': revenue, # Gross revenue before fees
                        'PnL_trade': pnl_trade, # Net PnL after costs
                        'Trade_type': 'Long Close (Forced)', # Or 'Long' if trade refers to the full cycle
                        'Profit_loss': pnl_trade, # Update Profit_loss to net PnL
                        'Exit_reason': 'Backtester_Forced_Close', # Reason for exit
                        'Exit_Order_Type': 'Market', # Set Exit_Order_Type to 'Market' or 'Forced'

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': last_data_point.get('Current_trailing_stop'), # Capture if calculated and available
                        'Trailing_stop_method': last_data_point.get('Trailing_stop_method'), # Capture if calculated and available
                        'Trailing_stop_value': last_data_point.get('Trailing_stop_value'), # Capture if calculated and available
                        'Stop_loss_price': last_data_point.get('Stop_loss_price'), # Capture if calculated and available


                        'Slippage': total_slippage, # Total slippage for the round trip
                        'Commission_Fees': total_commission, # Total commission for the round trip
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(trade_record)

                    # Remove position
                    del self.positions[instrument_key]

                    logger.info(f"Closed remaining position {open_trade_id} for {instrument_key} at {exit_timestamp} @ {closing_price}. Gross PnL: {gross_pnl:.2f}. Costs: Slippage={total_slippage:.4f}, Commission={total_commission:.2f}. Net PnL: {pnl_trade:.2f}. Capital: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'FORCED_CLOSE', 'open_trade_id': open_trade_id, 'close_trade_id': trade_id, 'instrument': instrument_key, 'time': exit_timestamp, 'price': closing_price, 'quantity': quantity_to_sell, 'gross_pnl': gross_pnl, 'net_pnl': pnl_trade, 'strategy_closed': 'Backtester_Forced_Close'})

                else:
                    logger.warning(f"Could not find last price for {instrument_key}. Cannot close position {position['trade_id']}. Logging as unresolved.")
                    # Log as an unresolved position or assume zero PnL

                    # Calculate Trade Duration even if closing price is NaN
                    trade_duration = (exit_timestamp - position.get('entry_time')).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(position.get('entry_time')) else None

                    # Transfer known costs
                    entry_slippage = position.get('Slippage_Entry', 0)
                    entry_commission = position.get('Commission_Fees_Entry', 0)


                    unresolved_trade_record = {
                        'open_trade_id': position.get('trade_id'),
                        'close_trade_id': None, # No closing trade ID
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'),
                        'side': position.get('side'),
                        'quantity': position.get('quantity'),
                        'entry_price': position.get('entry_price'),
                        'entry_time': position.get('entry_time'),
                        'exit_price': None, # No exit price
                        'exit_time': exit_timestamp, # Use the requested exit timestamp
                        'pnl': 0, # Assume zero Gross PnL if cannot close
                        'strategy_opened': position.get('strategy'),
                        'strategy_closed': 'Backtester_Forced_Close_Error', # Indicate error

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        # Corrected keys to match the expected format
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': entry_slippage, # Transfer entry slippage
                        'Commission_Fees_Entry': entry_commission, # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),


                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close_Error', # Indicate forced close error
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        # Corrected keys to match the expected format
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': position.get('quantity'), # Shares that were supposed to be exited
                        'Exit_cost': 0,
                        'Exit_revenue': 0, # Assuming zero revenue if cannot close
                        'PnL_trade': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Profit_loss': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Exit_reason': 'Backtester_Forced_Close_Error: No_Last_Price', # Reason for exit
                        'Exit_Order_Type': None, # Could not execute exit order

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': last_data_point.get('Current_trailing_stop'), # Capture if calculated and available
                        'Trailing_stop_method': last_data_point.get('Trailing_stop_method'), # Capture if calculated and available
                        'Trailing_stop_value': last_data_point.get('Trailing_stop_value'), # Capture if calculated and available
                        'Stop_loss_price': last_data_point.get('Stop_loss_price'), # Capture if calculated and available

                        'Slippage': entry_slippage, # Only entry slippage is known
                        'Commission_Fees': entry_commission, # Only entry commission is known
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(unresolved_trade_record)

                    # Remove position even if it couldn't be closed properly to prevent it from being processed again
                    del self.positions[instrument_key]


                    self.debug_log.append({'type': 'FORCED_CLOSE_ERROR', 'open_trade_id': position['trade_id'], 'instrument': instrument_key, 'time': exit_timestamp, 'reason': 'Last price not available'})


        logger.info("All remaining positions closed.")


    def analyze_backtest_results(self):
        """
        Analyzes the completed trades and provides performance metrics.
        Returns a DataFrame summarizing the analysis.
        """
        logger.info("Analyzing backtest results...")

        if not self.completed_trades:
            logger.warning("No completed trades to analyze.")
            return pd.DataFrame({'Message': ['No completed trades to analyze.']})

        # 1. Access the self.completed_trades list and Create a pandas DataFrame
        trades_df = pd.DataFrame(self.completed_trades)

        # 3. Ensure that relevant columns are converted to appropriate numeric types
        numeric_cols = [
            'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
            'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
            'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
            'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
            'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
            'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
            'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
            'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Include other potentially numeric cols
        ]
        for col in numeric_cols:
            if col in trades_df.columns:
                trades_df[col] = pd.to_numeric(trades_df[col], errors='coerce')

        # Handle potential NaN values during conversion - drop rows where PnL_trade (or pnl) is NaN
        pnl_col_for_analysis = 'PnL_trade' if 'PnL_trade' in trades_df.columns else 'pnl'
        if pnl_col_for_analysis in trades_df.columns:
            # Only consider trades with a valid PnL for core analysis metrics
            trades_df_analysis = trades_df.dropna(subset=[pnl_col_for_analysis]).copy() # Use .copy() to avoid SettingWithCopyWarning
        else:
            logger.warning("Neither 'PnL_trade' nor 'pnl' column found for analysis.")
            return pd.DataFrame({'Message': ['No PnL column found for analysis.']})


        if trades_df_analysis.empty:
            logger.warning("No valid trades after numeric conversion/dropna for analysis. Analysis stopped.")
            return pd.DataFrame({'Message': ['No valid trades after numeric conversion/dropna for analysis.']})


        # 4. Update the calculation of basic performance metrics using 'PnL_trade'
        total_trades = len(trades_df_analysis)
        total_pnl = trades_df_analysis[pnl_col_for_analysis].sum()

        winning_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] > 0]
        losing_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] < 0]
        breakeven_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] == 0]

        num_winning = len(winning_trades)
        num_losing = len(losing_trades)
        num_breakeven = len(breakeven_trades)

        win_rate = (num_winning / total_trades) * 100 if total_trades > 0 else 0
        avg_win = winning_trades[pnl_col_for_analysis].mean() if num_winning > 0 else 0
        avg_loss = losing_trades[pnl_col_for_analysis].mean() if num_losing > 0 else 0
        expectancy = (win_rate / 100) * avg_win + ((100 - win_rate) / 100) * avg_loss if total_trades > 0 else 0

        # 5. Update Max Drawdown calculation to use 'PnL_trade' and sort by exit time
        # Calculate cumulative PnL and then cumulative capital
        trades_df_analysis = trades_df_analysis.sort_values(by='exit_time') # Sort by exit time for cumulative calculation

        trades_df_analysis['cumulative_pnl'] = trades_df_analysis[pnl_col_for_analysis].cumsum()

        # Add initial capital to cumulative PnL
        trades_df_analysis['cumulative_capital'] = self.initial_capital + trades_df_analysis['cumulative_pnl']

        # Calculate peak capital up to each point
        trades_df_analysis['peak_capital'] = trades_df_analysis['cumulative_capital'].cummax()

        # Calculate drawdown at each point
        trades_df_analysis['drawdown'] = trades_df_analysis['peak_capital'] - trades_df_analysis['cumulative_capital']

        # Calculate percentage drawdown
        # Avoid division by zero if peak_capital is 0 or None
        trades_df_analysis['pct_drawdown'] = trades_df_analysis.apply(
            lambda row: (row['drawdown'] / row['peak_capital']) * 100 if row['peak_capital'] > 0 and pd.notna(row['peak_capital']) else 0,
            axis=1
        )


        max_drawdown_amount = trades_df_analysis['drawdown'].max() if not trades_df_analysis.empty else 0
        max_drawdown_pct = trades_df_analysis['pct_drawdown'].max() if not trades_df_analysis.empty else 0


        # 6. Update analysis summary metric names
        analysis_summary = {
            'Metric': [
                'Initial Capital',
                'Final Capital',
                'Total PnL (Net)', # Indicate Net PnL
                'Total Trades',
                'Winning Trades (Net)', # Indicate Net PnL
                'Losing Trades (Net)', # Indicate Net PnL
                'Breakeven Trades (Net)', # Indicate Net PnL
                'Win Rate (%) (Net PnL)', # Indicate Net PnL
                'Average Win (Net)', # Indicate Net PnL
                'Average Loss (Net)', # Indicate Net PnL
                'Expectancy per Trade (Net)', # Indicate Net PnL
                'Max Drawdown (Amount)',
                'Max Drawdown (%)',
            ],
            'Value': [
                self.initial_capital,
                self.current_capital,
                round(total_pnl, 2), # Format to 2 decimal places
                total_trades,
                num_winning,
                num_losing,
                num_breakeven,
                round(win_rate, 2), # Format to 2 decimal places
                round(avg_win, 2),
                round(avg_loss, 2),
                round(expectancy, 2),
                round(max_drawdown_amount, 2),
                round(max_drawdown_pct, 2),
            ]
        }

        analysis_df = pd.DataFrame(analysis_summary)

        logger.info("Backtest analysis completed.")
        # You can print the analysis_df here or return it
        # print("\n--- Backtest Analysis Summary ---")
        # display(analysis_df) # Use display for notebooks

        # 8. Ensure the method returns the updated analysis summary DataFrame
        return analysis_df

    def get_completed_trades(self):
        """Returns a DataFrame of completed trades."""
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

                    # Ensure numeric columns are numeric
                    numeric_cols = [
                        'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                        'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                        'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                        'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                        'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                        'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                        'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                        'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                        'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
                    ]
                    for col in numeric_cols:
                        if col in completed_trades_df.columns:
                            completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')

                    return completed_trades_df
                else:
                    return pd.DataFrame() # Return empty DataFrame if no trades

Logging level set to DEBUG for test.


**Reasoning**:
Review and confirm that the exit-specific data points are correctly captured from the `data_point` Series within the `execute_trade` and `close_all_positions` methods when closing a position, ensuring the use of `.get()` for safety and verifying the assignment of `Exit_signal_type` and `Exit_order_type`. Also, check and update the debug logging for exit columns.



In [3]:
# _1113_6BacktesterV6.py

import pandas as pd
import numpy as np
import logging
from datetime import datetime # Import datetime for timestamp IDs
import uuid  # Keep uuid import in case it's used elsewhere, although not for generate_trade_id now
import sys # Import sys to check for handlers
# from _012_instruments import get_instrument_type # Removed unused import
# --- Logging Configuration ---
logger = logging.getLogger(__name__)



logger.setLevel(logging.DEBUG)

# Also ensure the root logger has a handler and is set to DEBUG,
# in case basicConfig was called elsewhere previously.
if not logging.root.handlers:
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
else:
    # If handlers exist, ensure at least one handler's level is DEBUG
    # and the root logger's level is DEBUG
    logging.root.setLevel(logging.DEBUG)
    handler_found = False
    for handler in logging.root.handlers:
        if isinstance(handler, logging.StreamHandler) and handler.stream in [sys.stdout, sys.stderr]:
            handler.setLevel(logging.DEBUG)
            handler_found = True
    # If no suitable handler is found (e.g., only file handlers), add a StreamHandler
    if not handler_found:
         stream_handler = logging.StreamHandler(sys.stdout)
         stream_handler.setLevel(logging.DEBUG)
         formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(message)s')
         stream_handler.setFormatter(formatter)
         logging.root.addHandler(stream_handler)


print("Logging level set to DEBUG for test.")

# Examine the __init__ method for data validation logic
# Check for the existence of the required_columns and log warnings for potential entry/exit data columns.

# required_columns check
# missing_required = [col for col in required_columns if col not in data.columns]
# if missing_required:
#      raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")

# entry_columns_to_check and exit_data_columns_to_check check
# missing_data_cols = [col for col in entry_columns_to_check + exit_data_columns_to_check if col not in data.columns]
# if missing_data_cols:
#      logger.warning(f"Input data is missing potential indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records.")

# The current code already performs these checks.
# required_columns are checked and raise a ValueError if missing.
# entry_columns_to_check and exit_data_columns_to_check are checked and log a warning if missing.

# Add comments to clarify assumptions about columns expected in input data.

class BacktesterV3:
    """
    A simple backtesting engine for evaluating trading strategies.
    Processes historical data bar by bar, generates signals, and simulates trades.
    """
    def __init__(self, data: pd.DataFrame, instrument_keys: list, active_strategies_instances: dict, initial_capital: float):
        """
        Includes the same parameters as the original __init__

        Initializes the Backtester.

        Args:
            data: A pandas DataFrame containing historical market data for all instruments,
                expected to have columns like 'timestamp', 'instrument_key',
                'open', 'high', 'low', 'close', 'volume', etc. It is also expected
                to contain pre-calculated indicator and pattern columns used by
                the strategies and for recording trade details.
            instrument_keys: A list of unique instrument keys present in the data.
            active_strategies_instances: A dictionary where keys are strategy names
                                        (strings) and values are instantiated strategy
                                        objects with a `generate_signal(data_point)` method.
            initial_capital: The starting capital for the backtest simulation.
        """
        if data is None or data.empty:
            raise ValueError("Input data DataFrame is None or empty.")
        if not isinstance(data, pd.DataFrame):
            raise TypeError("Input 'data' must be a pandas DataFrame.")
        if data.index.name is not None:
            logger.warning("Input data index is not None. Consider resetting the index before passing to Backtester.")


        # Ensure essential columns are present and sorted
        required_columns = ['timestamp', 'instrument_key', 'open', 'high', 'low', 'close']
        # Define columns expected to be in the input data for recording trade details.
        # These are typically pre-calculated indicators or pattern detection results.
        entry_exit_data_columns_expected = [
            'Trend', 'SMA20', 'RSI', 'RSIMA', 'ATR', 'ADX', 'Volatility',
            'Breakout_Detected', 'Breakdown_Detected', 'Bullish_Candlestick_Detected',
            'Bearish_Candlestick_Detected', 'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
            'name', 'interval', 'Currency',
            'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
            'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price'
        ]

        # The backtester expects these columns to be pre-calculated and provided in the input data.
        # Strategies generate signals based on these columns, and their values at the time of
        # entry and exit are recorded in the completed_trades DataFrame.


        # Perform a relaxed check: log a warning if potential entry/exit columns from data are missing
        missing_data_cols = [col for col in entry_exit_data_columns_expected if col not in data.columns]
        if missing_data_cols:
            logger.warning(f"Input data is missing expected indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records. Ensure your data preparation includes these columns if strategies or analysis depend on them.")


        # Ensure mandatory required columns are present
        missing_required = [col for col in required_columns if col not in data.columns]
        if missing_required:
            raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")


        # Ensure timestamp is datetime and sorted
        try:
            if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
                data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', utc=True) # Convert to UTC
            # Drop rows where timestamp conversion failed
            data = data.dropna(subset=['timestamp'])
            # Sort by timestamp and then instrument_key to process bars chronologically per instrument
            self.data = data.sort_values(by=['timestamp', 'instrument_key']).reset_index(drop=True)
        except Exception as e:
            raise ValueError(f"Error processing timestamp column in data: {e}")


        self.instrument_keys = instrument_keys
        self.active_strategies_instances = active_strategies_instances
        self.initial_capital = initial_capital

        # --- Backtesting State Variables ---
        self.current_capital = initial_capital
        self.positions = {}  # Dictionary to track open positions {instrument_key: {...entry details...}}
        self.completed_trades = [] # List to store completed trades
        self.trade_id_counter = 0 # Simple counter for trade IDs
        self.debug_log = [] # List to store debug information

        # Debug lists to capture values
        self._debug_timestamps = []
        self._debug_close_values = []
        self._debug_validity = []

        # Simple Slippage and Commission model (can be customized)
        self.slippage_pct = 0.001  # 0.1% slippage per trade
        self.commission_per_trade = 0.01 # $0.01 fixed commission per trade


        logger.info(f"BacktesterV2 initialized with {len(self.instrument_keys)} instruments and {len(self.active_strategies_instances)} active strategies.")
        logger.info(f"Initial Capital: {self.initial_capital}")
        logger.info(f"Data shape for backtesting: {self.data.shape}")


    def generate_trade_id(self, timestamp: datetime):
        """Generates a unique trade ID using a provided timestamp."""
        # Using microseconds to increase the chance of uniqueness
        return timestamp.strftime('%Y%m%d%H%M%S%f')


    def execute_trade(self, trade_id: str, instrument_key: str, timestamp: datetime, signal: str, strategy_name: str, price: float, data_point: pd.Series):
        """
        Simulates executing a trade based on a signal.

        Args:
            trade_id: Unique identifier for the trade.
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the trade execution (bar close time).
            signal: The trading signal ('BUY' or 'SELL').
            strategy_name: The name of the strategy generating the signal.
            price: The execution price (typically the close price of the bar).
            data_point: The pandas Series representing the data row for this bar. This Series
                        is expected to contain pre-calculated indicator and pattern data
                        used for entry/exit conditions and recording.
        """
        # Determine instrument type to handle lot size/quantity logic
        # instrument_type = get_instrument_type(instrument_key) # Removed due to import error
        instrument_type = 'Unknown' # Placeholder


        # Simple fixed quantity logic (can be replaced with dynamic position sizing)
        quantity_to_trade = 1 # Example: trade 1 unit/lot

        if signal == 'BUY':
            # Check if we already have a position in this instrument (optional, depending on strategy)
            if instrument_key not in self.positions:
                # Simulate buying
                cost = quantity_to_trade * price
                # Check if we have enough capital
                if self.current_capital >= cost:
                    self.current_capital -= cost

                    # Calculate entry costs (slippage and commission on entry)
                    entry_slippage = cost * self.slippage_pct
                    entry_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (entry_slippage + entry_commission) # Deduct costs from capital

                    # Capture entry-specific details from the data_point and other variables
                    self.positions[instrument_key] = {
                        'quantity': quantity_to_trade,
                        'entry_price': price, # This is the execution price for this simple model
                        'entry_time': timestamp,
                        'strategy': strategy_name,
                        'trade_id': trade_id,
                        'instrument_type': instrument_type, # Store instrument type (now placeholder)
                        'side': 'BUY', # Store trade side

                        # --- Entry-Specific Columns (Populated from data_point at Entry) ---
                        'Strategy_name': strategy_name,
                        'instrument_key': instrument_key,
                        'name': data_point.get('name'), # Use .get() to avoid errors if column is missing
                        'interval': data_point.get('interval'),
                        'Position_type': 'Long', # Assuming BUY means Long position
                        'Entry_order_type': 'Market', # Assuming market order execution on close
                        'Entry_timestamp': timestamp,
                        'Entry_price_trigger': None, # Not explicitly handled in this simple model
                        'Entry_price_execution': price,
                        'Entry_shares': quantity_to_trade, # Using quantity_to_trade as shares
                        'Entry_cost': cost, # Gross cost before fees
                        'Entry_signal_type': signal, # Ensure signal is captured
                        'Entry_Trend': data_point.get('Trend'), # Capture Trend at Entry
                        'Entry_SMA20': data_point.get('SMA20'), # Capture SMA20 at Entry
                        'Entry_RSI': data_point.get('RSI'), # Capture RSI at Entry
                        'Entry_RSI_MA': data_point.get('RSI_MA'), # Capture RSI_MA at Entry
                        'Entry_ATR': data_point.get('ATR'), # Capture ATR at Entry
                        'Entry_ADX': data_point.get('ADX'), # Capture ADX at Entry
                        'Entry_Volatility': data_point.get('Volatility'), # Capture Volatility at Entry
                        'Entry_Breakout_Detected': data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Entry
                        'Entry_Breakdown_Detected': data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Entry
                        # Corrected column names to match expected input data
                        'Entry_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Entry
                        'Entry_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Entry
                        'Entry_Bullish_Chart_Pattern_Name': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Entry
                        'Entry_Bearish_Chart_Pattern_Name': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Entry
                        'Instrument_Type': instrument_type, # Store instrument type (now placeholder)
                        'Currency': data_point.get('Currency'),
                        'Slippage_Entry': entry_slippage, # Store entry slippage
                        'Commission_Fees_Entry': entry_commission, # Store entry commission

                        # Placeholder for other entry-specific details that might be calculated by strategy (e.g., initial stop/target)
                        'Initial_Stop_Loss_Distance (%)': data_point.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': data_point.get('Risk_Amount'),
                        'Reward_Amount': data_point.get('Reward_Amount'),


                        # Placeholders for exit/other info that will be filled on close
                        # These fields are included here so the structure is consistent for retrieval on exit,
                        # even though their values are None at the time of entry.
                         'Max_Favorable_Excursion_MFE': None, # Will be calculated on exit
                         'Max_Adverse_Excursion_MAE': None, # Will be calculated on exit
                        'Current_trailing_stop': None, # Need logic for trailing stops
                        'Trailing_stop_method': None,
                        'Trailing_stop_value': None,
                        'Stop_loss_price': None,


                        'Exit_Trend': None, 'Exit_signal_type': None, 'Exit_SMA20': None,
                        'Exit_RSI': None, 'Exit_RSI_MA': None, 'Exit_ATR': None, 'Exit_ADX': None,
                        'Exit_Volatility': None, 'Exit_Breakout_Detected': None,
                        'Exit_Breakdown_Detected': None, 'Exit_Bullish_Candlestick_Name': None,
                        'Exit_Bearish_Candlestick_Name': None, 'Exit_Bullish_Chart_Pattern_Detected': None,
                        'Exit_Bearish_Chart_Pattern_Detected': None, 'Exit_shares': None,
                        'Exit_cost': None, 'Exit_revenue': None, 'PnL_trade': None,
                        'Trade_type': None, 'Profit_loss': None, 'Exit_reason': None,
                        'Slippage': None, 'Commission_Fees': None, 'Trade_Duration': None,
                        'Exit_Order_Type': None
                    }

                    # --- Add debug logging for Entry columns here ---
                    logger.debug(f"DEBUG Entry Data Point for {instrument_key} at {timestamp}:")
                    debug_cols_to_check = [
                        'Trend', 'SMA20', 'RSI', 'RSI_MA', 'ATR', 'ADX', 'Volatility',
                        'Breakout_Detected', 'Breakdown_Detected',
                        # Corrected debug column names to match expected input data
                        'Bullish_Candlestick_Detected', 'Bearish_Candlestick_Detected',
                        'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
                        'Currency', 'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
                        'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE', 'Current_trailing_stop',
                        'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price', 'Exit_Trend',
                        'Exit_signal_type', 'Exit_SMA20', 'Exit_RSI', 'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX',
                        'Exit_Volatility', 'Exit_Breakout_Detected', 'Exit_Breakdown_Detected',
                        'Exit_Bullish_Candlestick_Name', 'Exit_Bearish_Candlestick_Name',
                        'Exit_Bullish_Chart_Pattern_Detected', 'Exit_Bearish_Chart_Pattern_Detected',
                        'Exit_cost'
                    ]
                    for col in debug_cols_to_check:
                         logger.debug(f"  {col}: {data_point.get(col, 'Column Not Found or None')}")
                    # --- End Debug Logging ---


                    logger.info(f"Executed BUY trade {trade_id} for {instrument_key} at {timestamp} @ {price} (Qty: {quantity_to_trade}). Costs: Slippage={entry_slippage:.4f}, Commission={entry_commission:.2f}. Capital left: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'BUY', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'slippage': entry_slippage, 'commission': entry_commission})
                else:
                    logger.warning(f"Insufficient capital ({self.current_capital:.2f}) to BUY {instrument_key} at {price} (Cost: {cost:.2f}). Skipping trade {trade_id}.")
                    self.debug_log.append({'type': 'SKIP_BUY_CAPITAL', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Insufficient Capital'})

            else:
                # Already in a position, maybe add to it or skip depending on strategy rules
                logger.debug(f"Skipping BUY signal for {instrument_key} at {timestamp}. Already in a position.")
                self.debug_log.append({'type': 'SKIP_BUY_POSITION', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Already in Position'})


        elif signal == 'SELL':
            # For backtesting, a 'SELL' signal usually means closing a long position or opening a short position
            # Let's assume 'SELL' means closing a long position if one exists for simplicity in this example.
            # For a shorting strategy, you'd need different logic.
            if instrument_key in self.positions and self.positions[instrument_key]['side'] == 'BUY':
                # Simulate selling to close a long position
                position = self.positions[instrument_key]
                quantity_to_sell = position['quantity']
                entry_price = position['entry_price']
                entry_time = position['entry_time']
                strategy_opened = position['strategy']
                open_trade_id = position['trade_id']


                revenue = quantity_to_sell * price
                self.current_capital += revenue

                # Calculate Profit/Loss (Gross PnL)
                gross_pnl = (price - entry_price) * quantity_to_sell # For long position

                # Calculate exit costs (slippage and commission on exit)
                exit_slippage = revenue * self.slippage_pct
                exit_commission = self.commission_per_trade # Fixed commission per trade


                self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

                # Calculate Net PnL
                total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
                total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
                pnl_trade = gross_pnl - total_slippage - total_commission

                # Calculate Trade Duration
                trade_duration = (timestamp - entry_time).total_seconds() if pd.notnull(timestamp) and pd.notnull(entry_time) else None

                # Record completed trade - Populate all desired columns
                trade_record = {
                    'open_trade_id': open_trade_id,
                    'close_trade_id': trade_id,
                    'instrument_key': instrument_key,
                    'instrument_type': position.get('instrument_type'), # Assuming this key is correct in position
                    'side': position.get('side'), # Side of the position being closed (BUY/LONG)
                    'quantity': quantity_to_sell, # Quantity closed
                    'entry_price': entry_price,
                    'entry_time': entry_time,
                    'exit_price': price,
                    'exit_time': timestamp,
                    'pnl': gross_pnl, # Keep gross PnL for reference
                    'strategy_opened': strategy_opened,
                    'strategy_closed': strategy_name, # Record which strategy/signal closed it

                    # --- Transfer Entry Details from Position ---
                    'Strategy_name': position.get('Strategy_name'),
                    'instrument_key': position.get('instrument_key'),
                    'name': position.get('name'),
                    'interval': position.get('interval'),
                    'Position_type': position.get('Position_type'),
                    'Entry_order_type': position.get('Entry_order_type'),
                    'Entry_timestamp': position.get('Entry_timestamp'),
                    'Entry_price_trigger': position.get('Entry_price_trigger'),
                    'Entry_price_execution': position.get('Entry_price_execution'),
                    'Entry_shares': position.get('Entry_shares'),
                    'Entry_cost': position.get('Entry_cost'),
                    'Entry_signal_type': position.get('Entry_signal_type'),
                    'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                    'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                    'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                    'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                    'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                    'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                    'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                    'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                    'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                    # Corrected keys to match the expected format
                    'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                    'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                    'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                    'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                    'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                    'Currency': position.get('Currency'), # Get Currency at Entry from Position
                    'Slippage_Entry': position.get('Slippage_Entry'), # Transfer entry slippage
                    'Commission_Fees_Entry': position.get('Commission_Fees_Entry'), # Transfer entry commission
                    'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                    'Risk_Amount': position.get('Risk_Amount'),
                    'Reward_Amount': position.get('Reward_Amount'),
                    'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                    'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),


                    # --- Exit-Specific Columns (Populated from data_point at Exit) ---
                    'Exit_Trend': data_point.get('Trend'), # Capture Trend at Exit
                    'Exit_signal_type': signal, # Signal that triggered the exit
                    'Exit_SMA20': data_point.get('SMA20'), # Capture SMA20 at Exit
                    'Exit_RSI': data_point.get('RSI'), # Capture RSI at Exit
                    'Exit_RSI_MA': data_point.get('RSI_MA'), # Capture RSI_MA at Exit
                    'Exit_ATR': data_point.get('ATR'), # Capture ATR at Exit
                    'Exit_ADX': data_point.get('ADX'), # Capture ADX at Exit
                    'Exit_Volatility': data_point.get('Volatility'), # Capture Volatility at Exit
                    'Exit_Breakout_Detected': data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Exit
                    'Exit_Breakdown_Detected': data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Exit
                    # Corrected keys to match the expected format
                    'Exit_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Exit
                    'Exit_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Exit
                    'Exit_Bullish_Chart_Pattern_Detected': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Exit
                    'Exit_Bearish_Chart_Pattern_Detected': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Exit
                    'Exit_shares': quantity_to_sell, # Shares exited
                    'Exit_cost': 0, # Assuming no cost to exit a long position
                    'Exit_revenue': revenue, # Gross revenue before fees
                    'PnL_trade': pnl_trade, # Net PnL after costs
                    'Trade_type': 'Long Close (Signal)', # Or 'Long' if trade refers to the full cycle
                    'Profit_loss': pnl_trade, # Update Profit_loss to net PnL
                    'Exit_reason': f'Signal_{signal}', # Reason for exit
                    'Exit_Order_Type': 'Market', # Set Exit_Order_Type to 'Market'

                    # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                    'Current_trailing_stop': data_point.get('Current_trailing_stop'), # Capture if calculated and available
                    'Trailing_stop_method': data_point.get('Trailing_stop_method'), # Capture if calculated and available
                    'Trailing_stop_value': data_point.get('Trailing_stop_value'), # Capture if calculated and available
                    'Stop_loss_price': data_point.get('Stop_loss_price'), # Capture if calculated and available


                    'Slippage': total_slippage, # Total slippage for the round trip
                    'Commission_Fees': total_commission, # Total commission for the round trip
                    'Trade_Duration': trade_duration,

                }
                self.completed_trades.append(trade_record)


                # Remove position
                del self.positions[instrument_key]

                logger.info(f"Executed SELL trade {trade_id} for {instrument_key} at {timestamp} @ {price} to CLOSE long position {open_trade_id}. Gross PnL: {gross_pnl:.2f}. Costs: Slippage={total_slippage:.4f}, Commission={total_commission:.2f}. Net PnL: {pnl_trade:.2f}. Capital left: {self.current_capital:.2f}")
                self.debug_log.append({'type': 'SELL_CLOSE_LONG', 'open_trade_id': open_trade_id, 'close_trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_sell, 'gross_pnl': gross_pnl, 'net_pnl': pnl_trade, 'strategy_closed': strategy_name})


            else:
                # No matching long position to close, or maybe a shorting signal
                # For this simple backtester, we'll just log and skip if no long position
                logger.debug(f"Skipping SELL signal for {instrument_key} at {timestamp}. No matching long position to close.")
                self.debug_log.append({'type': 'SKIP_SELL_NO_LONG', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'strategy': strategy_name, 'reason': 'No Long Position'})


    def run_backtest(self):
        """
        Runs the backtesting simulation bar by bar through the data.
        """
        logger.info("Starting backtest simulation...")

        # Group data by timestamp first, then iterate through timestamps
        # This processes all instruments available at a given time before moving to the next time.
        grouped_by_time = self.data.groupby('timestamp')

        for timestamp, time_slice_df in grouped_by_time:
            # Process data for all instruments available at this timestamp
            for index, data_point in time_slice_df.iterrows():
                instrument_key = data_point['instrument_key']
                current_price = data_point['close'] # Assume close price for execution

                # Debug capture
                self._debug_timestamps.append(timestamp)
                self._debug_close_values.append(current_price)
                self._debug_validity.append(pd.notna(current_price))

                # Ensure current_price is valid for trading
                if pd.isna(current_price):
                    logger.debug(f"Skipping signal generation for {instrument_key} at {timestamp} due to invalid close price ({current_price}).")
                    self.debug_log.append({'type': 'SKIP_SIGNAL_PRICE_NAN', 'instrument': instrument_key, 'time': timestamp, 'reason': 'Invalid Price'})
                    continue # Skip this data point if price is invalid

                # Check for signals from all active strategies for this data point
                # Pass the single data_point (as a Series) to the strategy
                # The strategy is expected to handle a single row/Series or convert it internally
                # For the backtester's execute_trade, we need the Series directly.

                for strategy_name, strategy_instance in self.active_strategies_instances.items():
                    try:
                        # Pass the single data_point Series to the strategy's generate_signal
                        # Ensure data_point is passed as a DataFrame slice if strategy expects DataFrame
                        signal = strategy_instance.generate_signal(pd.DataFrame([data_point]))
                        # Ensure signal is a string, handle potential None returns gracefully
                        signal = str(signal).upper() if signal is not None else 'HOLD'

                        if signal in ['BUY', 'SELL']:
                            # Generate a unique trade ID for this potential trade using the bar's timestamp
                            trade_id = self.generate_trade_id(timestamp) # Pass the historical timestamp
                            # Pass the original data_point Series to execute_trade
                            self.execute_trade(trade_id, instrument_key, timestamp, signal, strategy_name, current_price, data_point)

                    except Exception as e:
                        logger.error(f"Error generating signal for {instrument_key} at {timestamp} using strategy '{strategy_name}': {e}", exc_info=True)
                        self.debug_log.append({'type': 'STRATEGY_ERROR', 'instrument': instrument_key, 'time': timestamp, 'strategy': strategy_name, 'error': str(e)})


        # After iterating through all data, close any remaining open positions
        self.close_all_positions(self.data['timestamp'].max()) # Use the timestamp of the last data point as exit time

        logger.info("Backtest simulation completed.")
        logger.info(f"Final Capital: {self.current_capital:.2f}")
        logger.info(f"Number of completed trades: {len(self.completed_trades)}")
        logger.info(f"Number of open positions remaining: {len(self.positions)}")

        # Return completed trades as a DataFrame for analysis
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_df.columns:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

            # Ensure numeric columns are numeric
            numeric_cols = [
                'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
            ]
            for col in numeric_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')


            return completed_trades_df
        else:
            logger.warning("No completed trades recorded. Returning empty DataFrame.")
            return pd.DataFrame() # Return empty DataFrame if no trades


    def close_all_positions(self, exit_timestamp: datetime):
        """
        Closes all remaining open positions at the specified exit timestamp.
        Assumes closing at the price of the last available bar for each instrument.
        """
        logger.info(f"Closing all remaining {len(self.positions)} open positions at {exit_timestamp}...")

        # Get the last known price and data point for each instrument with an open position
        last_data_points = self.data.groupby('instrument_key').tail(1).set_index('instrument_key')
        last_prices = last_data_points['close'].to_dict()


        positions_to_close = list(self.positions.keys()) # Iterate over a copy

        for instrument_key in positions_to_close:
            if instrument_key in self.positions: # Check if position still exists (wasn't closed by a signal just before the end)
                position = self.positions[instrument_key]
                closing_price = last_prices.get(instrument_key, np.nan) # Get last price, default to NaN if instrument not found

                # Get the last data point for the instrument to capture exit conditions
                last_data_point = last_data_points.get(instrument_key, pd.Series({})) # Use empty Series if no data found


                if pd.notna(closing_price):
                    trade_id = self.generate_trade_id(exit_timestamp) # Generate a new trade ID for the closing trade using exit_timestamp
                    # Simulate selling to close a long position
                    quantity_to_sell = position['quantity']
                    entry_price = position['entry_price']
                    entry_time = position['entry_time']
                    strategy_opened = position['strategy']
                    open_trade_id = position['trade_id']


                    revenue = quantity_to_sell * closing_price
                    self.current_capital += revenue

                    # Calculate Profit/Loss (Gross PnL)
                    gross_pnl = (closing_price - entry_price) * quantity_to_sell # For long position

                    # Calculate exit costs (slippage and commission on exit)
                    exit_slippage = revenue * self.slippage_pct
                    exit_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

                    # Calculate Net PnL
                    total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
                    total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
                    pnl_trade = gross_pnl - total_slippage - total_commission


                    # Calculate Trade Duration
                    trade_duration = (exit_timestamp - entry_time).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(entry_time) else None


                    # Record completed trade - Populate all desired columns
                    trade_record = {
                        'open_trade_id': open_trade_id,
                        'close_trade_id': trade_id, # New ID for closing
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'), # Use .get() for safety
                        'side': position.get('side'), # Side of the position being closed (BUY/LONG)
                        'quantity': quantity_to_sell, # Quantity closed
                        'entry_price': entry_price,
                        'entry_time': entry_time,
                        'exit_price': closing_price,
                        'exit_time': exit_timestamp, # Use the provided exit timestamp
                        'pnl': gross_pnl, # Keep gross PnL for reference
                        'strategy_opened': strategy_opened,
                        'strategy_closed': 'Backtester_Forced_Close', # Indicate it was closed by the backtester

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        # Corrected keys to match the expected format
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': position.get('Slippage_Entry'), # Transfer entry slippage
                        'Commission_Fees_Entry': position.get('Commission_Fees_Entry'), # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),


                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close', # Indicate forced close
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        # Corrected keys to match the expected format
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': quantity_to_sell, # Shares exited
                        'Exit_cost': 0, # Assuming no cost to exit a long position
                        'Exit_revenue': revenue, # Gross revenue before fees
                        'PnL_trade': pnl_trade, # Net PnL after costs
                        'Trade_type': 'Long Close (Forced)', # Or 'Long' if trade refers to the full cycle
                        'Profit_loss': pnl_trade, # Update Profit_loss to net PnL
                        'Exit_reason': 'Backtester_Forced_Close', # Reason for exit
                        'Exit_Order_Type': 'Market', # Set Exit_Order_Type to 'Market' or 'Forced'

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': last_data_point.get('Current_trailing_stop'), # Capture if calculated and available
                        'Trailing_stop_method': last_data_point.get('Trailing_stop_method'), # Capture if calculated and available
                        'Trailing_stop_value': last_data_point.get('Trailing_stop_value'), # Capture if calculated and available
                        'Stop_loss_price': last_data_point.get('Stop_loss_price'), # Capture if calculated and available


                        'Slippage': total_slippage, # Total slippage for the round trip
                        'Commission_Fees': total_commission, # Total commission for the round trip
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(trade_record)

                    # Remove position
                    del self.positions[instrument_key]

                    logger.info(f"Closed remaining position {open_trade_id} for {instrument_key} at {exit_timestamp} @ {closing_price}. Gross PnL: {gross_pnl:.2f}. Costs: Slippage={total_slippage:.4f}, Commission={total_commission:.2f}. Net PnL: {pnl_trade:.2f}. Capital: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'FORCED_CLOSE', 'open_trade_id': open_trade_id, 'close_trade_id': trade_id, 'instrument': instrument_key, 'time': exit_timestamp, 'price': closing_price, 'quantity': quantity_to_sell, 'gross_pnl': gross_pnl, 'net_pnl': pnl_trade, 'strategy_closed': 'Backtester_Forced_Close'})

                else:
                    logger.warning(f"Could not find last price for {instrument_key}. Cannot close position {position['trade_id']}. Logging as unresolved.")
                    # Log as an unresolved position or assume zero PnL

                    # Calculate Trade Duration even if closing price is NaN
                    trade_duration = (exit_timestamp - position.get('entry_time')).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(position.get('entry_time')) else None

                    # Transfer known costs
                    entry_slippage = position.get('Slippage_Entry', 0)
                    entry_commission = position.get('Commission_Fees_Entry', 0)


                    unresolved_trade_record = {
                        'open_trade_id': position.get('trade_id'),
                        'close_trade_id': None, # No closing trade ID
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'),
                        'side': position.get('side'),
                        'quantity': position.get('quantity'),
                        'entry_price': position.get('entry_price'),
                        'entry_time': position.get('entry_time'),
                        'exit_price': None, # No exit price
                        'exit_time': exit_timestamp, # Use the requested exit timestamp
                        'pnl': 0, # Assume zero Gross PnL if cannot close
                        'strategy_opened': position.get('strategy'),
                        'strategy_closed': 'Backtester_Forced_Close_Error', # Indicate error

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        # Corrected keys to match the expected format
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': entry_slippage, # Transfer entry slippage
                        'Commission_Fees_Entry': entry_commission, # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),


                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close_Error', # Indicate forced close error
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        # Corrected keys to match the expected format
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': position.get('quantity'), # Shares that were supposed to be exited
                        'Exit_cost': 0,
                        'Exit_revenue': 0, # Assuming zero revenue if cannot close
                        'PnL_trade': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Profit_loss': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Exit_reason': 'Backtester_Forced_Close_Error: No_Last_Price', # Reason for exit
                        'Exit_Order_Type': None, # Could not execute exit order

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': last_data_point.get('Current_trailing_stop'), # Capture if calculated and available
                        'Trailing_stop_method': last_data_point.get('Trailing_stop_method'), # Capture if calculated and available
                        'Trailing_stop_value': last_data_point.get('Trailing_stop_value'), # Capture if calculated and available
                        'Stop_loss_price': last_data_point.get('Stop_loss_price'), # Capture if calculated and available

                        'Slippage': entry_slippage, # Only entry slippage is known
                        'Commission_Fees': entry_commission, # Only entry commission is known
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(unresolved_trade_record)

                    # Remove position even if it couldn't be closed properly to prevent it from being processed again
                    del self.positions[instrument_key]


                    self.debug_log.append({'type': 'FORCED_CLOSE_ERROR', 'open_trade_id': position['trade_id'], 'instrument': instrument_key, 'time': exit_timestamp, 'reason': 'Last price not available'})


        logger.info("All remaining positions closed.")


    def analyze_backtest_results(self):
        """
        Analyzes the completed trades and provides performance metrics.
        Returns a DataFrame summarizing the analysis.
        """
        logger.info("Analyzing backtest results...")

        if not self.completed_trades:
            logger.warning("No completed trades to analyze.")
            return pd.DataFrame({'Message': ['No completed trades to analyze.']})

        # 1. Access the self.completed_trades list and Create a pandas DataFrame
        trades_df = pd.DataFrame(self.completed_trades)

        # 3. Ensure that relevant columns are converted to appropriate numeric types
        numeric_cols = [
            'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
            'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
            'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
            'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
            'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
            'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
            'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
            'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Include other potentially numeric cols
        ]
        for col in numeric_cols:
            if col in trades_df.columns:
                trades_df[col] = pd.to_numeric(trades_df[col], errors='coerce')

        # Handle potential NaN values during conversion - drop rows where PnL_trade (or pnl) is NaN
        pnl_col_for_analysis = 'PnL_trade' if 'PnL_trade' in trades_df.columns else 'pnl'
        if pnl_col_for_analysis in trades_df.columns:
            # Only consider trades with a valid PnL for core analysis metrics
            trades_df_analysis = trades_df.dropna(subset=[pnl_col_for_analysis]).copy() # Use .copy() to avoid SettingWithCopyWarning
        else:
            logger.warning("Neither 'PnL_trade' nor 'pnl' column found for analysis.")
            return pd.DataFrame({'Message': ['No PnL column found for analysis.']})


        if trades_df_analysis.empty:
            logger.warning("No valid trades after numeric conversion/dropna for analysis. Analysis stopped.")
            return pd.DataFrame({'Message': ['No valid trades after numeric conversion/dropna for analysis.']})


        # 4. Update the calculation of basic performance metrics using 'PnL_trade'
        total_trades = len(trades_df_analysis)
        total_pnl = trades_df_analysis[pnl_col_for_analysis].sum()

        winning_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] > 0]
        losing_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] < 0]
        breakeven_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] == 0]

        num_winning = len(winning_trades)
        num_losing = len(losing_trades)
        num_breakeven = len(breakeven_trades)

        win_rate = (num_winning / total_trades) * 100 if total_trades > 0 else 0
        avg_win = winning_trades[pnl_col_for_analysis].mean() if num_winning > 0 else 0
        avg_loss = losing_trades[pnl_col_for_analysis].mean() if num_losing > 0 else 0
        expectancy = (win_rate / 100) * avg_win + ((100 - win_rate) / 100) * avg_loss if total_trades > 0 else 0

        # 5. Update Max Drawdown calculation to use 'PnL_trade' and sort by exit time
        # Calculate cumulative PnL and then cumulative capital
        trades_df_analysis = trades_df_analysis.sort_values(by='exit_time') # Sort by exit time for cumulative calculation

        trades_df_analysis['cumulative_pnl'] = trades_df_analysis[pnl_col_for_analysis].cumsum()

        # Add initial capital to cumulative PnL
        trades_df_analysis['cumulative_capital'] = self.initial_capital + trades_df_analysis['cumulative_pnl']

        # Calculate peak capital up to each point
        trades_df_analysis['peak_capital'] = trades_df_analysis['cumulative_capital'].cummax()

        # Calculate drawdown at each point
        trades_df_analysis['drawdown'] = trades_df_analysis['peak_capital'] - trades_df_analysis['cumulative_capital']

        # Calculate percentage drawdown
        # Avoid division by zero if peak_capital is 0 or None
        trades_df_analysis['pct_drawdown'] = trades_df_analysis.apply(
            lambda row: (row['drawdown'] / row['peak_capital']) * 100 if row['peak_capital'] > 0 and pd.notna(row['peak_capital']) else 0,
            axis=1
        )


        max_drawdown_amount = trades_df_analysis['drawdown'].max() if not trades_df_analysis.empty else 0
        max_drawdown_pct = trades_df_analysis['pct_drawdown'].max() if not trades_df_analysis.empty else 0


        # 6. Update analysis summary metric names
        analysis_summary = {
            'Metric': [
                'Initial Capital',
                'Final Capital',
                'Total PnL (Net)', # Indicate Net PnL
                'Total Trades',
                'Winning Trades (Net)', # Indicate Net PnL
                'Losing Trades (Net)', # Indicate Net PnL
                'Breakeven Trades (Net)', # Indicate Net PnL
                'Win Rate (%) (Net PnL)', # Indicate Net PnL
                'Average Win (Net)', # Indicate Net PnL
                'Average Loss (Net)', # Indicate Net PnL
                'Expectancy per Trade (Net)', # Indicate Net PnL
                'Max Drawdown (Amount)',
                'Max Drawdown (%)',
            ],
            'Value': [
                self.initial_capital,
                self.current_capital,
                round(total_pnl, 2), # Format to 2 decimal places
                total_trades,
                num_winning,
                num_losing,
                num_breakeven,
                round(win_rate, 2), # Format to 2 decimal places
                round(avg_win, 2),
                round(avg_loss, 2),
                round(expectancy, 2),
                round(max_drawdown_amount, 2),
                round(max_drawdown_pct, 2),
            ]
        }

        analysis_df = pd.DataFrame(analysis_summary)

        logger.info("Backtest analysis completed.")
        # You can print the analysis_df here or return it
        # print("\n--- Backtest Analysis Summary ---")
        # display(analysis_df) # Use display for notebooks

        # 8. Ensure the method returns the updated analysis summary DataFrame
        return analysis_df

    def get_completed_trades(self):
        """Returns a DataFrame of completed trades."""
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

                    # Ensure numeric columns are numeric
                    numeric_cols = [
                        'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                        'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                        'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                        'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                        'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                        'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                        'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                        'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                        'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
                    ]
                    for col in numeric_cols:
                        if col in completed_trades_df.columns:
                            completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')

                    return completed_trades_df
                else:
                    return pd.DataFrame() # Return empty DataFrame if no trades

Logging level set to DEBUG for test.


## Implement stop-loss/take-profit check

### Subtask:
Add a loop or check within `run_backtest` to evaluate stop-loss and take-profit conditions for open positions on each bar before processing new signals.


**Reasoning**:
Add the `close_position_on_trigger` method to handle closing trades based on stop-loss or take-profit triggers. This method consolidates the logic for updating capital and recording completed trades, similar to the existing closing logic in `execute_trade`.



In [4]:
# _1113_6BacktesterV6.py

import pandas as pd
import numpy as np
import logging
from datetime import datetime
import uuid
import sys
# from _012_instruments import get_instrument_type
# --- Logging Configuration ---
logger = logging.getLogger(__name__)



logger.setLevel(logging.DEBUG)

# Also ensure the root logger has a handler and is set to DEBUG,
# in case basicConfig was called elsewhere previously.
if not logging.root.handlers:
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
else:
    # If handlers exist, ensure at least one handler's level is DEBUG
    # and the root logger's level is DEBUG
    logging.root.setLevel(logging.DEBUG)
    handler_found = False
    for handler in logging.root.handlers:
        if isinstance(handler, logging.StreamHandler) and handler.stream in [sys.stdout, sys.stderr]:
            handler.setLevel(logging.DEBUG)
            handler_found = True
    # If no suitable handler is found (e.g., only file handlers), add a StreamHandler
    if not handler_found:
         stream_handler = logging.StreamHandler(sys.stdout)
         stream_handler.setLevel(logging.DEBUG)
         formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(message)s')
         stream_handler.setFormatter(formatter)
         logging.root.addHandler(stream_handler)


print("Logging level set to DEBUG for test.")


class BacktesterV3:
    """
    A simple backtesting engine for evaluating trading strategies.
    Processes historical data bar by bar, generates signals, and simulates trades.
    """
    def __init__(self, data: pd.DataFrame, instrument_keys: list, active_strategies_instances: dict, initial_capital: float):
        """
        Includes the same parameters as the original __init__

        Initializes the Backtester.

        Args:
            data: A pandas DataFrame containing historical market data for all instruments,
                expected to have columns like 'timestamp', 'instrument_key',
                'open', 'high', 'low', 'close', 'volume', etc. It is also expected
                to contain pre-calculated indicator and pattern columns used by
                the strategies and for recording trade details.
            instrument_keys: A list of unique instrument keys present in the data.
            active_strategies_instances: A dictionary where keys are strategy names
                                        (strings) and values are instantiated strategy
                                        objects with a `generate_signal(data_point)` method.
            initial_capital: The starting capital for the backtest simulation.
        """
        if data is None or data.empty:
            raise ValueError("Input data DataFrame is None or empty.")
        if not isinstance(data, pd.DataFrame):
            raise TypeError("Input 'data' must be a pandas DataFrame.")
        if data.index.name is not None:
            logger.warning("Input data index is not None. Consider resetting the index before passing to Backtester.")


        # Ensure essential columns are present and sorted
        required_columns = ['timestamp', 'instrument_key', 'open', 'high', 'low', 'close']
        # Define columns expected to be in the input data for recording trade details.
        # These are typically pre-calculated indicators or pattern detection results.
        entry_exit_data_columns_expected = [
            'Trend', 'SMA20', 'RSI', 'RSIMA', 'ATR', 'ADX', 'Volatility',
            'Breakout_Detected', 'Breakdown_Detected', 'Bullish_Candlestick_Detected',
            'Bearish_Candlestick_Detected', 'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
            'name', 'interval', 'Currency',
            'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
            'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price'
        ]

        # The backtester expects these columns to be pre-calculated and provided in the input data.
        # Strategies generate signals based on these columns, and their values at the time of
        # entry and exit are recorded in the completed_trades DataFrame.


        # Perform a relaxed check: log a warning if potential entry/exit columns from data are missing
        missing_data_cols = [col for col in entry_exit_data_columns_expected if col not in data.columns]
        if missing_data_cols:
            logger.warning(f"Input data is missing expected indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records. Ensure your data preparation includes these columns if strategies or analysis depend on them.")


        # Ensure mandatory required columns are present
        missing_required = [col for col in required_columns if col not in data.columns]
        if missing_required:
            raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")


        # Ensure timestamp is datetime and sorted
        try:
            if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
                data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', utc=True) # Convert to UTC
            # Drop rows where timestamp conversion failed
            data = data.dropna(subset=['timestamp'])
            # Sort by timestamp and then instrument_key to process bars chronologically per instrument
            self.data = data.sort_values(by=['timestamp', 'instrument_key']).reset_index(drop=True)
        except Exception as e:
            raise ValueError(f"Error processing timestamp column in data: {e}")


        self.instrument_keys = instrument_keys
        self.active_strategies_instances = active_strategies_instances
        self.initial_capital = initial_capital

        # --- Backtesting State Variables ---
        self.current_capital = initial_capital
        self.positions = {}  # Dictionary to track open positions {instrument_key: {...entry details...}}
        self.completed_trades = [] # List to store completed trades
        self.trade_id_counter = 0 # Simple counter for trade IDs
        self.debug_log = [] # List to store debug information

        # Debug lists to capture values
        self._debug_timestamps = []
        self._debug_close_values = []
        self._debug_validity = []

        # Simple Slippage and Commission model (can be customized)
        self.slippage_pct = 0.001  # 0.1% slippage per trade
        self.commission_per_trade = 0.01 # $0.01 fixed commission per trade


        logger.info(f"BacktesterV2 initialized with {len(self.instrument_keys)} instruments and {len(self.active_strategies_instances)} active strategies.")
        logger.info(f"Initial Capital: {self.initial_capital}")
        logger.info(f"Data shape for backtesting: {self.data.shape}")


    def generate_trade_id(self, timestamp: datetime):
        """Generates a unique trade ID using a provided timestamp."""
        # Using microseconds to increase the chance of uniqueness
        return timestamp.strftime('%Y%m%d%H%M%S%f')


    def execute_trade(self, trade_id: str, instrument_key: str, timestamp: datetime, signal: str, strategy_name: str, price: float, data_point: pd.Series):
        """
        Simulates executing a trade based on a signal.

        Args:
            trade_id: Unique identifier for the trade.
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the trade execution (bar close time).
            signal: The trading signal ('BUY' or 'SELL').
            strategy_name: The name of the strategy generating the signal.
            price: The execution price (typically the close price of the bar).
            data_point: The pandas Series representing the data row for this bar. This Series
                        is expected to contain pre-calculated indicator and pattern data
                        used for entry/exit conditions and recording.
        """
        # Determine instrument type to handle lot size/quantity logic
        # instrument_type = get_instrument_type(instrument_key) # Removed due to import error
        instrument_type = 'Unknown' # Placeholder


        # Simple fixed quantity logic (can be replaced with dynamic position sizing)
        quantity_to_trade = 1 # Example: trade 1 unit/lot

        if signal == 'BUY':
            # Check if we already have a position in this instrument (optional, depending on strategy)
            if instrument_key not in self.positions:
                # Simulate buying
                cost = quantity_to_trade * price
                # Check if we have enough capital
                if self.current_capital >= cost:
                    self.current_capital -= cost

                    # Calculate entry costs (slippage and commission on entry)
                    entry_slippage = cost * self.slippage_pct
                    entry_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (entry_slippage + entry_commission) # Deduct costs from capital

                    # Capture entry-specific details from the data_point and other variables
                    self.positions[instrument_key] = {
                        'quantity': quantity_to_trade,
                        'entry_price': price, # This is the execution price for this simple model
                        'entry_time': timestamp,
                        'strategy': strategy_name,
                        'trade_id': trade_id,
                        'instrument_type': instrument_type, # Store instrument type (now placeholder)
                        'side': 'BUY', # Store trade side

                        # --- Entry-Specific Columns (Populated from data_point at Entry) ---
                        'Strategy_name': strategy_name,
                        'instrument_key': instrument_key,
                        'name': data_point.get('name'), # Use .get() to avoid errors if column is missing
                        'interval': data_point.get('interval'),
                        'Position_type': 'Long', # Assuming BUY means Long position
                        'Entry_order_type': 'Market', # Assuming market order execution on close
                        'Entry_timestamp': timestamp,
                        'Entry_price_trigger': None, # Not explicitly handled in this simple model
                        'Entry_price_execution': price,
                        'Entry_shares': quantity_to_trade, # Using quantity_to_trade as shares
                        'Entry_cost': cost, # Gross cost before fees
                        'Entry_signal_type': signal, # Ensure signal is captured
                        'Entry_Trend': data_point.get('Trend'), # Capture Trend at Entry
                        'Entry_SMA20': data_point.get('SMA20'), # Capture SMA20 at Entry
                        'Entry_RSI': data_point.get('RSI'), # Capture RSI at Entry
                        'Entry_RSI_MA': data_point.get('RSI_MA'), # Capture RSI_MA at Entry
                        'Entry_ATR': data_point.get('ATR'), # Capture ATR at Entry
                        'Entry_ADX': data_point.get('ADX'), # Capture ADX at Entry
                        'Entry_Volatility': data_point.get('Volatility'), # Capture Volatility at Entry
                        'Entry_Breakout_Detected': data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Entry
                        'Entry_Breakdown_Detected': data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Entry
                        # Corrected column names to match expected input data
                        'Entry_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Entry
                        'Entry_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Entry
                        'Entry_Bullish_Chart_Pattern_Name': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Entry
                        'Entry_Bearish_Chart_Pattern_Name': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Entry
                        'Instrument_Type': instrument_type, # Store instrument type (now placeholder)
                        'Currency': data_point.get('Currency'),
                        'Slippage_Entry': entry_slippage, # Store entry slippage
                        'Commission_Fees_Entry': entry_commission, # Store entry commission

                        # Placeholder for other entry-specific details that might be calculated by strategy (e.g., initial stop/target)
                        'Initial_Stop_Loss_Distance (%)': data_point.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': data_point.get('Risk_Amount'),
                        'Reward_Amount': data_point.get('Reward_Amount'),


                        # Placeholders for exit/other info that will be filled on close
                        # These fields are included here so the structure is consistent for retrieval on exit,
                        # even though their values are None at the time of entry.
                         'Max_Favorable_Excursion_MFE': None, # Will be calculated on exit
                         'Max_Adverse_Excursion_MAE': None, # Will be calculated on exit
                        'Current_trailing_stop': None, # Need logic for trailing stops
                        'Trailing_stop_method': None,
                        'Trailing_stop_value': None,
                        'Stop_loss_price': None,


                        'Exit_Trend': None, 'Exit_signal_type': None, 'Exit_SMA20': None,
                        'Exit_RSI': None, 'Exit_RSI_MA': None, 'Exit_ATR': None, 'Exit_ADX': None,
                        'Exit_Volatility': None, 'Exit_Breakout_Detected': None,
                        'Exit_Breakdown_Detected': None, 'Exit_Bullish_Candlestick_Name': None,
                        'Exit_Bearish_Candlestick_Name': None, 'Exit_Bullish_Chart_Pattern_Detected': None,
                        'Exit_Bearish_Chart_Pattern_Detected': None, 'Exit_shares': None,
                        'Exit_cost': None, 'Exit_revenue': None, 'PnL_trade': None,
                        'Trade_type': None, 'Profit_loss': None, 'Exit_reason': None,
                        'Slippage': None, 'Commission_Fees': None, 'Trade_Duration': None,
                        'Exit_Order_Type': None
                    }

                    # --- Add debug logging for Entry columns here ---
                    logger.debug(f"DEBUG Entry Data Point for {instrument_key} at {timestamp}:")
                    debug_cols_to_check = [
                        'Trend', 'SMA20', 'RSI', 'RSI_MA', 'ATR', 'ADX', 'Volatility',
                        'Breakout_Detected', 'Breakdown_Detected',
                        # Corrected debug column names to match expected input data
                        'Bullish_Candlestick_Detected', 'Bearish_Candlestick_Detected',
                        'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
                        'Currency', 'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
                        'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE', 'Current_trailing_stop',
                        'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price', 'Exit_Trend',
                        'Exit_signal_type', 'Exit_SMA20', 'Exit_RSI', 'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX',
                        'Exit_Volatility', 'Exit_Breakout_Detected', 'Exit_Breakdown_Detected',
                        'Exit_Bullish_Candlestick_Name', 'Exit_Bearish_Candlestick_Name',
                        'Exit_Bullish_Chart_Pattern_Detected', 'Exit_Bearish_Chart_Pattern_Detected',
                        'Exit_cost'
                    ]
                    for col in debug_cols_to_check:
                         logger.debug(f"  {col}: {data_point.get(col, 'Column Not Found or None')}")
                    # --- End Debug Logging ---


                    logger.info(f"Executed BUY trade {trade_id} for {instrument_key} at {timestamp} @ {price} (Qty: {quantity_to_trade}). Costs: Slippage={entry_slippage:.4f}, Commission={entry_commission:.2f}. Capital left: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'BUY', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'slippage': entry_slippage, 'commission': entry_commission})
                else:
                    logger.warning(f"Insufficient capital ({self.current_capital:.2f}) to BUY {instrument_key} at {price} (Cost: {cost:.2f}). Skipping trade {trade_id}.")
                    self.debug_log.append({'type': 'SKIP_BUY_CAPITAL', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Insufficient Capital'})

            else:
                # Already in a position, maybe add to it or skip depending on strategy rules
                logger.debug(f"Skipping BUY signal for {instrument_key} at {timestamp}. Already in a position.")
                self.debug_log.append({'type': 'SKIP_BUY_POSITION', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Already in Position'})


        elif signal == 'SELL':
            # For backtesting, a 'SELL' signal usually means closing a long position or opening a short position
            # Let's assume 'SELL' means closing a long position if one exists for simplicity in this example.
            # For a shorting strategy, you'd need different logic.
            if instrument_key in self.positions and self.positions[instrument_key]['side'] == 'BUY':
                # Use the dedicated close method
                self.close_position_on_trigger(instrument_key, timestamp, price, 'Signal_SELL', data_point)

            else:
                # No matching long position to close, or maybe a shorting signal
                # For this simple backtester, we'll just log and skip if no long position
                logger.debug(f"Skipping SELL signal for {instrument_key} at {timestamp}. No matching long position to close.")
                self.debug_log.append({'type': 'SKIP_SELL_NO_LONG', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'strategy': strategy_name, 'reason': 'No Long Position'})

    def close_position_on_trigger(self, instrument_key: str, timestamp: datetime, closing_price: float, reason: str, exit_data_point: pd.Series):
        """
        Closes an open position for a specific instrument due to a trigger (SL, TP, etc.).

        Args:
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the closure (bar close time).
            closing_price: The price at which the position is closed.
            reason: The reason for closing ('Stop Loss', 'Take Profit', 'Signal_SELL', 'Forced_Close').
            exit_data_point: The pandas Series representing the data row for this bar at exit.
        """
        if instrument_key not in self.positions:
            logger.warning(f"Attempted to close non-existent position for {instrument_key} at {timestamp} (Reason: {reason}).")
            return

        position = self.positions[instrument_key]

        quantity_to_sell = position['quantity']
        entry_price = position['entry_price']
        entry_time = position['entry_time']
        strategy_opened = position['strategy']
        open_trade_id = position['trade_id']
        side = position['side']

        # Ensure side is 'BUY' for long position closure logic
        if side != 'BUY':
             logger.warning(f"Attempted to close non-long position for {instrument_key} at {timestamp} (Side: {side}). Skipping.")
             return

        revenue = quantity_to_sell * closing_price
        self.current_capital += revenue

        # Calculate Profit/Loss (Gross PnL) - For long positions
        gross_pnl = (closing_price - entry_price) * quantity_to_sell

        # Calculate exit costs (slippage and commission on exit)
        exit_slippage = revenue * self.slippage_pct
        exit_commission = self.commission_per_trade # Fixed commission per trade

        self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

        # Calculate Net PnL
        total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
        total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
        pnl_trade = gross_pnl - total_slippage - total_commission

        # Calculate Trade Duration
        trade_duration = (timestamp - entry_time).total_seconds() if pd.notnull(timestamp) and pd.notnull(entry_time) else None

        # Generate a unique trade ID for the closing trade
        close_trade_id = self.generate_trade_id(timestamp)

        # Record completed trade - Populate all desired columns
        trade_record = {
            'open_trade_id': open_trade_id,
            'close_trade_id': close_trade_id,
            'instrument_key': instrument_key,
            'instrument_type': position.get('instrument_type'),
            'side': side,
            'quantity': quantity_to_sell,
            'entry_price': entry_price,
            'entry_time': entry_time,
            'exit_price': closing_price,
            'exit_time': timestamp,
            'pnl': gross_pnl, # Keep gross PnL for reference
            'strategy_opened': strategy_opened,
            'strategy_closed': 'Backtester_Trigger' if reason not in ['Signal_SELL', 'Forced_Close'] else reason, # Indicate trigger closure or signal close
            'Position_type': position.get('Position_type'),
            'Entry_order_type': position.get('Entry_order_type'),
            'Entry_timestamp': position.get('Entry_timestamp'),
            'Entry_price_trigger': position.get('Entry_price_trigger'),
            'Entry_price_execution': position.get('Entry_price_execution'),
            'Entry_shares': position.get('Entry_shares'),
            'Entry_cost': position.get('Entry_cost'),
            'Entry_signal_type': position.get('Entry_signal_type'),
            'Entry_Trend': position.get('Entry_Trend'),
            'Entry_SMA20': position.get('Entry_SMA20'),
            'Entry_RSI': position.get('Entry_RSI'),
            'Entry_RSI_MA': position.get('Entry_RSI_MA'),
            'Entry_ATR': position.get('Entry_ATR'),
            'Entry_ADX': position.get('Entry_ADX'),
            'Entry_Volatility': position.get('Entry_Volatility'),
            'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'),
            'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'),
            'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'),
            'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'),
            'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'),
            'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'),
            'Instrument_Type': position.get('Instrument_Type'),
            'Currency': position.get('Currency'),
            'Slippage_Entry': position.get('Slippage_Entry'),
            'Commission_Fees_Entry': position.get('Commission_Fees_Entry'),
            'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
            'Risk_Amount': position.get('Risk_Amount'),
            'Reward_Amount': position.get('Reward_Amount'),
            'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
            'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),

            # --- Exit-Specific Columns (Populated from exit_data_point) ---
            'Exit_Trend': exit_data_point.get('Trend'),
            'Exit_signal_type': reason, # The reason is the "signal" for exit
            'Exit_SMA20': exit_data_point.get('SMA20'),
            'Exit_RSI': exit_data_point.get('RSI'),
            'Exit_RSI_MA': exit_data_point.get('RSI_MA'),
            'Exit_ATR': exit_data_point.get('ATR'),
            'Exit_ADX': exit_data_point.get('ADX'),
            'Exit_Volatility': exit_data_point.get('Volatility'),
            'Exit_Breakout_Detected': exit_data_point.get('Breakout_Detected'),
            'Exit_Breakdown_Detected': exit_data_point.get('Breakdown_Detected'),
            'Exit_Bullish_Candlestick_Name': exit_data_point.get('Bullish_Candlestick_Detected'),
            'Exit_Bearish_Candlestick_Name': exit_data_point.get('Bearish_Candlestick_Detected'),
            'Exit_Bullish_Chart_Pattern_Detected': exit_data_point.get('Detected_Bullish_Chart_Pattern_Name'),
            'Exit_Bearish_Chart_Pattern_Detected': exit_data_point.get('Detected_Bearish_Chart_Pattern_Name'),
            'Exit_shares': quantity_to_sell,
            'Exit_cost': 0, # Assuming no cost to exit a long position
            'Exit_revenue': revenue,
            'PnL_trade': pnl_trade,
            'Trade_type': f'Long Close ({reason})', # Indicate the closure type
            'Profit_loss': pnl_trade,
            'Exit_reason': reason,
            'Exit_Order_Type': 'Market' if reason != 'Forced_Close_Error: No_Last_Price' else None, # Assume market order unless error

            # Placeholder for other exit-specific details
            'Current_trailing_stop': exit_data_point.get('Current_trailing_stop'),
            'Trailing_stop_method': exit_data_point.get('Trailing_stop_method'),
            'Trailing_stop_value': exit_data_point.get('Trailing_stop_value'),
            'Stop_loss_price': exit_data_point.get('Stop_loss_price'),

            'Slippage': total_slippage,
            'Commission_Fees': total_commission,
            'Trade_Duration': trade_duration,
        }
        self.completed_trades.append(trade_record)

        # Remove position
        del self.positions[instrument_key]

        logger.info(f"Closed position {open_trade_id} for {instrument_key} at {timestamp} @ {closing_price} (Reason: {reason}). Net PnL: {pnl_trade:.2f}. Capital left: {self.current_capital:.2f}")
        self.debug_log.append({'type': 'CLOSE_TRIGGER', 'open_trade_id': open_trade_id, 'close_trade_id': close_trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': closing_price, 'quantity': quantity_to_sell, 'net_pnl': pnl_trade, 'reason': reason})


    def run_backtest(self):
        """
        Runs the backtesting simulation bar by bar through the data.
        """
        logger.info("Starting backtest simulation...")

        # Group data by timestamp first, then iterate through timestamps
        grouped_by_time = self.data.groupby('timestamp')

        for timestamp, time_slice_df in grouped_by_time:
            logger.debug(f"Processing timestamp: {timestamp}")

            # --- Check for Stop Loss and Take Profit triggers BEFORE signal generation ---
            # Iterate over a copy of the positions dictionary
            positions_to_check = list(self.positions.keys())
            for instrument_key in positions_to_check:
                # Ensure the position still exists in case it was closed by another trigger
                # for a different instrument in the same time slice (unlikely with current structure, but safe)
                if instrument_key in self.positions:
                    position = self.positions[instrument_key]
                    entry_price = position.get('entry_price')
                    stop_loss_price = position.get('Stop_loss_price') # Get SL from position details
                    reward_amount = position.get('Reward_Amount') # Get Reward from position details
                    position_side = position.get('side') # Get side from position details

                    # Find the current price for this instrument in the current time slice
                    current_bar_data = time_slice_df[time_slice_df['instrument_key'] == instrument_key]

                    if current_bar_data.empty:
                        logger.warning(f"No data found for instrument {instrument_key} at timestamp {timestamp}. Cannot check triggers.")
                        continue # Skip trigger check for this instrument at this timestamp

                    # Assuming we check triggers against the close price of the current bar
                    # You might want to check against low for SL and high for TP for more realism
                    current_price = current_bar_data['close'].iloc[0] # Get the close price (assuming one row per instrument per timestamp)

                    # Also get the full data point for passing to close method
                    current_data_point = current_bar_data.iloc[0]


                    if pd.isna(current_price):
                        logger.debug(f"Invalid current price for {instrument_key} at {timestamp}. Cannot check triggers.")
                        continue # Cannot check triggers with invalid price

                    # Check Stop Loss for Long Positions
                    if position_side == 'BUY' and pd.notna(stop_loss_price) and current_price <= stop_loss_price:
                        logger.info(f"Stop Loss triggered for {instrument_key} at {timestamp} @ {current_price} (SL: {stop_loss_price}).")
                        self.close_position_on_trigger(instrument_key, timestamp, current_price, 'Stop Loss', current_data_point)
                        # Position is removed inside close_position_on_trigger, so no need to re-check or process signals for it

                    # Check Take Profit for Long Positions
                    # Calculate TP price: Entry Price + Reward Amount (assuming Reward Amount is an absolute value)
                    # If Reward_Amount is a percentage, calculate accordingly. Assuming absolute for now.
                    take_profit_price = None
                    if pd.notna(entry_price) and pd.notna(reward_amount):
                        take_profit_price = entry_price + reward_amount


                    if position_side == 'BUY' and pd.notna(take_profit_price) and current_price >= take_profit_price:
                        logger.info(f"Take Profit triggered for {instrument_key} at {timestamp} @ {current_price} (TP: {take_profit_price}).")
                        self.close_position_on_trigger(instrument_key, timestamp, current_price, 'Take Profit', current_data_point)
                        # Position is removed inside close_position_on_trigger


            # --- Process data for all instruments available at this timestamp for signals ---
            # Filter out instruments for which positions were just closed by triggers
            instruments_for_signal = time_slice_df[~time_slice_df['instrument_key'].isin(self.positions.keys())].copy()
            # Add back instruments for which positions are still open (though they won't generate new signals of the same type usually)
            # This might not be strictly necessary if your strategy logic correctly handles existing positions,
            # but ensures all instrument data points are iterated through for potential trailing stop updates, MFE/MAE calculation, etc.
            instruments_for_signal = pd.concat([instruments_for_signal, time_slice_df[time_slice_df['instrument_key'].isin(self.positions.keys())].copy()])
            instruments_for_signal = instruments_for_signal.drop_duplicates(subset=['instrument_key']).reset_index(drop=True) # Remove duplicates if any

            for index, data_point in instruments_for_signal.iterrows():
                 instrument_key = data_point['instrument_key']
                 current_price = data_point['close'] # Assume close price for execution

                 # Debug capture (already done above, can be kept or moved)
                 # self._debug_timestamps.append(timestamp)
                 # self._debug_close_values.append(current_price)
                 # self._debug_validity.append(pd.notna(current_price))


                 # Ensure current_price is valid for trading (already checked above, but good redundancy)
                 if pd.isna(current_price):
                     logger.debug(f"Skipping signal generation for {instrument_key} at {timestamp} due to invalid close price ({current_price}).")
                     self.debug_log.append({'type': 'SKIP_SIGNAL_PRICE_NAN', 'instrument': instrument_key, 'time': timestamp, 'reason': 'Invalid Price'})
                     continue # Skip this data point if price is invalid


                 # Check for signals from all active strategies for this data point
                 for strategy_name, strategy_instance in self.active_strategies_instances.items():
                     try:
                         # Pass the single data_point (as a Series converted to DataFrame) to the strategy
                         signal = strategy_instance.generate_signal(pd.DataFrame([data_point]))
                         # Ensure signal is a string, handle potential None returns gracefully
                         signal = str(signal).upper() if signal is not None else 'HOLD'

                         if signal in ['BUY', 'SELL']:
                             # Generate a unique trade ID for this potential trade using the bar's timestamp
                             trade_id = self.generate_trade_id(timestamp) # Pass the historical timestamp
                             # Pass the original data_point Series to execute_trade
                             self.execute_trade(trade_id, instrument_key, timestamp, signal, strategy_name, current_price, data_point)

                     except Exception as e:
                         logger.error(f"Error generating signal for {instrument_key} at {timestamp} using strategy '{strategy_name}': {e}", exc_info=True)
                         self.debug_log.append({'type': 'STRATEGY_ERROR', 'instrument': instrument_key, 'time': timestamp, 'strategy': strategy_name, 'error': str(e)})


        # After iterating through all data, close any remaining open positions
        self.close_all_positions(self.data['timestamp'].max()) # Use the timestamp of the last data point as exit time

        logger.info("Backtest simulation completed.")
        logger.info(f"Final Capital: {self.current_capital:.2f}")
        logger.info(f"Number of completed trades: {len(self.completed_trades)}")
        logger.info(f"Number of open positions remaining: {len(self.positions)}")

        # Return completed trades as a DataFrame for analysis
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

            # Ensure numeric columns are numeric
            numeric_cols = [
                'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
            ]
            for col in numeric_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')


            return completed_trades_df
        else:
            logger.warning("No completed trades recorded. Returning empty DataFrame.")
            return pd.DataFrame() # Return empty DataFrame if no trades


    def close_all_positions(self, exit_timestamp: datetime):
        """
        Closes all remaining open positions at the specified exit timestamp.
        Assumes closing at the price of the last available bar for each instrument.
        """
        logger.info(f"Closing all remaining {len(self.positions)} open positions at {exit_timestamp}...")

        # Get the last known price and data point for each instrument with an open position
        last_data_points = self.data.groupby('instrument_key').tail(1).set_index('instrument_key')
        last_prices = last_data_points['close'].to_dict()


        positions_to_close = list(self.positions.keys()) # Iterate over a copy

        for instrument_key in positions_to_close:
            # Check if position still exists (wasn't closed by a trigger just before the end)
            if instrument_key in self.positions:
                position = self.positions[instrument_key]
                closing_price = last_prices.get(instrument_key, np.nan) # Get last price, default to NaN if instrument not found

                # Get the last data point for the instrument to capture exit conditions
                last_data_point = last_data_points.get(instrument_key, pd.Series({})) # Use empty Series if no data found


                if pd.notna(closing_price):
                    # Use the dedicated close method for forced closure
                    self.close_position_on_trigger(instrument_key, exit_timestamp, closing_price, 'Forced_Close', last_data_point)

                else:
                    logger.warning(f"Could not find last price for {instrument_key}. Cannot close position {position['trade_id']}. Logging as unresolved.")
                    # Log as an unresolved position or assume zero PnL

                    # Calculate Trade Duration even if closing price is NaN
                    trade_duration = (exit_timestamp - position.get('entry_time')).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(position.get('entry_time')) else None

                    # Transfer known costs
                    entry_slippage = position.get('Slippage_Entry', 0)
                    entry_commission = position.get('Commission_Fees_Entry', 0)

                    unresolved_trade_record = {
                        'open_trade_id': position.get('trade_id'),
                        'close_trade_id': None, # No closing trade ID
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'),
                        'side': position.get('side'),
                        'quantity': position.get('quantity'),
                        'entry_price': position.get('entry_price'),
                        'entry_time': position.get('entry_time'),
                        'exit_price': None, # No exit price
                        'exit_time': exit_timestamp, # Use the requested exit timestamp
                        'pnl': 0, # Assume zero Gross PnL if cannot close
                        'strategy_opened': position.get('strategy'),
                        'strategy_closed': 'Backtester_Forced_Close_Error', # Indicate error

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': entry_slippage, # Transfer entry slippage
                        'Commission_Fees_Entry': entry_commission, # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),

                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close_Error', # Indicate forced close error
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': position.get('quantity'), # Shares that were supposed to be exited
                        'Exit_cost': 0,
                        'Exit_revenue': 0, # Assuming zero revenue if cannot close
                        'PnL_trade': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Profit_loss': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Exit_reason': 'Backtester_Forced_Close_Error: No_Last_Price', # Reason for exit
                        'Exit_Order_Type': None, # Could not execute exit order

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': last_data_point.get('Current_trailing_stop'), # Capture if calculated and available
                        'Trailing_stop_method': last_data_point.get('Trailing_stop_method'), # Capture if calculated and available
                        'Trailing_stop_value': last_data_point.get('Trailing_stop_value'), # Capture if calculated and available
                        'Stop_loss_price': last_data_point.get('Stop_loss_price'), # Capture if calculated and available


                        'Slippage': entry_slippage, # Only entry slippage is known
                        'Commission_Fees': entry_commission, # Only entry commission is known
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(unresolved_trade_record)

                    # Remove position even if it couldn't be closed properly to prevent it from being processed again
                    del self.positions[instrument_key]

                    self.debug_log.append({'type': 'FORCED_CLOSE_ERROR', 'open_trade_id': position['trade_id'], 'instrument': instrument_key, 'time': exit_timestamp, 'reason': 'Last price not available'})


        logger.info("All remaining positions closed.")


    def analyze_backtest_results(self):
        """
        Analyzes the completed trades and provides performance metrics.
        Returns a DataFrame summarizing the analysis.
        """
        logger.info("Analyzing backtest results...")

        if not self.completed_trades:
            logger.warning("No completed trades to analyze.")
            return pd.DataFrame({'Message': ['No completed trades to analyze.']})

        # 1. Access the self.completed_trades list and Create a pandas DataFrame
        trades_df = pd.DataFrame(self.completed_trades)

        # 3. Ensure that relevant columns are converted to appropriate numeric types
        numeric_cols = [
            'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
            'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
            'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
            'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
            'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
            'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
            'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
            'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Include other potentially numeric cols
        ]
        for col in numeric_cols:
            if col in trades_df.columns:
                trades_df[col] = pd.to_numeric(trades_df[col], errors='coerce')

        # Handle potential NaN values during conversion - drop rows where PnL_trade (or pnl) is NaN
        pnl_col_for_analysis = 'PnL_trade' if 'PnL_trade' in trades_df.columns else 'pnl'
        if pnl_col_for_analysis in trades_df.columns:
            # Only consider trades with a valid PnL for core analysis metrics
            trades_df_analysis = trades_df.dropna(subset=[pnl_col_for_analysis]).copy() # Use .copy() to avoid SettingWithCopyWarning
        else:
            logger.warning("Neither 'PnL_trade' nor 'pnl' column found for analysis.")
            return pd.DataFrame({'Message': ['No PnL column found for analysis.']})


        if trades_df_analysis.empty:
            logger.warning("No valid trades after numeric conversion/dropna for analysis. Analysis stopped.")
            return pd.DataFrame({'Message': ['No valid trades after numeric conversion/dropna for analysis.']})


        # 4. Update the calculation of basic performance metrics using 'PnL_trade'
        total_trades = len(trades_df_analysis)
        total_pnl = trades_df_analysis[pnl_col_for_analysis].sum()

        winning_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] > 0]
        losing_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] < 0]
        breakeven_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] == 0]

        num_winning = len(winning_trades)
        num_losing = len(losing_trades)
        num_breakeven = len(breakeven_trades)

        win_rate = (num_winning / total_trades) * 100 if total_trades > 0 else 0
        avg_win = winning_trades[pnl_col_for_analysis].mean() if num_winning > 0 else 0
        avg_loss = losing_trades[pnl_col_for_analysis].mean() if num_losing > 0 else 0
        expectancy = (win_rate / 100) * avg_win + ((100 - win_rate) / 100) * avg_loss if total_trades > 0 else 0

        # 5. Update Max Drawdown calculation to use 'PnL_trade' and sort by exit time
        # Calculate cumulative PnL and then cumulative capital
        trades_df_analysis = trades_df_analysis.sort_values(by='exit_time') # Sort by exit time for cumulative calculation

        trades_df_analysis['cumulative_pnl'] = trades_df_analysis[pnl_col_for_analysis].cumsum()

        # Add initial capital to cumulative PnL
        trades_df_analysis['cumulative_capital'] = self.initial_capital + trades_df_analysis['cumulative_pnl']

        # Calculate peak capital up to each point
        trades_df_analysis['peak_capital'] = trades_df_analysis['cumulative_capital'].cummax()

        # Calculate drawdown at each point
        trades_df_analysis['drawdown'] = trades_df_analysis['peak_capital'] - trades_df_analysis['cumulative_capital']

        # Calculate percentage drawdown
        # Avoid division by zero if peak_capital is 0 or None
        trades_df_analysis['pct_drawdown'] = trades_df_analysis.apply(
            lambda row: (row['drawdown'] / row['peak_capital']) * 100 if row['peak_capital'] > 0 and pd.notna(row['peak_capital']) else 0,
            axis=1
        )


        max_drawdown_amount = trades_df_analysis['drawdown'].max() if not trades_df_analysis.empty else 0
        max_drawdown_pct = trades_df_analysis['pct_drawdown'].max() if not trades_df_analysis.empty else 0


        # 6. Update analysis summary metric names
        analysis_summary = {
            'Metric': [
                'Initial Capital',
                'Final Capital',
                'Total PnL (Net)', # Indicate Net PnL
                'Total Trades',
                'Winning Trades (Net)', # Indicate Net PnL
                'Losing Trades (Net)', # Indicate Net PnL
                'Breakeven Trades (Net)', # Indicate Net PnL
                'Win Rate (%) (Net PnL)', # Indicate Net PnL
                'Average Win (Net)', # Indicate Net PnL
                'Average Loss (Net)', # Indicate Net PnL
                'Expectancy per Trade (Net)', # Indicate Net PnL
                'Max Drawdown (Amount)',
                'Max Drawdown (%)',
            ],
            'Value': [
                self.initial_capital,
                self.current_capital,
                round(total_pnl, 2), # Format to 2 decimal places
                total_trades,
                num_winning,
                num_losing,
                num_breakeven,
                round(win_rate, 2), # Format to 2 decimal places
                round(avg_win, 2),
                round(avg_loss, 2),
                round(expectancy, 2),
                round(max_drawdown_amount, 2),
                round(max_drawdown_pct, 2),
            ]
        }

        analysis_df = pd.DataFrame(analysis_summary)

        logger.info("Backtest analysis completed.")
        # You can print the analysis_df here or return it
        # print("\n--- Backtest Analysis Summary ---")
        # display(analysis_df) # Use display for notebooks

        # 8. Ensure the method returns the updated analysis summary DataFrame
        return analysis_df

    def get_completed_trades(self):
        """Returns a DataFrame of completed trades."""
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

                    # Ensure numeric columns are numeric
                    numeric_cols = [
                        'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                        'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                        'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                        'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                        'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                        'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                        'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                        'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                        'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
                    ]
                    for col in numeric_cols:
                        if col in completed_trades_df.columns:
                            completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')

                    return completed_trades_df
                else:
                    return pd.DataFrame() # Return empty DataFrame if no trades

Logging level set to DEBUG for test.


**Reasoning**:
Implement the logic within the `run_backtest` method to iterate through open positions and check for stop-loss and take-profit triggers using the current price from the time slice DataFrame. Call the `close_position_on_trigger` method if a trigger is detected.



In [5]:
# _1113_6BacktesterV6.py

import pandas as pd
import numpy as np
import logging
from datetime import datetime
import uuid
import sys
# from _012_instruments import get_instrument_type
# --- Logging Configuration ---
logger = logging.getLogger(__name__)



logger.setLevel(logging.DEBUG)

# Also ensure the root logger has a handler and is set to DEBUG,
# in case basicConfig was called elsewhere previously.
if not logging.root.handlers:
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
else:
    # If handlers exist, ensure at least one handler's level is DEBUG
    # and the root logger's level is DEBUG
    logging.root.setLevel(logging.DEBUG)
    handler_found = False
    for handler in logging.root.handlers:
        if isinstance(handler, logging.StreamHandler) and handler.stream in [sys.stdout, sys.stderr]:
            handler.setLevel(logging.DEBUG)
            handler_found = True
    # If no suitable handler is found (e.g., only file handlers), add a StreamHandler
    if not handler_found:
         stream_handler = logging.StreamHandler(sys.stdout)
         stream_handler.setLevel(logging.DEBUG)
         formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(message)s')
         stream_handler.setFormatter(formatter)
         logging.root.addHandler(stream_handler)


print("Logging level set to DEBUG for test.")


class BacktesterV3:
    """
    A simple backtesting engine for evaluating trading strategies.
    Processes historical data bar by bar, generates signals, and simulates trades.
    """
    def __init__(self, data: pd.DataFrame, instrument_keys: list, active_strategies_instances: dict, initial_capital: float):
        """
        Includes the same parameters as the original __init__

        Initializes the Backtester.

        Args:
            data: A pandas DataFrame containing historical market data for all instruments,
                expected to have columns like 'timestamp', 'instrument_key',
                'open', 'high', 'low', 'close', 'volume', etc. It is also expected
                to contain pre-calculated indicator and pattern columns used by
                the strategies and for recording trade details.
            instrument_keys: A list of unique instrument keys present in the data.
            active_strategies_instances: A dictionary where keys are strategy names
                                        (strings) and values are instantiated strategy
                                        objects with a `generate_signal(data_point)` method.
            initial_capital: The starting capital for the backtest simulation.
        """
        if data is None or data.empty:
            raise ValueError("Input data DataFrame is None or empty.")
        if not isinstance(data, pd.DataFrame):
            raise TypeError("Input 'data' must be a pandas DataFrame.")
        if data.index.name is not None:
            logger.warning("Input data index is not None. Consider resetting the index before passing to Backtester.")


        # Ensure essential columns are present and sorted
        required_columns = ['timestamp', 'instrument_key', 'open', 'high', 'low', 'close']
        # Define columns expected to be in the input data for recording trade details.
        # These are typically pre-calculated indicators or pattern detection results.
        entry_exit_data_columns_expected = [
            'Trend', 'SMA20', 'RSI', 'RSIMA', 'ATR', 'ADX', 'Volatility',
            'Breakout_Detected', 'Breakdown_Detected', 'Bullish_Candlestick_Detected',
            'Bearish_Candlestick_Detected', 'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
            'name', 'interval', 'Currency',
            'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
            'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price'
        ]

        # The backtester expects these columns to be pre-calculated and provided in the input data.
        # Strategies generate signals based on these columns, and their values at the time of
        # entry and exit are recorded in the completed_trades DataFrame.


        # Perform a relaxed check: log a warning if potential entry/exit columns from data are missing
        missing_data_cols = [col for col in entry_exit_data_columns_expected if col not in data.columns]
        if missing_data_cols:
            logger.warning(f"Input data is missing expected indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records. Ensure your data preparation includes these columns if strategies or analysis depend on them.")


        # Ensure mandatory required columns are present
        missing_required = [col for col in required_columns if col not in data.columns]
        if missing_required:
            raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")


        # Ensure timestamp is datetime and sorted
        try:
            if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
                data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', utc=True) # Convert to UTC
            # Drop rows where timestamp conversion failed
            data = data.dropna(subset=['timestamp'])
            # Sort by timestamp and then instrument_key to process bars chronologically per instrument
            self.data = data.sort_values(by=['timestamp', 'instrument_key']).reset_index(drop=True)
        except Exception as e:
            raise ValueError(f"Error processing timestamp column in data: {e}")


        self.instrument_keys = instrument_keys
        self.active_strategies_instances = active_strategies_instances
        self.initial_capital = initial_capital

        # --- Backtesting State Variables ---
        self.current_capital = initial_capital
        self.positions = {}  # Dictionary to track open positions {instrument_key: {...entry details...}}
        self.completed_trades = [] # List to store completed trades
        self.trade_id_counter = 0 # Simple counter for trade IDs
        self.debug_log = [] # List to store debug information

        # Debug lists to capture values
        self._debug_timestamps = []
        self._debug_close_values = []
        self._debug_validity = []

        # Simple Slippage and Commission model (can be customized)
        self.slippage_pct = 0.001  # 0.1% slippage per trade
        self.commission_per_trade = 0.01 # $0.01 fixed commission per trade


        logger.info(f"BacktesterV2 initialized with {len(self.instrument_keys)} instruments and {len(self.active_strategies_instances)} active strategies.")
        logger.info(f"Initial Capital: {self.initial_capital}")
        logger.info(f"Data shape for backtesting: {self.data.shape}")


    def generate_trade_id(self, timestamp: datetime):
        """Generates a unique trade ID using a provided timestamp."""
        # Using microseconds to increase the chance of uniqueness
        return timestamp.strftime('%Y%m%d%H%M%S%f')


    def execute_trade(self, trade_id: str, instrument_key: str, timestamp: datetime, signal: str, strategy_name: str, price: float, data_point: pd.Series):
        """
        Simulates executing a trade based on a signal.

        Args:
            trade_id: Unique identifier for the trade.
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the trade execution (bar close time).
            signal: The trading signal ('BUY' or 'SELL').
            strategy_name: The name of the strategy generating the signal.
            price: The execution price (typically the close price of the bar).
            data_point: The pandas Series representing the data row for this bar. This Series
                        is expected to contain pre-calculated indicator and pattern data
                        used for entry/exit conditions and recording.
        """
        # Determine instrument type to handle lot size/quantity logic
        # instrument_type = get_instrument_type(instrument_key) # Removed due to import error
        instrument_type = 'Unknown' # Placeholder


        # Simple fixed quantity logic (can be replaced with dynamic position sizing)
        quantity_to_trade = 1 # Example: trade 1 unit/lot

        if signal == 'BUY':
            # Check if we already have a position in this instrument (optional, depending on strategy)
            if instrument_key not in self.positions:
                # Simulate buying
                cost = quantity_to_trade * price
                # Check if we have enough capital
                if self.current_capital >= cost:
                    self.current_capital -= cost

                    # Calculate entry costs (slippage and commission on entry)
                    entry_slippage = cost * self.slippage_pct
                    entry_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (entry_slippage + entry_commission) # Deduct costs from capital

                    # Capture entry-specific details from the data_point and other variables
                    self.positions[instrument_key] = {
                        'quantity': quantity_to_trade,
                        'entry_price': price, # This is the execution price for this simple model
                        'entry_time': timestamp,
                        'strategy': strategy_name,
                        'trade_id': trade_id,
                        'instrument_type': instrument_type, # Store instrument type (now placeholder)
                        'side': 'BUY', # Store trade side

                        # --- Entry-Specific Columns (Populated from data_point at Entry) ---
                        'Strategy_name': strategy_name,
                        'instrument_key': instrument_key,
                        'name': data_point.get('name'), # Use .get() to avoid errors if column is missing
                        'interval': data_point.get('interval'),
                        'Position_type': 'Long', # Assuming BUY means Long position
                        'Entry_order_type': 'Market', # Assuming market order execution on close
                        'Entry_timestamp': timestamp,
                        'Entry_price_trigger': None, # Not explicitly handled in this simple model
                        'Entry_price_execution': price,
                        'Entry_shares': quantity_to_trade, # Using quantity_to_trade as shares
                        'Entry_cost': cost, # Gross cost before fees
                        'Entry_signal_type': signal, # Ensure signal is captured
                        'Entry_Trend': data_point.get('Trend'), # Capture Trend at Entry
                        'Entry_SMA20': data_point.get('SMA20'), # Capture SMA20 at Entry
                        'Entry_RSI': data_point.get('RSI'), # Capture RSI at Entry
                        'Entry_RSI_MA': data_point.get('RSI_MA'), # Capture RSI_MA at Entry
                        'Entry_ATR': data_point.get('ATR'), # Capture ATR at Entry
                        'Entry_ADX': data_point.get('ADX'), # Capture ADX at Entry
                        'Entry_Volatility': data_point.get('Volatility'), # Capture Volatility at Entry
                        'Entry_Breakout_Detected': data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Entry
                        'Entry_Breakdown_Detected': data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Entry
                        # Corrected column names to match expected input data
                        'Entry_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Entry
                        'Entry_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Entry
                        'Entry_Bullish_Chart_Pattern_Name': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Entry
                        'Entry_Bearish_Chart_Pattern_Name': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Entry
                        'Instrument_Type': instrument_type, # Store instrument type (now placeholder)
                        'Currency': data_point.get('Currency'),
                        'Slippage_Entry': entry_slippage, # Store entry slippage
                        'Commission_Fees_Entry': entry_commission, # Store entry commission

                        # Placeholder for other entry-specific details that might be calculated by strategy (e.g., initial stop/target)
                        'Initial_Stop_Loss_Distance (%)': data_point.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': data_point.get('Risk_Amount'),
                        'Reward_Amount': data_point.get('Reward_Amount'),


                        # Placeholders for exit/other info that will be filled on close
                        # These fields are included here so the structure is consistent for retrieval on exit,
                        # even though their values are None at the time of entry.
                         'Max_Favorable_Excursion_MFE': None, # Will be calculated on exit
                         'Max_Adverse_Excursion_MAE': None, # Will be calculated on exit
                        'Current_trailing_stop': None, # Need logic for trailing stops
                        'Trailing_stop_method': None,
                        'Trailing_stop_value': None,
                        'Stop_loss_price': None,


                        'Exit_Trend': None, 'Exit_signal_type': None, 'Exit_SMA20': None,
                        'Exit_RSI': None, 'Exit_RSI_MA': None, 'Exit_ATR': None, 'Exit_ADX': None,
                        'Exit_Volatility': None, 'Exit_Breakout_Detected': None,
                        'Exit_Breakdown_Detected': None, 'Exit_Bullish_Candlestick_Name': None,
                        'Exit_Bearish_Candlestick_Name': None, 'Exit_Bullish_Chart_Pattern_Detected': None,
                        'Exit_Bearish_Chart_Pattern_Detected': None, 'Exit_shares': None,
                        'Exit_cost': None, 'Exit_revenue': None, 'PnL_trade': None,
                        'Trade_type': None, 'Profit_loss': None, 'Exit_reason': None,
                        'Slippage': None, 'Commission_Fees': None, 'Trade_Duration': None,
                        'Exit_Order_Type': None
                    }

                    # --- Add debug logging for Entry columns here ---
                    logger.debug(f"DEBUG Entry Data Point for {instrument_key} at {timestamp}:")
                    debug_cols_to_check = [
                        'Trend', 'SMA20', 'RSI', 'RSI_MA', 'ATR', 'ADX', 'Volatility',
                        'Breakout_Detected', 'Breakdown_Detected',
                        # Corrected debug column names to match expected input data
                        'Bullish_Candlestick_Detected', 'Bearish_Candlestick_Detected',
                        'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
                        'Currency', 'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
                        'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE', 'Current_trailing_stop',
                        'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price', 'Exit_Trend',
                        'Exit_signal_type', 'Exit_SMA20', 'Exit_RSI', 'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX',
                        'Exit_Volatility', 'Exit_Breakout_Detected', 'Exit_Breakdown_Detected',
                        'Exit_Bullish_Candlestick_Name', 'Exit_Bearish_Candlestick_Name',
                        'Exit_Bullish_Chart_Pattern_Detected', 'Exit_Bearish_Chart_Pattern_Detected',
                        'Exit_cost'
                    ]
                    for col in debug_cols_to_check:
                         logger.debug(f"  {col}: {data_point.get(col, 'Column Not Found or None')}")
                    # --- End Debug Logging ---


                    logger.info(f"Executed BUY trade {trade_id} for {instrument_key} at {timestamp} @ {price} (Qty: {quantity_to_trade}). Costs: Slippage={entry_slippage:.4f}, Commission={entry_commission:.2f}. Capital left: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'BUY', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'slippage': entry_slippage, 'commission': entry_commission})
                else:
                    logger.warning(f"Insufficient capital ({self.current_capital:.2f}) to BUY {instrument_key} at {price} (Cost: {cost:.2f}). Skipping trade {trade_id}.")
                    self.debug_log.append({'type': 'SKIP_BUY_CAPITAL', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Insufficient Capital'})

            else:
                # Already in a position, maybe add to it or skip depending on strategy rules
                logger.debug(f"Skipping BUY signal for {instrument_key} at {timestamp}. Already in a position.")
                self.debug_log.append({'type': 'SKIP_BUY_POSITION', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Already in Position'})


        elif signal == 'SELL':
            # For backtesting, a 'SELL' signal usually means closing a long position or opening a short position
            # Let's assume 'SELL' means closing a long position if one exists for simplicity in this example.
            # For a shorting strategy, you'd need different logic.
            if instrument_key in self.positions and self.positions[instrument_key]['side'] == 'BUY':
                # Use the dedicated close method
                self.close_position_on_trigger(instrument_key, timestamp, price, 'Signal_SELL', data_point)

            else:
                # No matching long position to close, or maybe a shorting signal
                # For this simple backtester, we'll just log and skip if no long position
                logger.debug(f"Skipping SELL signal for {instrument_key} at {timestamp}. No matching long position to close.")
                self.debug_log.append({'type': 'SKIP_SELL_NO_LONG', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'strategy': strategy_name, 'reason': 'No Long Position'})


    def close_position_on_trigger(self, instrument_key: str, timestamp: datetime, closing_price: float, reason: str, exit_data_point: pd.Series):
        """
        Closes an open position for a specific instrument due to a trigger (SL, TP, etc.).

        Args:
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the closure (bar close time).
            closing_price: The price at which the position is closed.
            reason: The reason for closing ('Stop Loss', 'Take Profit', 'Signal_SELL', 'Forced_Close').
            exit_data_point: The pandas Series representing the data row for this bar at exit.
        """
        if instrument_key not in self.positions:
            logger.warning(f"Attempted to close non-existent position for {instrument_key} at {timestamp} (Reason: {reason}).")
            return

        position = self.positions[instrument_key]

        quantity_to_sell = position['quantity']
        entry_price = position['entry_price']
        entry_time = position['entry_time']
        strategy_opened = position['strategy']
        open_trade_id = position['trade_id']
        side = position['side']

        # Ensure side is 'BUY' for long position closure logic
        if side != 'BUY':
             logger.warning(f"Attempted to close non-long position for {instrument_key} at {timestamp} (Side: {side}). Skipping.")
             return

        revenue = quantity_to_sell * closing_price
        self.current_capital += revenue

        # Calculate Profit/Loss (Gross PnL) - For long positions
        gross_pnl = (closing_price - entry_price) * quantity_to_sell

        # Calculate exit costs (slippage and commission on exit)
        exit_slippage = revenue * self.slippage_pct
        exit_commission = self.commission_per_trade # Fixed commission per trade

        self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

        # Calculate Net PnL
        total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
        total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
        pnl_trade = gross_pnl - total_slippage - total_commission

        # Calculate Trade Duration
        trade_duration = (timestamp - entry_time).total_seconds() if pd.notnull(timestamp) and pd.notnull(entry_time) else None

        # Generate a unique trade ID for the closing trade
        close_trade_id = self.generate_trade_id(timestamp)

        # Record completed trade - Populate all desired columns
        trade_record = {
            'open_trade_id': open_trade_id,
            'close_trade_id': close_trade_id,
            'instrument_key': instrument_key,
            'instrument_type': position.get('instrument_type'),
            'side': side,
            'quantity': quantity_to_sell,
            'entry_price': entry_price,
            'entry_time': entry_time,
            'exit_price': closing_price,
            'exit_time': timestamp,
            'pnl': gross_pnl, # Keep gross PnL for reference
            'strategy_opened': strategy_opened,
            'strategy_closed': 'Backtester_Trigger' if reason not in ['Signal_SELL', 'Forced_Close'] else reason, # Indicate trigger closure or signal close
            'Position_type': position.get('Position_type'),
            'Entry_order_type': position.get('Entry_order_type'),
            'Entry_timestamp': position.get('Entry_timestamp'),
            'Entry_price_trigger': position.get('Entry_price_trigger'),
            'Entry_price_execution': position.get('Entry_price_execution'),
            'Entry_shares': position.get('Entry_shares'),
            'Entry_cost': position.get('Entry_cost'),
            'Entry_signal_type': position.get('Entry_signal_type'),
            'Entry_Trend': position.get('Entry_Trend'),
            'Entry_SMA20': position.get('Entry_SMA20'),
            'Entry_RSI': position.get('Entry_RSI'),
            'Entry_RSI_MA': position.get('Entry_RSI_MA'),
            'Entry_ATR': position.get('Entry_ATR'),
            'Entry_ADX': position.get('Entry_ADX'),
            'Entry_Volatility': position.get('Entry_Volatility'),
            'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'),
            'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'),
            'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'),
            'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'),
            'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'),
            'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'),
            'Instrument_Type': position.get('Instrument_Type'),
            'Currency': position.get('Currency'),
            'Slippage_Entry': position.get('Slippage_Entry'),
            'Commission_Fees_Entry': position.get('Commission_Fees_Entry'),
            'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
            'Risk_Amount': position.get('Risk_Amount'),
            'Reward_Amount': position.get('Reward_Amount'),
            'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
            'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),

            # --- Exit-Specific Columns (Populated from exit_data_point) ---
            'Exit_Trend': exit_data_point.get('Trend'),
            'Exit_signal_type': reason, # The reason is the "signal" for exit
            'Exit_SMA20': exit_data_point.get('SMA20'),
            'Exit_RSI': exit_data_point.get('RSI'),
            'Exit_RSI_MA': exit_data_point.get('RSI_MA'),
            'Exit_ATR': exit_data_point.get('ATR'),
            'Exit_ADX': exit_data_point.get('ADX'),
            'Exit_Volatility': exit_data_point.get('Volatility'),
            'Exit_Breakout_Detected': exit_data_point.get('Breakout_Detected'),
            'Exit_Breakdown_Detected': exit_data_point.get('Breakdown_Detected'),
            'Exit_Bullish_Candlestick_Name': exit_data_point.get('Bullish_Candlestick_Detected'),
            'Exit_Bearish_Candlestick_Name': exit_data_point.get('Bearish_Candlestick_Detected'),
            'Exit_Bullish_Chart_Pattern_Detected': exit_data_point.get('Detected_Bullish_Chart_Pattern_Name'),
            'Exit_Bearish_Chart_Pattern_Detected': exit_data_point.get('Detected_Bearish_Chart_Pattern_Name'),
            'Exit_shares': quantity_to_sell,
            'Exit_cost': 0, # Assuming no cost to exit a long position
            'Exit_revenue': revenue,
            'PnL_trade': pnl_trade,
            'Trade_type': f'Long Close ({reason})', # Indicate the closure type
            'Profit_loss': pnl_trade,
            'Exit_reason': reason,
            'Exit_Order_Type': 'Market' if reason != 'Forced_Close_Error: No_Last_Price' else None, # Assume market order unless error

            # Placeholder for other exit-specific details
            'Current_trailing_stop': exit_data_point.get('Current_trailing_stop'),
            'Trailing_stop_method': exit_data_point.get('Trailing_stop_method'),
            'Trailing_stop_value': exit_data_point.get('Trailing_stop_value'),
            'Stop_loss_price': exit_data_point.get('Stop_loss_price'),

            'Slippage': total_slippage,
            'Commission_Fees': total_commission,
            'Trade_Duration': trade_duration,
        }
        self.completed_trades.append(trade_record)

        # Remove position
        del self.positions[instrument_key]

        logger.info(f"Closed position {open_trade_id} for {instrument_key} at {timestamp} @ {closing_price} (Reason: {reason}). Net PnL: {pnl_trade:.2f}. Capital left: {self.current_capital:.2f}")
        self.debug_log.append({'type': 'CLOSE_TRIGGER', 'open_trade_id': open_trade_id, 'close_trade_id': close_trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': closing_price, 'quantity': quantity_to_sell, 'net_pnl': pnl_trade, 'reason': reason})


    def run_backtest(self):
        """
        Runs the backtesting simulation bar by bar through the data.
        """
        logger.info("Starting backtest simulation...")

        # Group data by timestamp first, then iterate through timestamps
        grouped_by_time = self.data.groupby('timestamp')

        for timestamp, time_slice_df in grouped_by_time:
            logger.debug(f"Processing timestamp: {timestamp}")

            # --- Check for Stop Loss and Take Profit triggers BEFORE signal generation ---
            # Iterate over a copy of the positions dictionary
            positions_to_check = list(self.positions.keys())
            for instrument_key in positions_to_check:
                # Ensure the position still exists in case it was closed by another trigger
                # for a different instrument in the same time slice (unlikely with current structure, but safe)
                if instrument_key in self.positions:
                    position = self.positions[instrument_key]
                    entry_price = position.get('entry_price')
                    stop_loss_price = position.get('Stop_loss_price') # Get SL from position details
                    reward_amount = position.get('Reward_Amount') # Get Reward from position details
                    position_side = position.get('side') # Get side from position details

                    # Find the current price for this instrument in the current time slice
                    current_bar_data = time_slice_df[time_slice_df['instrument_key'] == instrument_key]

                    if current_bar_data.empty:
                        logger.warning(f"No data found for instrument {instrument_key} at timestamp {timestamp}. Cannot check triggers.")
                        continue # Skip trigger check for this instrument at this timestamp

                    # Assuming we check triggers against the close price of the current bar
                    # You might want to check against low for SL and high for TP for more realism
                    current_price = current_bar_data['close'].iloc[0] # Get the close price (assuming one row per instrument per timestamp)

                    # Also get the full data point for passing to close method
                    current_data_point = current_bar_data.iloc[0]


                    if pd.isna(current_price):
                        logger.debug(f"Invalid current price for {instrument_key} at {timestamp}. Cannot check triggers.")
                        continue # Cannot check triggers with invalid price

                    # Check Stop Loss for Long Positions
                    if position_side == 'BUY' and pd.notna(stop_loss_price) and current_price <= stop_loss_price:
                        logger.info(f"Stop Loss triggered for {instrument_key} at {timestamp} @ {current_price} (SL: {stop_loss_price}).")
                        self.close_position_on_trigger(instrument_key, timestamp, current_price, 'Stop Loss', current_data_point)
                        # Position is removed inside close_position_on_trigger, so no need to re-check or process signals for it
                        continue # Move to the next position check if this one was closed

                    # Check Take Profit for Long Positions
                    # Calculate TP price: Entry Price + Reward Amount (assuming Reward Amount is an absolute value)
                    # If Reward_Amount is a percentage, calculate accordingly. Assuming absolute for now.
                    take_profit_price = None
                    if pd.notna(entry_price) and pd.notna(reward_amount):
                        take_profit_price = entry_price + reward_amount


                    if position_side == 'BUY' and pd.notna(take_profit_price) and current_price >= take_profit_price:
                        logger.info(f"Take Profit triggered for {instrument_key} at {timestamp} @ {current_price} (TP: {take_profit_price}).")
                        self.close_position_on_trigger(instrument_key, timestamp, current_price, 'Take Profit', current_data_point)
                        # Position is removed inside close_position_on_trigger
                        continue # Move to the next position check if this one was closed


            # --- Process data for all instruments available at this timestamp for signals ---
            # Filter out instruments for which positions were just closed by triggers
            instruments_for_signal = time_slice_df[~time_slice_df['instrument_key'].isin(self.positions.keys())].copy()
            # Add back instruments for which positions are still open (though they won't generate new signals of the same type usually)
            # This might not be strictly necessary if your strategy logic correctly handles existing positions,
            # but ensures all instrument data points are iterated through for potential trailing stop updates, MFE/MAE calculation, etc.
            instruments_for_signal = pd.concat([instruments_for_signal, time_slice_df[time_slice_df['instrument_key'].isin(self.positions.keys())].copy()])
            instruments_for_signal = instruments_for_signal.drop_duplicates(subset=['instrument_key']).reset_index(drop=True) # Remove duplicates if any

            for index, data_point in instruments_for_signal.iterrows():
                 instrument_key = data_point['instrument_key']
                 current_price = data_point['close'] # Assume close price for execution

                 # Debug capture (already done above, can be kept or moved)
                 # self._debug_timestamps.append(timestamp)
                 # self._debug_close_values.append(current_price)
                 # self._debug_validity.append(pd.notna(current_price))


                 # Ensure current_price is valid for trading (already checked above, but good redundancy)
                 if pd.isna(current_price):
                     logger.debug(f"Skipping signal generation for {instrument_key} at {timestamp} due to invalid close price ({current_price}).")
                     self.debug_log.append({'type': 'SKIP_SIGNAL_PRICE_NAN', 'instrument': instrument_key, 'time': timestamp, 'reason': 'Invalid Price'})
                     continue # Skip this data point if price is invalid


                 # Check for signals from all active strategies for this data point
                 for strategy_name, strategy_instance in self.active_strategies_instances.items():
                     try:
                         # Pass the single data_point (as a Series converted to DataFrame) to the strategy
                         signal = strategy_instance.generate_signal(pd.DataFrame([data_point]))
                         # Ensure signal is a string, handle potential None returns gracefully
                         signal = str(signal).upper() if signal is not None else 'HOLD'

                         if signal in ['BUY', 'SELL']:
                             # Generate a unique trade ID for this potential trade using the bar's timestamp
                             trade_id = self.generate_trade_id(timestamp) # Pass the historical timestamp
                             # Pass the original data_point Series to execute_trade
                             self.execute_trade(trade_id, instrument_key, timestamp, signal, strategy_name, current_price, data_point)

                     except Exception as e:
                         logger.error(f"Error generating signal for {instrument_key} at {timestamp} using strategy '{strategy_name}': {e}", exc_info=True)
                         self.debug_log.append({'type': 'STRATEGY_ERROR', 'instrument': instrument_key, 'time': timestamp, 'strategy': strategy_name, 'error': str(e)})


        # After iterating through all data, close any remaining open positions
        self.close_all_positions(self.data['timestamp'].max()) # Use the timestamp of the last data point as exit time

        logger.info("Backtest simulation completed.")
        logger.info(f"Final Capital: {self.current_capital:.2f}")
        logger.info(f"Number of completed trades: {len(self.completed_trades)}")
        logger.info(f"Number of open positions remaining: {len(self.positions)}")

        # Return completed trades as a DataFrame for analysis
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

            # Ensure numeric columns are numeric
            numeric_cols = [
                'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
            ]
            for col in numeric_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')


            return completed_trades_df
        else:
            logger.warning("No completed trades recorded. Returning empty DataFrame.")
            return pd.DataFrame() # Return empty DataFrame if no trades


    def close_all_positions(self, exit_timestamp: datetime):
        """
        Closes all remaining open positions at the specified exit timestamp.
        Assumes closing at the price of the last available bar for each instrument.
        """
        logger.info(f"Closing all remaining {len(self.positions)} open positions at {exit_timestamp}...")

        # Get the last known price and data point for each instrument with an open position
        last_data_points = self.data.groupby('instrument_key').tail(1).set_index('instrument_key')
        last_prices = last_data_points['close'].to_dict()


        positions_to_close = list(self.positions.keys()) # Iterate over a copy

        for instrument_key in positions_to_close:
            # Check if position still exists (wasn't closed by a trigger just before the end)
            if instrument_key in self.positions:
                position = self.positions[instrument_key]
                closing_price = last_prices.get(instrument_key, np.nan) # Get last price, default to NaN if instrument not found

                # Get the last data point for the instrument to capture exit conditions
                last_data_point = last_data_points.get(instrument_key, pd.Series({})) # Use empty Series if no data found


                if pd.notna(closing_price):
                    # Use the dedicated close method for forced closure
                    self.close_position_on_trigger(instrument_key, exit_timestamp, closing_price, 'Forced_Close', last_data_point)

                else:
                    logger.warning(f"Could not find last price for {instrument_key}. Cannot close position {position['trade_id']}. Logging as unresolved.")
                    # Log as an unresolved position or assume zero PnL

                    # Calculate Trade Duration even if closing price is NaN
                    trade_duration = (exit_timestamp - position.get('entry_time')).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(position.get('entry_time')) else None

                    # Transfer known costs
                    entry_slippage = position.get('Slippage_Entry', 0)
                    entry_commission = position.get('Commission_Fees_Entry', 0)

                    unresolved_trade_record = {
                        'open_trade_id': position.get('trade_id'),
                        'close_trade_id': None, # No closing trade ID
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'),
                        'side': position.get('side'),
                        'quantity': position.get('quantity'),
                        'entry_price': position.get('entry_price'),
                        'entry_time': position.get('entry_time'),
                        'exit_price': None, # No exit price
                        'exit_time': exit_timestamp, # Use the requested exit timestamp
                        'pnl': -(entry_slippage + entry_commission), # Only entry costs are known
                        'strategy_opened': position.get('strategy'),
                        'strategy_closed': 'Backtester_Forced_Close_Error', # Indicate error

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': entry_slippage, # Transfer entry slippage
                        'Commission_Fees_Entry': entry_commission, # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),

                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close_Error', # Indicate forced close error
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': position.get('quantity'), # Shares that were supposed to be exited
                        'Exit_cost': 0,
                        'Exit_revenue': 0, # Assuming zero revenue if cannot close
                        'PnL_trade': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Profit_loss': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Exit_reason': 'Backtester_Forced_Close_Error: No_Last_Price', # Reason for exit
                        'Exit_Order_Type': None, # Could not execute exit order

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': last_data_point.get('Current_trailing_stop'), # Capture if calculated and available
                        'Trailing_stop_method': last_data_point.get('Trailing_stop_method'), # Capture if calculated and available
                        'Trailing_stop_value': last_data_point.get('Trailing_stop_value'), # Capture if calculated and available
                        'Stop_loss_price': last_data_point.get('Stop_loss_price'), # Capture if calculated and available


                        'Slippage': entry_slippage, # Only entry slippage is known
                        'Commission_Fees': entry_commission, # Only entry commission is known
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(unresolved_trade_record)

                    # Remove position even if it couldn't be closed properly to prevent it from being processed again
                    del self.positions[instrument_key]

                    self.debug_log.append({'type': 'FORCED_CLOSE_ERROR', 'open_trade_id': position['trade_id'], 'instrument': instrument_key, 'time': exit_timestamp, 'reason': 'Last price not available'})


        logger.info("All remaining positions closed.")


    def analyze_backtest_results(self):
        """
        Analyzes the completed trades and provides performance metrics.
        Returns a DataFrame summarizing the analysis.
        """
        logger.info("Analyzing backtest results...")

        if not self.completed_trades:
            logger.warning("No completed trades to analyze.")
            return pd.DataFrame({'Message': ['No completed trades to analyze.']})

        # 1. Access the self.completed_trades list and Create a pandas DataFrame
        trades_df = pd.DataFrame(self.completed_trades)

        # 3. Ensure that relevant columns are converted to appropriate numeric types
        numeric_cols = [
            'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
            'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
            'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
            'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
            'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
            'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
            'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
            'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Include other potentially numeric cols
        ]
        for col in numeric_cols:
            if col in trades_df.columns:
                trades_df[col] = pd.to_numeric(trades_df[col], errors='coerce')

        # Handle potential NaN values during conversion - drop rows where PnL_trade (or pnl) is NaN
        pnl_col_for_analysis = 'PnL_trade' if 'PnL_trade' in trades_df.columns else 'pnl'
        if pnl_col_for_analysis in trades_df.columns:
            # Only consider trades with a valid PnL for core analysis metrics
            trades_df_analysis = trades_df.dropna(subset=[pnl_col_for_analysis]).copy() # Use .copy() to avoid SettingWithCopyWarning
        else:
            logger.warning("Neither 'PnL_trade' nor 'pnl' column found for analysis.")
            return pd.DataFrame({'Message': ['No PnL column found for analysis.']})


        if trades_df_analysis.empty:
            logger.warning("No valid trades after numeric conversion/dropna for analysis. Analysis stopped.")
            return pd.DataFrame({'Message': ['No valid trades after numeric conversion/dropna for analysis.']})


        # 4. Update the calculation of basic performance metrics using 'PnL_trade'
        total_trades = len(trades_df_analysis)
        total_pnl = trades_df_analysis[pnl_col_for_analysis].sum()

        winning_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] > 0]
        losing_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] < 0]
        breakeven_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] == 0]

        num_winning = len(winning_trades)
        num_losing = len(losing_trades)
        num_breakeven = len(breakeven_trades)

        win_rate = (num_winning / total_trades) * 100 if total_trades > 0 else 0
        avg_win = winning_trades[pnl_col_for_analysis].mean() if num_winning > 0 else 0
        avg_loss = losing_trades[pnl_col_for_analysis].mean() if num_losing > 0 else 0
        expectancy = (win_rate / 100) * avg_win + ((100 - win_rate) / 100) * avg_loss if total_trades > 0 else 0

        # 5. Update Max Drawdown calculation to use 'PnL_trade' and sort by exit time
        # Calculate cumulative PnL and then cumulative capital
        trades_df_analysis = trades_df_analysis.sort_values(by='exit_time') # Sort by exit time for cumulative calculation

        trades_df_analysis['cumulative_pnl'] = trades_df_analysis[pnl_col_for_analysis].cumsum()

        # Add initial capital to cumulative PnL
        trades_df_analysis['cumulative_capital'] = self.initial_capital + trades_df_analysis['cumulative_pnl']

        # Calculate peak capital up to each point
        trades_df_analysis['peak_capital'] = trades_df_analysis['cumulative_capital'].cummax()

        # Calculate drawdown at each point
        trades_df_analysis['drawdown'] = trades_df_analysis['peak_capital'] - trades_df_analysis['cumulative_capital']

        # Calculate percentage drawdown
        # Avoid division by zero if peak_capital is 0 or None
        trades_df_analysis['pct_drawdown'] = trades_df_analysis.apply(
            lambda row: (row['drawdown'] / row['peak_capital']) * 100 if row['peak_capital'] > 0 and pd.notna(row['peak_capital']) else 0,
            axis=1
        )


        max_drawdown_amount = trades_df_analysis['drawdown'].max() if not trades_df_analysis.empty else 0
        max_drawdown_pct = trades_df_analysis['pct_drawdown'].max() if not trades_df_analysis.empty else 0


        # 6. Update analysis summary metric names
        analysis_summary = {
            'Metric': [
                'Initial Capital',
                'Final Capital',
                'Total PnL (Net)', # Indicate Net PnL
                'Total Trades',
                'Winning Trades (Net)', # Indicate Net PnL
                'Losing Trades (Net)', # Indicate Net PnL
                'Breakeven Trades (Net)', # Indicate Net PnL
                'Win Rate (%) (Net PnL)', # Indicate Net PnL
                'Average Win (Net)', # Indicate Net PnL
                'Average Loss (Net)', # Indicate Net PnL
                'Expectancy per Trade (Net)', # Indicate Net PnL
                'Max Drawdown (Amount)',
                'Max Drawdown (%)',
            ],
            'Value': [
                self.initial_capital,
                self.current_capital,
                round(total_pnl, 2), # Format to 2 decimal places
                total_trades,
                num_winning,
                num_losing,
                num_breakeven,
                round(win_rate, 2), # Format to 2 decimal places
                round(avg_win, 2),
                round(avg_loss, 2),
                round(expectancy, 2),
                round(max_drawdown_amount, 2),
                round(max_drawdown_pct, 2),
            ]
        }

        analysis_df = pd.DataFrame(analysis_summary)

        logger.info("Backtest analysis completed.")
        # You can print the analysis_df here or return it
        # print("\n--- Backtest Analysis Summary ---")
        # display(analysis_df) # Use display for notebooks

        # 8. Ensure the method returns the updated analysis summary DataFrame
        return analysis_df

    def get_completed_trades(self):
        """Returns a DataFrame of completed trades."""
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

                    # Ensure numeric columns are numeric
                    numeric_cols = [
                        'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                        'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                        'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                        'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                        'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                        'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                        'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                        'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                        'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
                    ]
                    for col in numeric_cols:
                        if col in completed_trades_df.columns:
                            completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')

                    return completed_trades_df
                else:
                    return pd.DataFrame() # Return empty DataFrame if no trades

Logging level set to DEBUG for test.


## Add trailing stop execution

### Subtask:
Add a loop or check within `run_backtest` to evaluate trailing stop conditions for open positions on each bar.


**Reasoning**:
Add the logic to check and update trailing stop conditions for open long positions within the `run_backtest` method, before the signal generation loop and after the initial SL/TP checks. This involves calculating the new trailing stop based on the current bar's high and the specified method, updating the trailing stop if it moves favorably, and closing the position if the low breaches the updated trailing stop.



In [6]:
# _1113_6BacktesterV6.py

import pandas as pd
import numpy as np
import logging
from datetime import datetime
import uuid
import sys
# from _012_instruments import get_instrument_type
# --- Logging Configuration ---
logger = logging.getLogger(__name__)



logger.setLevel(logging.DEBUG)

# Also ensure the root logger has a handler and is set to DEBUG,
# in case basicConfig was called elsewhere previously.
if not logging.root.handlers:
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
else:
    # If handlers exist, ensure at least one handler's level is DEBUG
    # and the root logger's level is DEBUG
    logging.root.setLevel(logging.DEBUG)
    handler_found = False
    for handler in logging.root.handlers:
        if isinstance(handler, logging.StreamHandler) and handler.stream in [sys.stdout, sys.stderr]:
            handler.setLevel(logging.DEBUG)
            handler_found = True
    # If no suitable handler is found (e.g., only file handlers), add a StreamHandler
    if not handler_found:
         stream_handler = logging.StreamHandler(sys.stdout)
         stream_handler.setLevel(logging.DEBUG)
         formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(message)s')
         stream_handler.setFormatter(formatter)
         logging.root.addHandler(stream_handler)


print("Logging level set to DEBUG for test.")


class BacktesterV3:
    """
    A simple backtesting engine for evaluating trading strategies.
    Processes historical data bar by bar, generates signals, and simulates trades.
    """
    def __init__(self, data: pd.DataFrame, instrument_keys: list, active_strategies_instances: dict, initial_capital: float):
        """
        Includes the same parameters as the original __init__

        Initializes the Backtester.

        Args:
            data: A pandas DataFrame containing historical market data for all instruments,
                expected to have columns like 'timestamp', 'instrument_key',
                'open', 'high', 'low', 'close', 'volume', etc. It is also expected
                to contain pre-calculated indicator and pattern columns used by
                the strategies and for recording trade details.
            instrument_keys: A list of unique instrument keys present in the data.
            active_strategies_instances: A dictionary where keys are strategy names
                                        (strings) and values are instantiated strategy
                                        objects with a `generate_signal(data_point)` method.
            initial_capital: The starting capital for the backtest simulation.
        """
        if data is None or data.empty:
            raise ValueError("Input data DataFrame is None or empty.")
        if not isinstance(data, pd.DataFrame):
            raise TypeError("Input 'data' must be a pandas DataFrame.")
        if data.index.name is not None:
            logger.warning("Input data index is not None. Consider resetting the index before passing to Backtester.")


        # Ensure essential columns are present and sorted
        required_columns = ['timestamp', 'instrument_key', 'open', 'high', 'low', 'close', 'high', 'low'] # Added high, low to required
        # Define columns expected to be in the input data for recording trade details.
        # These are typically pre-calculated indicators or pattern detection results.
        entry_exit_data_columns_expected = [
            'Trend', 'SMA20', 'RSI', 'RSIMA', 'ATR', 'ADX', 'Volatility',
            'Breakout_Detected', 'Breakdown_Detected', 'Bullish_Candlestick_Detected',
            'Bearish_Candlestick_Detected', 'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
            'name', 'interval', 'Currency',
            'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
            'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price' # Added Trailing_stop_value
        ]

        # The backtester expects these columns to be pre-calculated and provided in the input data.
        # Strategies generate signals based on these columns, and their values at the time of
        # entry and exit are recorded in the completed_trades DataFrame.


        # Perform a relaxed check: log a warning if potential entry/exit columns from data are missing
        missing_data_cols = [col for col in entry_exit_data_columns_expected if col not in data.columns]
        if missing_data_cols:
            logger.warning(f"Input data is missing expected indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records. Ensure your data preparation includes these columns if strategies or analysis depend on them.")


        # Ensure mandatory required columns are present
        missing_required = [col for col in required_columns if col not in data.columns]
        if missing_required:
            raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")


        # Ensure timestamp is datetime and sorted
        try:
            if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
                data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', utc=True) # Convert to UTC
            # Drop rows where timestamp conversion failed
            data = data.dropna(subset=['timestamp'])
            # Sort by timestamp and then instrument_key to process bars chronologically per instrument
            self.data = data.sort_values(by=['timestamp', 'instrument_key']).reset_index(drop=True)
        except Exception as e:
            raise ValueError(f"Error processing timestamp column in data: {e}")


        self.instrument_keys = instrument_keys
        self.active_strategies_instances = active_strategies_instances
        self.initial_capital = initial_capital

        # --- Backtesting State Variables ---
        self.current_capital = initial_capital
        self.positions = {}  # Dictionary to track open positions {instrument_key: {...entry details...}}
        self.completed_trades = [] # List to store completed trades
        self.trade_id_counter = 0 # Simple counter for trade IDs
        self.debug_log = [] # List to store debug information

        # Debug lists to capture values
        self._debug_timestamps = []
        self._debug_close_values = []
        self._debug_validity = []

        # Simple Slippage and Commission model (can be customized)
        self.slippage_pct = 0.001  # 0.1% slippage per trade
        self.commission_per_trade = 0.01 # $0.01 fixed commission per trade


        logger.info(f"BacktesterV2 initialized with {len(self.instrument_keys)} instruments and {len(self.active_strategies_instances)} active strategies.")
        logger.info(f"Initial Capital: {self.initial_capital}")
        logger.info(f"Data shape for backtesting: {self.data.shape}")


    def generate_trade_id(self, timestamp: datetime):
        """Generates a unique trade ID using a provided timestamp."""
        # Using microseconds to increase the chance of uniqueness
        return timestamp.strftime('%Y%m%d%H%M%S%f')


    def execute_trade(self, trade_id: str, instrument_key: str, timestamp: datetime, signal: str, strategy_name: str, price: float, data_point: pd.Series):
        """
        Simulates executing a trade based on a signal.

        Args:
            trade_id: Unique identifier for the trade.
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the trade execution (bar close time).
            signal: The trading signal ('BUY' or 'SELL').
            strategy_name: The name of the strategy generating the signal.
            price: The execution price (typically the close price of the bar).
            data_point: The pandas Series representing the data row for this bar. This Series
                        is expected to contain pre-calculated indicator and pattern data
                        used for entry/exit conditions and recording.
        """
        # Determine instrument type to handle lot size/quantity logic
        # instrument_type = get_instrument_type(instrument_key) # Removed due to import error
        instrument_type = 'Unknown' # Placeholder


        # Simple fixed quantity logic (can be replaced with dynamic position sizing)
        quantity_to_trade = 1 # Example: trade 1 unit/lot

        if signal == 'BUY':
            # Check if we already have a position in this instrument (optional, depending on strategy)
            if instrument_key not in self.positions:
                # Simulate buying
                cost = quantity_to_trade * price
                # Check if we have enough capital
                if self.current_capital >= cost:
                    self.current_capital -= cost

                    # Calculate entry costs (slippage and commission on entry)
                    entry_slippage = cost * self.slippage_pct
                    entry_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (entry_slippage + entry_commission) # Deduct costs from capital

                    # Capture entry-specific details from the data_point and other variables
                    self.positions[instrument_key] = {
                        'quantity': quantity_to_trade,
                        'entry_price': price, # This is the execution price for this simple model
                        'entry_time': timestamp,
                        'strategy': strategy_name,
                        'trade_id': trade_id,
                        'instrument_type': instrument_type, # Store instrument type (now placeholder)
                        'side': 'BUY', # Store trade side

                        # --- Entry-Specific Columns (Populated from data_point at Entry) ---
                        'Strategy_name': strategy_name,
                        'instrument_key': instrument_key,
                        'name': data_point.get('name'), # Use .get() to avoid errors if column is missing
                        'interval': data_point.get('interval'),
                        'Position_type': 'Long', # Assuming BUY means Long position
                        'Entry_order_type': 'Market', # Assuming market order execution on close
                        'Entry_timestamp': timestamp,
                        'Entry_price_trigger': None, # Not explicitly handled in this simple model
                        'Entry_price_execution': price,
                        'Entry_shares': quantity_to_trade, # Using quantity_to_trade as shares
                        'Entry_cost': cost, # Gross cost before fees
                        'Entry_signal_type': signal, # Ensure signal is captured
                        'Entry_Trend': data_point.get('Trend'), # Capture Trend at Entry
                        'Entry_SMA20': data_point.get('SMA20'), # Capture SMA20 at Entry
                        'Entry_RSI': data_point.get('RSI'), # Capture RSI at Entry
                        'Entry_RSI_MA': data_point.get('RSI_MA'), # Capture RSI_MA at Entry
                        'Entry_ATR': data_point.get('ATR'), # Capture ATR at Entry
                        'Entry_ADX': data_point.get('ADX'), # Capture ADX at Entry
                        'Entry_Volatility': data_point.get('Volatility'), # Capture Volatility at Entry
                        'Entry_Breakout_Detected': data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Entry
                        'Entry_Breakdown_Detected': data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Entry
                        # Corrected column names to match expected input data
                        'Entry_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Entry
                        'Entry_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Entry
                        'Entry_Bullish_Chart_Pattern_Name': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Entry
                        'Entry_Bearish_Chart_Pattern_Name': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Entry
                        'Instrument_Type': instrument_type, # Store instrument type (now placeholder)
                        'Currency': data_point.get('Currency'),
                        'Slippage_Entry': entry_slippage, # Store entry slippage
                        'Commission_Fees_Entry': entry_commission, # Store entry commission

                        # Placeholder for other entry-specific details that might be calculated by strategy (e.g., initial stop/target)
                        'Initial_Stop_Loss_Distance (%)': data_point.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': data_point.get('Risk_Amount'),
                        'Reward_Amount': data_point.get('Reward_Amount'),


                        # Placeholders for exit/other info that will be filled on close
                        # These fields are included here so the structure is consistent for retrieval on exit,
                        # even though their values are None at the time of entry.
                         'Max_Favorable_Excursion_MFE': None, # Will be calculated on exit
                         'Max_Adverse_Excursion_MAE': None, # Will be calculated on exit
                        'Current_trailing_stop': None, # Need logic for trailing stops
                        'Trailing_stop_method': data_point.get('Trailing_stop_method'), # Capture method from data point
                        'Trailing_stop_value': data_point.get('Trailing_stop_value'), # Capture initial value from data point (if applicable)
                        'Stop_loss_price': data_point.get('Stop_loss_price'), # Capture initial SL from data point


                        'Exit_Trend': None, 'Exit_signal_type': None, 'Exit_SMA20': None,
                        'Exit_RSI': None, 'Exit_RSI_MA': None, 'Exit_ATR': None, 'Exit_ADX': None,
                        'Exit_Volatility': None, 'Exit_Breakout_Detected': None,
                        'Exit_Breakdown_Detected': None, 'Exit_Bullish_Candlestick_Name': None,
                        'Exit_Bearish_Candlestick_Name': None, 'Exit_Bullish_Chart_Pattern_Detected': None,
                        'Exit_Bearish_Chart_Pattern_Detected': None, 'Exit_shares': None,
                        'Exit_cost': None, 'Exit_revenue': None, 'PnL_trade': None,
                        'Trade_type': None, 'Profit_loss': None, 'Exit_reason': None,
                        'Slippage': None, 'Commission_Fees': None, 'Trade_Duration': None,
                        'Exit_Order_Type': None
                    }
                    # Initialize 'Current_trailing_stop' with the initial 'Stop_loss_price' if a trailing method is specified
                    if pd.notna(self.positions[instrument_key].get('Trailing_stop_method')):
                         self.positions[instrument_key]['Current_trailing_stop'] = self.positions[instrument_key].get('Stop_loss_price')
                         logger.debug(f"Initialized Trailing Stop for {instrument_key} at {timestamp} to initial SL: {self.positions[instrument_key].get('Current_trailing_stop'):.4f}")


                    # --- Add debug logging for Entry columns here ---
                    logger.debug(f"DEBUG Entry Data Point for {instrument_key} at {timestamp}:")
                    debug_cols_to_check = [
                        'Trend', 'SMA20', 'RSI', 'RSI_MA', 'ATR', 'ADX', 'Volatility',
                        'Breakout_Detected', 'Breakdown_Detected',
                        # Corrected debug column names to match expected input data
                        'Bullish_Candlestick_Detected', 'Bearish_Candlestick_Detected',
                        'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
                        'Currency', 'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
                        'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE', 'Current_trailing_stop',
                        'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price', 'Exit_Trend',
                        'Exit_signal_type', 'Exit_SMA20', 'Exit_RSI', 'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX',
                        'Exit_Volatility', 'Exit_Breakout_Detected', 'Exit_Breakdown_Detected',
                        'Exit_Bullish_Candlestick_Name', 'Exit_Bearish_Candlestick_Name',
                        'Exit_Bullish_Chart_Pattern_Detected', 'Exit_Bearish_Chart_Pattern_Detected',
                        'Exit_cost'
                    ]
                    for col in debug_cols_to_check:
                         logger.debug(f"  {col}: {data_point.get(col, 'Column Not Found or None')}")
                    # --- End Debug Logging ---


                    logger.info(f"Executed BUY trade {trade_id} for {instrument_key} at {timestamp} @ {price} (Qty: {quantity_to_trade}). Costs: Slippage={entry_slippage:.4f}, Commission={entry_commission:.2f}. Capital left: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'BUY', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'slippage': entry_slippage, 'commission': entry_commission})
                else:
                    logger.warning(f"Insufficient capital ({self.current_capital:.2f}) to BUY {instrument_key} at {price} (Cost: {cost:.2f}). Skipping trade {trade_id}.")
                    self.debug_log.append({'type': 'SKIP_BUY_CAPITAL', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Insufficient Capital'})

            else:
                # Already in a position, maybe add to it or skip depending on strategy rules
                logger.debug(f"Skipping BUY signal for {instrument_key} at {timestamp}. Already in a position.")
                self.debug_log.append({'type': 'SKIP_BUY_POSITION', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Already in Position'})


        elif signal == 'SELL':
            # For backtesting, a 'SELL' signal usually means closing a long position or opening a short position
            # Let's assume 'SELL' means closing a long position if one exists for simplicity in this example.
            # For a shorting strategy, you'd need different logic.
            if instrument_key in self.positions and self.positions[instrument_key]['side'] == 'BUY':
                # Use the dedicated close method
                self.close_position_on_trigger(instrument_key, timestamp, price, 'Signal_SELL', data_point)

            else:
                # No matching long position to close, or maybe a shorting signal
                # For this simple backtester, we'll just log and skip if no long position
                logger.debug(f"Skipping SELL signal for {instrument_key} at {timestamp}. No matching long position to close.")
                self.debug_log.append({'type': 'SKIP_SELL_NO_LONG', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'strategy': strategy_name, 'reason': 'No Long Position'})


    def close_position_on_trigger(self, instrument_key: str, timestamp: datetime, closing_price: float, reason: str, exit_data_point: pd.Series):
        """
        Closes an open position for a specific instrument due to a trigger (SL, TP, etc.).

        Args:
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the closure (bar close time).
            closing_price: The price at which the position is closed.
            reason: The reason for closing ('Stop Loss', 'Take Profit', 'Signal_SELL', 'Forced_Close', 'Trailing Stop').
            exit_data_point: The pandas Series representing the data row for this bar at exit.
        """
        if instrument_key not in self.positions:
            logger.warning(f"Attempted to close non-existent position for {instrument_key} at {timestamp} (Reason: {reason}).")
            return

        position = self.positions[instrument_key]

        quantity_to_sell = position['quantity']
        entry_price = position['entry_price']
        entry_time = position['entry_time']
        strategy_opened = position['strategy']
        open_trade_id = position['trade_id']
        side = position['side']

        # Ensure side is 'BUY' for long position closure logic
        if side != 'BUY':
             logger.warning(f"Attempted to close non-long position for {instrument_key} at {timestamp} (Side: {side}). Skipping.")
             return

        revenue = quantity_to_sell * closing_price
        self.current_capital += revenue

        # Calculate Profit/Loss (Gross PnL) - For long positions
        gross_pnl = (closing_price - entry_price) * quantity_to_sell

        # Calculate exit costs (slippage and commission on exit)
        exit_slippage = revenue * self.slippage_pct
        exit_commission = self.commission_per_trade # Fixed commission per trade

        self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

        # Calculate Net PnL
        total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
        total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
        pnl_trade = gross_pnl - total_slippage - total_commission

        # Calculate Trade Duration
        trade_duration = (timestamp - entry_time).total_seconds() if pd.notnull(timestamp) and pd.notnull(entry_time) else None

        # Generate a unique trade ID for the closing trade
        close_trade_id = self.generate_trade_id(timestamp)

        # Record completed trade - Populate all desired columns
        trade_record = {
            'open_trade_id': open_trade_id,
            'close_trade_id': close_trade_id,
            'instrument_key': instrument_key,
            'instrument_type': position.get('instrument_type'),
            'side': side,
            'quantity': quantity_to_sell,
            'entry_price': entry_price,
            'entry_time': entry_time,
            'exit_price': closing_price,
            'exit_time': timestamp,
            'pnl': gross_pnl, # Keep gross PnL for reference
            'strategy_opened': strategy_opened,
            'strategy_closed': 'Backtester_Trigger' if reason not in ['Signal_SELL', 'Forced_Close'] else reason, # Indicate trigger closure or signal close
            'Position_type': position.get('Position_type'),
            'Entry_order_type': position.get('Entry_order_type'),
            'Entry_timestamp': position.get('Entry_timestamp'),
            'Entry_price_trigger': position.get('Entry_price_trigger'),
            'Entry_price_execution': position.get('Entry_price_execution'),
            'Entry_shares': position.get('Entry_shares'),
            'Entry_cost': position.get('Entry_cost'),
            'Entry_signal_type': position.get('Entry_signal_type'),
            'Entry_Trend': position.get('Entry_Trend'),
            'Entry_SMA20': position.get('Entry_SMA20'),
            'Entry_RSI': position.get('Entry_RSI'),
            'Entry_RSI_MA': position.get('Entry_RSI_MA'),
            'Entry_ATR': position.get('Entry_ATR'),
            'Entry_ADX': position.get('Entry_ADX'),
            'Entry_Volatility': position.get('Entry_Volatility'),
            'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'),
            'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'),
            'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'),
            'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'),
            'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'),
            'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'),
            'Instrument_Type': position.get('Instrument_Type'),
            'Currency': position.get('Currency'),
            'Slippage_Entry': position.get('Slippage_Entry'),
            'Commission_Fees_Entry': position.get('Commission_Fees_Entry'),
            'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
            'Risk_Amount': position.get('Risk_Amount'),
            'Reward_Amount': position.get('Reward_Amount'),
            'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
            'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),

            # --- Exit-Specific Columns (Populated from exit_data_point) ---
            'Exit_Trend': exit_data_point.get('Trend'),
            'Exit_signal_type': reason, # The reason is the "signal" for exit
            'Exit_SMA20': exit_data_point.get('SMA20'),
            'Exit_RSI': exit_data_point.get('RSI'),
            'Exit_RSI_MA': exit_data_point.get('RSI_MA'),
            'Exit_ATR': exit_data_point.get('ATR'),
            'Exit_ADX': exit_data_point.get('ADX'),
            'Exit_Volatility': exit_data_point.get('Volatility'),
            'Exit_Breakout_Detected': exit_data_point.get('Breakout_Detected'),
            'Exit_Breakdown_Detected': exit_data_point.get('Breakdown_Detected'),
            'Exit_Bullish_Candlestick_Name': exit_data_point.get('Bullish_Candlestick_Detected'),
            'Exit_Bearish_Candlestick_Name': exit_data_point.get('Bearish_Candlestick_Detected'),
            'Exit_Bullish_Chart_Pattern_Detected': exit_data_point.get('Detected_Bullish_Chart_Pattern_Name'),
            'Exit_Bearish_Chart_Pattern_Detected': exit_data_point.get('Detected_Bearish_Chart_Pattern_Name'),
            'Exit_shares': quantity_to_sell,
            'Exit_cost': 0, # Assuming no cost to exit a long position
            'Exit_revenue': revenue,
            'PnL_trade': pnl_trade,
            'Trade_type': f'Long Close ({reason})', # Indicate the closure type
            'Profit_loss': pnl_trade,
            'Exit_reason': reason,
            'Exit_Order_Type': 'Market' if reason != 'Forced_Close_Error: No_Last_Price' else None, # Assume market order unless error

            # Placeholder for other exit-specific details
            'Current_trailing_stop': position.get('Current_trailing_stop'), # Capture the stop level that was active
            'Trailing_stop_method': position.get('Trailing_stop_method'),
            'Trailing_stop_value': position.get('Trailing_stop_value'), # The value used by the method (e.g., percentage, ATR multiplier)
            'Stop_loss_price': position.get('Stop_loss_price'), # Initial Stop Loss

            'Slippage': total_slippage,
            'Commission_Fees': total_commission,
            'Trade_Duration': trade_duration,
        }
        self.completed_trades.append(trade_record)

        # Remove position
        del self.positions[instrument_key]

        logger.info(f"Closed position {open_trade_id} for {instrument_key} at {timestamp} @ {closing_price} (Reason: {reason}). Net PnL: {pnl_trade:.2f}. Capital left: {self.current_capital:.2f}")
        self.debug_log.append({'type': 'CLOSE_TRIGGER', 'open_trade_id': open_trade_id, 'close_trade_id': close_trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': closing_price, 'quantity': quantity_to_sell, 'net_pnl': pnl_trade, 'reason': reason})


    def run_backtest(self):
        """
        Runs the backtesting simulation bar by bar through the data.
        """
        logger.info("Starting backtest simulation...")

        # Group data by timestamp first, then iterate through timestamps
        grouped_by_time = self.data.groupby('timestamp')

        for timestamp, time_slice_df in grouped_by_time:
            logger.debug(f"Processing timestamp: {timestamp}")

            # --- Check for Stop Loss, Take Profit, and Trailing Stop triggers BEFORE signal generation ---
            # Iterate over a copy of the positions dictionary
            positions_to_check = list(self.positions.keys())
            for instrument_key in positions_to_check:
                # Ensure the position still exists in case it was closed by another trigger
                if instrument_key not in self.positions:
                    continue # Position already closed

                position = self.positions[instrument_key]
                entry_price = position.get('entry_price')
                stop_loss_price = position.get('Stop_loss_price') # Get SL from position details
                reward_amount = position.get('Reward_Amount') # Get Reward from position details
                position_side = position.get('side') # Get side from position details
                trailing_stop_method = position.get('Trailing_stop_method')
                current_trailing_stop = position.get('Current_trailing_stop')
                trailing_stop_value = position.get('Trailing_stop_value') # The value for the method (e.g., percentage)


                # Find the current bar data for this instrument in the current time slice
                current_bar_data = time_slice_df[time_slice_df['instrument_key'] == instrument_key]

                if current_bar_data.empty:
                    logger.warning(f"No data found for instrument {instrument_key} at timestamp {timestamp}. Cannot check triggers.")
                    continue # Skip trigger check for this instrument at this timestamp

                # Get high, low, and close for trigger checks
                current_high = current_bar_data['high'].iloc[0]
                current_low = current_bar_data['low'].iloc[0]
                current_close = current_bar_data['close'].iloc[0] # Use close for TP/SL check consistency


                if pd.isna(current_high) or pd.isna(current_low) or pd.isna(current_close):
                    logger.debug(f"Invalid bar data (high, low, or close) for {instrument_key} at {timestamp}. Cannot check triggers.")
                    continue # Cannot check triggers with invalid bar data

                # Also get the full data point Series for passing to close method
                current_data_point = current_bar_data.iloc[0]


                # --- Check Stop Loss (using Low price for long positions for more realistic check) ---
                if position_side == 'BUY' and pd.notna(stop_loss_price) and current_low <= stop_loss_price:
                    logger.info(f"Stop Loss triggered for {instrument_key} at {timestamp}. Low: {current_low:.4f} <= SL: {stop_loss_price:.4f}. Closing @ {current_close:.4f}")
                    # Close at the trigger price (stop_loss_price) or the bar's low? Using close for simplicity now.
                    # A more realistic model would use the stop_loss_price itself or low, adjusted for slippage.
                    self.close_position_on_trigger(instrument_key, timestamp, current_close, 'Stop Loss', current_data_point)
                    continue # Position closed, move to the next position check


                # --- Check Take Profit (using High price for long positions for more realistic check) ---
                take_profit_price = None
                if pd.notna(entry_price) and pd.notna(reward_amount):
                     # Assuming Reward_Amount is an absolute value add-on to entry price for long positions
                    take_profit_price = entry_price + reward_amount

                if position_side == 'BUY' and pd.notna(take_profit_price) and current_high >= take_profit_price:
                    logger.info(f"Take Profit triggered for {instrument_key} at {timestamp}. High: {current_high:.4f} >= TP: {take_profit_price:.4f}. Closing @ {current_close:.4f}")
                     # Close at the trigger price (take_profit_price) or the bar's high/close? Using close for simplicity now.
                     # A more realistic model would use the take_profit_price itself or high, adjusted for slippage.
                    self.close_position_on_trigger(instrument_key, timestamp, current_close, 'Take Profit', current_data_point)
                    continue # Position closed, move to the next position check

                # --- Check and Update Trailing Stop for Long Positions ---
                if position_side == 'BUY' and pd.notna(trailing_stop_method) and pd.notna(trailing_stop_value):
                     # Ensure Current_trailing_stop is initialized if not already
                    if pd.isna(current_trailing_stop):
                        # Initialize with the initial stop loss price or entry price minus value, depending on method
                        # For simplicity, initialize with entry price - trailing_stop_value (assuming value is an offset)
                        # This needs to be aligned with how 'Trailing_stop_value' is defined in the strategy
                        current_trailing_stop = entry_price - trailing_stop_value # Example initialization
                        self.positions[instrument_key]['Current_trailing_stop'] = current_trailing_stop
                        logger.debug(f"Initialized Current_trailing_stop for {instrument_key} to {current_trailing_stop:.4f} based on Trailing_stop_method: {trailing_stop_method}")


                    new_trailing_stop = None
                    # Example trailing stop logic (needs implementation based on method)
                    if trailing_stop_method == 'Percentage':
                        # Trailing stop value is a percentage below the high
                        new_trailing_stop = current_high * (1 - trailing_stop_value / 100.0)
                    elif trailing_stop_method == 'ATR':
                         # Trailing stop value is an ATR multiplier
                         # You need the current ATR value available in data_point or calculated
                         current_atr = data_point.get('ATR') # Get ATR from data_point
                         if pd.notna(current_atr):
                              new_trailing_stop = current_high - (trailing_stop_value * current_atr)
                         else:
                              logger.warning(f"ATR value not found for {instrument_key} at {timestamp}. Cannot calculate ATR trailing stop.")


                    # Update trailing stop only if it moves favorably (up for long positions)
                    if pd.notna(new_trailing_stop) and new_trailing_stop > current_trailing_stop:
                        self.positions[instrument_key]['Current_trailing_stop'] = new_trailing_stop
                        logger.debug(f"Updated Trailing Stop for {instrument_key} at {timestamp} to {new_trailing_stop:.4f}")


                    # Check if the bar's low breaches the current trailing stop
                    updated_trailing_stop = self.positions[instrument_key].get('Current_trailing_stop') # Get the potentially updated stop
                    if pd.notna(updated_trailing_stop) and current_low <= updated_trailing_stop:
                        logger.info(f"Trailing Stop triggered for {instrument_key} at {timestamp}. Low: {current_low:.4f} <= TS: {updated_trailing_stop:.4f}. Closing @ {current_close:.4f}")
                        # Close at the trailing stop price (updated_trailing_stop) or the bar's low/close? Using close for simplicity.
                        # A more realistic model would use the stop price itself or low, adjusted for slippage.
                        self.close_position_on_trigger(instrument_key, timestamp, current_close, 'Trailing Stop', current_data_point)
                        continue # Position closed, move to the next position check


            # --- Process data for all instruments available at this timestamp for signals ---
            # Filter out instruments for which positions were just closed by triggers
            instruments_for_signal = time_slice_df[~time_slice_df['instrument_key'].isin(self.positions.keys())].copy()
            # Add back instruments for which positions are still open (though they won't generate new signals of the same type usually)
            # This might not be strictly necessary if your strategy logic correctly handles existing positions,
            # but ensures all instrument data points are iterated through for potential trailing stop updates, MFE/MAE calculation, etc.
            instruments_for_signal = pd.concat([instruments_for_signal, time_slice_df[time_slice_df['instrument_key'].isin(self.positions.keys())].copy()])
            instruments_for_signal = instruments_for_signal.drop_duplicates(subset=['instrument_key']).reset_index(drop=True) # Remove duplicates if any


            for index, data_point in instruments_for_signal.iterrows():
                 instrument_key = data_point['instrument_key']
                 current_price = data_point['close'] # Assume close price for execution

                 # Debug capture (already done above, can be kept or moved)
                 # self._debug_timestamps.append(timestamp)
                 # self._debug_close_values.append(current_price)
                 # self._debug_validity.append(pd.notna(current_price))


                 # Ensure current_price is valid for trading (already checked above, but good redundancy)
                 if pd.isna(current_price):
                     logger.debug(f"Skipping signal generation for {instrument_key} at {timestamp} due to invalid close price ({current_price}).")
                     self.debug_log.append({'type': 'SKIP_SIGNAL_PRICE_NAN', 'instrument': instrument_key, 'time': timestamp, 'reason': 'Invalid Price'})
                     continue # Skip this data point if price is invalid


                 # Check for signals from all active strategies for this data point
                 for strategy_name, strategy_instance in self.active_strategies_instances.items():
                     try:
                         # Pass the single data_point (as a Series converted to DataFrame) to the strategy
                         signal = strategy_instance.generate_signal(pd.DataFrame([data_point]))
                         # Ensure signal is a string, handle potential None returns gracefully
                         signal = str(signal).upper() if signal is not None else 'HOLD'

                         if signal in ['BUY', 'SELL']:
                             # Generate a unique trade ID for this potential trade using the bar's timestamp
                             trade_id = self.generate_trade_id(timestamp) # Pass the historical timestamp
                             # Pass the original data_point Series to execute_trade
                             self.execute_trade(trade_id, instrument_key, timestamp, signal, strategy_name, current_price, data_point)

                     except Exception as e:
                         logger.error(f"Error generating signal for {instrument_key} at {timestamp} using strategy '{strategy_name}': {e}", exc_info=True)
                         self.debug_log.append({'type': 'STRATEGY_ERROR', 'instrument': instrument_key, 'time': timestamp, 'strategy': strategy_name, 'error': str(e)})


        # After iterating through all data, close any remaining open positions
        self.close_all_positions(self.data['timestamp'].max()) # Use the timestamp of the last data point as exit time

        logger.info("Backtest simulation completed.")
        logger.info(f"Final Capital: {self.current_capital:.2f}")
        logger.info(f"Number of completed trades: {len(self.completed_trades)}")
        logger.info(f"Number of open positions remaining: {len(self.positions)}")

        # Return completed trades as a DataFrame for analysis
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

            # Ensure numeric columns are numeric
            numeric_cols = [
                'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
            ]
            for col in numeric_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')


            return completed_trades_df
        else:
            logger.warning("No completed trades recorded. Returning empty DataFrame.")
            return pd.DataFrame() # Return empty DataFrame if no trades


    def close_all_positions(self, exit_timestamp: datetime):
        """
        Closes all remaining open positions at the specified exit timestamp.
        Assumes closing at the price of the last available bar for each instrument.
        """
        logger.info(f"Closing all remaining {len(self.positions)} open positions at {exit_timestamp}...")

        # Get the last known price and data point for each instrument with an open position
        last_data_points = self.data.groupby('instrument_key').tail(1).set_index('instrument_key')
        last_prices = last_data_points['close'].to_dict()


        positions_to_close = list(self.positions.keys()) # Iterate over a copy

        for instrument_key in positions_to_close:
            # Check if position still exists (wasn't closed by a trigger just before the end)
            if instrument_key in self.positions:
                position = self.positions[instrument_key]
                closing_price = last_prices.get(instrument_key, np.nan) # Get last price, default to NaN if instrument not found

                # Get the last data point for the instrument to capture exit conditions
                last_data_point = last_data_points.get(instrument_key, pd.Series({})) # Use empty Series if no data found


                if pd.notna(closing_price):
                    # Use the dedicated close method for forced closure
                    self.close_position_on_trigger(instrument_key, exit_timestamp, closing_price, 'Forced_Close', last_data_point)

                else:
                    logger.warning(f"Could not find last price for {instrument_key}. Cannot close position {position['trade_id']}. Logging as unresolved.")
                    # Log as an unresolved position or assume zero PnL

                    # Calculate Trade Duration even if closing price is NaN
                    trade_duration = (exit_timestamp - position.get('entry_time')).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(position.get('entry_time')) else None

                    # Transfer known costs
                    entry_slippage = position.get('Slippage_Entry', 0)
                    entry_commission = position.get('Commission_Fees_Entry', 0)

                    unresolved_trade_record = {
                        'open_trade_id': position.get('trade_id'),
                        'close_trade_id': None, # No closing trade ID
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'),
                        'side': position.get('side'),
                        'quantity': position.get('quantity'),
                        'entry_price': position.get('entry_price'),
                        'entry_time': position.get('entry_time'),
                        'exit_price': None, # No exit price
                        'exit_time': exit_timestamp, # Use the requested exit timestamp
                        'pnl': -(entry_slippage + entry_commission), # Only entry costs are known
                        'strategy_opened': position.get('strategy'),
                        'strategy_closed': 'Backtester_Forced_Close_Error', # Indicate error

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('Entry_RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('Entry_ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Entry_Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': entry_slippage, # Transfer entry slippage
                        'Commission_Fees_Entry': entry_commission, # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),

                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close_Error', # Indicate forced close error
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': position.get('quantity'), # Shares that were supposed to be exited
                        'Exit_cost': 0,
                        'Exit_revenue': 0, # Assuming zero revenue if cannot close
                        'PnL_trade': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Profit_loss': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Exit_reason': 'Backtester_Forced_Close_Error: No_Last_Price', # Reason for exit
                        'Exit_Order_Type': None, # Could not execute exit order

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': position.get('Current_trailing_stop'), # Capture the stop level that was active
                        'Trailing_stop_method': position.get('Trailing_stop_method'),
                        'Trailing_stop_value': position.get('Trailing_stop_value'), # The value used by the method (e.g., percentage)
                        'Stop_loss_price': position.get('Stop_loss_price'), # Initial Stop Loss


                        'Slippage': entry_slippage, # Only entry slippage is known
                        'Commission_Fees': entry_commission, # Only entry commission is known
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(unresolved_trade_record)

                    # Remove position even if it couldn't be closed properly to prevent it from being processed again
                    del self.positions[instrument_key]

                    self.debug_log.append({'type': 'FORCED_CLOSE_ERROR', 'open_trade_id': position['trade_id'], 'instrument': instrument_key, 'time': exit_timestamp, 'reason': 'Last price not available'})


        logger.info("All remaining positions closed.")


    def analyze_backtest_results(self):
        """
        Analyzes the completed trades and provides performance metrics.
        Returns a DataFrame summarizing the analysis.
        """
        logger.info("Analyzing backtest results...")

        if not self.completed_trades:
            logger.warning("No completed trades to analyze.")
            return pd.DataFrame({'Message': ['No completed trades to analyze.']})

        # 1. Access the self.completed_trades list and Create a pandas DataFrame
        trades_df = pd.DataFrame(self.completed_trades)

        # 3. Ensure that relevant columns are converted to appropriate numeric types
        numeric_cols = [
            'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
            'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
            'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
            'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
            'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
            'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
            'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
            'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Include other potentially numeric cols
        ]
        for col in numeric_cols:
            if col in trades_df.columns:
                trades_df[col] = pd.to_numeric(trades_df[col], errors='coerce')

        # Handle potential NaN values during conversion - drop rows where PnL_trade (or pnl) is NaN
        pnl_col_for_analysis = 'PnL_trade' if 'PnL_trade' in trades_df.columns else 'pnl'
        if pnl_col_for_analysis in trades_df.columns:
            # Only consider trades with a valid PnL for core analysis metrics
            trades_df_analysis = trades_df.dropna(subset=[pnl_col_for_analysis]).copy() # Use .copy() to avoid SettingWithCopyWarning
        else:
            logger.warning("Neither 'PnL_trade' nor 'pnl' column found for analysis.")
            return pd.DataFrame({'Message': ['No PnL column found for analysis.']})


        if trades_df_analysis.empty:
            logger.warning("No valid trades after numeric conversion/dropna for analysis. Analysis stopped.")
            return pd.DataFrame({'Message': ['No valid trades after numeric conversion/dropna for analysis.']})


        # 4. Update the calculation of basic performance metrics using 'PnL_trade'
        total_trades = len(trades_df_analysis)
        total_pnl = trades_df_analysis[pnl_col_for_analysis].sum()

        winning_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] > 0]
        losing_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] < 0]
        breakeven_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] == 0]

        num_winning = len(winning_trades)
        num_losing = len(losing_trades)
        num_breakeven = len(breakeven_trades)

        win_rate = (num_winning / total_trades) * 100 if total_trades > 0 else 0
        avg_win = winning_trades[pnl_col_for_analysis].mean() if num_winning > 0 else 0
        avg_loss = losing_trades[pnl_col_for_analysis].mean() if num_losing > 0 else 0
        expectancy = (win_rate / 100) * avg_win + ((100 - win_rate) / 100) * avg_loss if total_trades > 0 else 0

        # 5. Update Max Drawdown calculation to use 'PnL_trade' and sort by exit time
        # Calculate cumulative PnL and then cumulative capital
        trades_df_analysis = trades_df_analysis.sort_values(by='exit_time') # Sort by exit time for cumulative calculation

        trades_df_analysis['cumulative_pnl'] = trades_df_analysis[pnl_col_for_analysis].cumsum()

        # Add initial capital to cumulative PnL
        trades_df_analysis['cumulative_capital'] = self.initial_capital + trades_df_analysis['cumulative_pnl']

        # Calculate peak capital up to each point
        trades_df_analysis['peak_capital'] = trades_df_analysis['cumulative_capital'].cummax()

        # Calculate drawdown at each point
        trades_df_analysis['drawdown'] = trades_df_analysis['peak_capital'] - trades_df_analysis['cumulative_capital']

        # Calculate percentage drawdown
        # Avoid division by zero if peak_capital is 0 or None
        trades_df_analysis['pct_drawdown'] = trades_df_analysis.apply(
            lambda row: (row['drawdown'] / row['peak_capital']) * 100 if row['peak_capital'] > 0 and pd.notna(row['peak_capital']) else 0,
            axis=1
        )


        max_drawdown_amount = trades_df_analysis['drawdown'].max() if not trades_df_analysis.empty else 0
        max_drawdown_pct = trades_df_analysis['pct_drawdown'].max() if not trades_df_analysis.empty else 0


        # 6. Update analysis summary metric names
        analysis_summary = {
            'Metric': [
                'Initial Capital',
                'Final Capital',
                'Total PnL (Net)', # Indicate Net PnL
                'Total Trades',
                'Winning Trades (Net)', # Indicate Net PnL
                'Losing Trades (Net)', # Indicate Net PnL
                'Breakeven Trades (Net)', # Indicate Net PnL
                'Win Rate (%) (Net PnL)', # Indicate Net PnL
                'Average Win (Net)', # Indicate Net PnL
                'Average Loss (Net)', # Indicate Net PnL
                'Expectancy per Trade (Net)', # Indicate Net PnL
                'Max Drawdown (Amount)',
                'Max Drawdown (%)',
            ],
            'Value': [
                self.initial_capital,
                self.current_capital,
                round(total_pnl, 2), # Format to 2 decimal places
                total_trades,
                num_winning,
                num_losing,
                num_breakeven,
                round(win_rate, 2), # Format to 2 decimal places
                round(avg_win, 2),
                round(avg_loss, 2),
                round(expectancy, 2),
                round(max_drawdown_amount, 2),
                round(max_drawdown_pct, 2),
            ]
        }

        analysis_df = pd.DataFrame(analysis_summary)

        logger.info("Backtest analysis completed.")
        # You can print the analysis_df here or return it
        # print("\n--- Backtest Analysis Summary ---")
        # display(analysis_df) # Use display for notebooks

        # 8. Ensure the method returns the updated analysis summary DataFrame
        return analysis_df

    def get_completed_trades(self):
        """Returns a DataFrame of completed trades."""
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

                    # Ensure numeric columns are numeric
                    numeric_cols = [
                        'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                        'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                        'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                        'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                        'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                        'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                        'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                        'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                        'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
                    ]
                    for col in numeric_cols:
                        if col in completed_trades_df.columns:
                            completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')

                    return completed_trades_df
                else:
                    return pd.DataFrame() # Return empty DataFrame if no trades

Logging level set to DEBUG for test.


**Reasoning**:
The previous step implemented the basic trailing stop logic. Now, complete the subtask by ensuring the logic correctly handles different trailing stop methods ('Percentage' and 'ATR') and updates the 'Current_trailing_stop' value based on the current bar's high for long positions. Also, add the check to trigger position closure if the bar's low breaches the 'Current_trailing_stop'.



In [7]:
# _1113_6BacktesterV3.py

import pandas as pd
import numpy as np
import logging
from datetime import datetime
import uuid
import sys
# from _012_instruments import get_instrument_type
# --- Logging Configuration ---
logger = logging.getLogger(__name__)



logger.setLevel(logging.DEBUG)

# Also ensure the root logger has a handler and is set to DEBUG,
# in case basicConfig was called elsewhere previously.
if not logging.root.handlers:
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
else:
    # If handlers exist, ensure at least one handler's level is DEBUG
    # and the root logger's level is DEBUG
    logging.root.setLevel(logging.DEBUG)
    handler_found = False
    for handler in logging.root.handlers:
        if isinstance(handler, logging.StreamHandler) and handler.stream in [sys.stdout, sys.stderr]:
            handler.setLevel(logging.DEBUG)
            handler_found = True
    # If no suitable handler is found (e.g., only file handlers), add a StreamHandler
    if not handler_found:
         stream_handler = logging.StreamHandler(sys.stdout)
         stream_handler.setLevel(logging.DEBUG)
         formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s - %(message)s')
         stream_handler.setFormatter(formatter)
         logging.root.addHandler(stream_handler)


print("Logging level set to DEBUG for test.")


class BacktesterV3:
    """
    A simple backtesting engine for evaluating trading strategies.
    Processes historical data bar by bar, generates signals, and simulates trades.
    """
    def __init__(self, data: pd.DataFrame, instrument_keys: list, active_strategies_instances: dict, initial_capital: float):
        """
        Includes the same parameters as the original __init__

        Initializes the Backtester.

        Args:
            data: A pandas DataFrame containing historical market data for all instruments,
                expected to have columns like 'timestamp', 'instrument_key',
                'open', 'high', 'low', 'close', 'volume', etc. It is also expected
                to contain pre-calculated indicator and pattern columns used by
                the strategies and for recording trade details.
            instrument_keys: A list of unique instrument keys present in the data.
            active_strategies_instances: A dictionary where keys are strategy names
                                        (strings) and values are instantiated strategy
                                        objects with a `generate_signal(data_point)` method.
            initial_capital: The starting capital for the backtest simulation.
        """
        if data is None or data.empty:
            raise ValueError("Input data DataFrame is None or empty.")
        if not isinstance(data, pd.DataFrame):
            raise TypeError("Input 'data' must be a pandas DataFrame.")
        if data.index.name is not None:
            logger.warning("Input data index is not None. Consider resetting the index before passing to Backtester.")


        # Ensure essential columns are present and sorted
        required_columns = ['timestamp', 'instrument_key', 'open', 'high', 'low', 'close', 'high', 'low'] # Added high, low to required
        # Define columns expected to be in the input data for recording trade details.
        # These are typically pre-calculated indicators or pattern detection results.
        entry_exit_data_columns_expected = [
            'Trend', 'SMA20', 'RSI', 'RSIMA', 'ATR', 'ADX', 'Volatility',
            'Breakout_Detected', 'Breakdown_Detected', 'Bullish_Candlestick_Detected',
            'Bearish_Candlestick_Detected', 'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
            'name', 'interval', 'Currency',
            'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
            'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price' # Added Trailing_stop_value
        ]

        # The backtester expects these columns to be pre-calculated and provided in the input data.
        # Strategies generate signals based on these columns, and their values at the time of
        # entry and exit are recorded in the completed_trades DataFrame.


        # Perform a relaxed check: log a warning if potential entry/exit columns from data are missing
        missing_data_cols = [col for col in entry_exit_data_columns_expected if col not in data.columns]
        if missing_data_cols:
            logger.warning(f"Input data is missing expected indicator/pattern columns: {missing_data_cols}. These will be recorded as None in trade records. Ensure your data preparation includes these columns if strategies or analysis depend on them.")


        # Ensure mandatory required columns are present
        missing_required = [col for col in required_columns if col not in data.columns]
        if missing_required:
            raise ValueError(f"Input data is missing mandatory required columns: {missing_required}")


        # Ensure timestamp is datetime and sorted
        try:
            if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
                data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', utc=True) # Convert to UTC
            # Drop rows where timestamp conversion failed
            data = data.dropna(subset=['timestamp'])
            # Sort by timestamp and then instrument_key to process bars chronologically per instrument
            self.data = data.sort_values(by=['timestamp', 'instrument_key']).reset_index(drop=True)
        except Exception as e:
            raise ValueError(f"Error processing timestamp column in data: {e}")


        self.instrument_keys = instrument_keys
        self.active_strategies_instances = active_strategies_instances
        self.initial_capital = initial_capital

        # --- Backtesting State Variables ---
        self.current_capital = initial_capital
        self.positions = {}  # Dictionary to track open positions {instrument_key: {...entry details...}}
        self.completed_trades = [] # List to store completed trades
        self.trade_id_counter = 0 # Simple counter for trade IDs
        self.debug_log = [] # List to store debug information

        # Debug lists to capture values
        self._debug_timestamps = []
        self._debug_close_values = []
        self._debug_validity = []

        # Simple Slippage and Commission model (can be customized)
        self.slippage_pct = 0.001  # 0.1% slippage per trade
        self.commission_per_trade = 0.01 # $0.01 fixed commission per trade


        logger.info(f"BacktesterV2 initialized with {len(self.instrument_keys)} instruments and {len(self.active_strategies_instances)} active strategies.")
        logger.info(f"Initial Capital: {self.initial_capital}")
        logger.info(f"Data shape for backtesting: {self.data.shape}")


    def generate_trade_id(self, timestamp: datetime):
        """Generates a unique trade ID using a provided timestamp."""
        # Using microseconds to increase the chance of uniqueness
        return timestamp.strftime('%Y%m%d%H%M%S%f')


    def execute_trade(self, trade_id: str, instrument_key: str, timestamp: datetime, signal: str, strategy_name: str, price: float, data_point: pd.Series):
        """
        Simulates executing a trade based on a signal.

        Args:
            trade_id: Unique identifier for the trade.
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the trade execution (bar close time).
            signal: The trading signal ('BUY' or 'SELL').
            strategy_name: The name of the strategy generating the signal.
            price: The execution price (typically the close price of the bar).
            data_point: The pandas Series representing the data row for this bar. This Series
                        is expected to contain pre-calculated indicator and pattern data
                        used for entry/exit conditions and recording.
        """
        # Determine instrument type to handle lot size/quantity logic
        # instrument_type = get_instrument_type(instrument_key) # Removed due to import error
        instrument_type = 'Unknown' # Placeholder


        # Simple fixed quantity logic (can be replaced with dynamic position sizing)
        quantity_to_trade = 1 # Example: trade 1 unit/lot

        if signal == 'BUY':
            # Check if we already have a position in this instrument (optional, depending on strategy)
            if instrument_key not in self.positions:
                # Simulate buying
                cost = quantity_to_trade * price
                # Check if we have enough capital
                if self.current_capital >= cost:
                    self.current_capital -= cost

                    # Calculate entry costs (slippage and commission on entry)
                    entry_slippage = cost * self.slippage_pct
                    entry_commission = self.commission_per_trade # Fixed commission per trade


                    self.current_capital -= (entry_slippage + entry_commission) # Deduct costs from capital

                    # Capture entry-specific details from the data_point and other variables
                    self.positions[instrument_key] = {
                        'quantity': quantity_to_trade,
                        'entry_price': price, # This is the execution price for this simple model
                        'entry_time': timestamp,
                        'strategy': strategy_name,
                        'trade_id': trade_id,
                        'instrument_type': instrument_type, # Store instrument type (now placeholder)
                        'side': 'BUY', # Store trade side

                        # --- Entry-Specific Columns (Populated from data_point at Entry) ---
                        'Strategy_name': strategy_name,
                        'instrument_key': instrument_key,
                        'name': data_point.get('name'), # Use .get() to avoid errors if column is missing
                        'interval': data_point.get('interval'),
                        'Position_type': 'Long', # Assuming BUY means Long position
                        'Entry_order_type': 'Market', # Assuming market order execution on close
                        'Entry_timestamp': timestamp,
                        'Entry_price_trigger': None, # Not explicitly handled in this simple model
                        'Entry_price_execution': price,
                        'Entry_shares': quantity_to_trade, # Using quantity_to_trade as shares
                        'Entry_cost': cost, # Gross cost before fees
                        'Entry_signal_type': signal, # Ensure signal is captured
                        'Entry_Trend': data_point.get('Trend'), # Capture Trend at Entry
                        'Entry_SMA20': data_point.get('SMA20'), # Capture SMA20 at Entry
                        'Entry_RSI': data_point.get('RSI'), # Capture RSI at Entry
                        'Entry_RSI_MA': data_point.get('RSI_MA'), # Capture RSI_MA at Entry
                        'Entry_ATR': data_point.get('ATR'), # Capture ATR at Entry
                        'Entry_ADX': data_point.get('ADX'), # Capture ADX at Entry
                        'Entry_Volatility': data_point.get('Volatility'), # Capture Volatility at Entry
                        'Entry_Breakout_Detected': data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Entry
                        'Entry_Breakdown_Detected': data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Entry
                        # Corrected column names to match expected input data
                        'Entry_Bullish_Candlestick_Name': data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Entry
                        'Entry_Bearish_Candlestick_Name': data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Entry
                        'Entry_Bullish_Chart_Pattern_Name': data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Entry
                        'Entry_Bearish_Chart_Pattern_Name': data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Entry
                        'Instrument_Type': instrument_type, # Store instrument type (now placeholder)
                        'Currency': data_point.get('Currency'),
                        'Slippage_Entry': entry_slippage, # Store entry slippage
                        'Commission_Fees_Entry': entry_commission, # Store entry commission

                        # Placeholder for other entry-specific details that might be calculated by strategy (e.g., initial stop/target)
                        'Initial_Stop_Loss_Distance (%)': data_point.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': data_point.get('Risk_Amount'),
                        'Reward_Amount': data_point.get('Reward_Amount'),


                        # Placeholders for exit/other info that will be filled on close
                        # These fields are included here so the structure is consistent for retrieval on exit,
                        # even though their values are None at the time of entry.
                         'Max_Favorable_Excursion_MFE': None, # Will be calculated on exit
                         'Max_Adverse_Excursion_MAE': None, # Will be calculated on exit
                        'Current_trailing_stop': None, # Need logic for trailing stops
                        'Trailing_stop_method': data_point.get('Trailing_stop_method'), # Capture method from data point
                        'Trailing_stop_value': data_point.get('Trailing_stop_value'), # Capture initial value from data point (if applicable)
                        'Stop_loss_price': data_point.get('Stop_loss_price'), # Capture initial SL from data point


                        'Exit_Trend': None, 'Exit_signal_type': None, 'Exit_SMA20': None,
                        'Exit_RSI': None, 'Exit_RSI_MA': None, 'Exit_ATR': None, 'Exit_ADX': None,
                        'Exit_Volatility': None, 'Exit_Breakout_Detected': None,
                        'Exit_Breakdown_Detected': None, 'Exit_Bullish_Candlestick_Name': None,
                        'Exit_Bearish_Candlestick_Name': None, 'Exit_Bullish_Chart_Pattern_Detected': None,
                        'Exit_Bearish_Chart_Pattern_Detected': None, 'Exit_shares': None,
                        'Exit_cost': None, 'Exit_revenue': None, 'PnL_trade': None,
                        'Trade_type': None, 'Profit_loss': None, 'Exit_reason': None,
                        'Slippage': None, 'Commission_Fees': None, 'Trade_Duration': None,
                        'Exit_Order_Type': None
                    }
                    # Initialize 'Current_trailing_stop' with the initial 'Stop_loss_price' if a trailing method is specified
                    if pd.notna(self.positions[instrument_key].get('Trailing_stop_method')):
                         self.positions[instrument_key]['Current_trailing_stop'] = self.positions[instrument_key].get('Stop_loss_price')
                         logger.debug(f"Initialized Trailing Stop for {instrument_key} at {timestamp} to initial SL: {self.positions[instrument_key].get('Current_trailing_stop'):.4f}")


                    # --- Add debug logging for Entry columns here ---
                    logger.debug(f"DEBUG Entry Data Point for {instrument_key} at {timestamp}:")
                    debug_cols_to_check = [
                        'Trend', 'SMA20', 'RSI', 'RSI_MA', 'ATR', 'ADX', 'Volatility',
                        'Breakout_Detected', 'Breakdown_Detected',
                        # Corrected debug column names to match expected input data
                        'Bullish_Candlestick_Detected', 'Bearish_Candlestick_Detected',
                        'Detected_Bullish_Chart_Pattern_Name', 'Detected_Bearish_Chart_Pattern_Name',
                        'Currency', 'Initial_Stop_Loss_Distance (%)', 'Risk_Amount', 'Reward_Amount',
                        'Max_Favorable_Excursion_MFE', 'Max_Adverse_Excursion_MAE', 'Current_trailing_stop',
                        'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price', 'Exit_Trend',
                        'Exit_signal_type', 'Exit_SMA20', 'Exit_RSI', 'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX',
                        'Exit_Volatility', 'Exit_Breakout_Detected', 'Exit_Breakdown_Detected',
                        'Exit_Bullish_Candlestick_Name', 'Exit_Bearish_Candlestick_Name',
                        'Exit_Bullish_Chart_Pattern_Detected', 'Exit_Bearish_Chart_Pattern_Detected',
                        'Exit_cost'
                    ]
                    for col in debug_cols_to_check:
                         logger.debug(f"  {col}: {data_point.get(col, 'Column Not Found or None')}")
                    # --- End Debug Logging ---


                    logger.info(f"Executed BUY trade {trade_id} for {instrument_key} at {timestamp} @ {price} (Qty: {quantity_to_trade}). Costs: Slippage={entry_slippage:.4f}, Commission={entry_commission:.2f}. Capital left: {self.current_capital:.2f}")
                    self.debug_log.append({'type': 'BUY', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'slippage': entry_slippage, 'commission': entry_commission})
                else:
                    logger.warning(f"Insufficient capital ({self.current_capital:.2f}) to BUY {instrument_key} at {price} (Cost: {cost:.2f}). Skipping trade {trade_id}.")
                    self.debug_log.append({'type': 'SKIP_BUY_CAPITAL', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Insufficient Capital'})

            else:
                # Already in a position, maybe add to it or skip depending on strategy rules
                logger.debug(f"Skipping BUY signal for {instrument_key} at {timestamp}. Already in a position.")
                self.debug_log.append({'type': 'SKIP_BUY_POSITION', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'quantity': quantity_to_trade, 'strategy': strategy_name, 'reason': 'Already in Position'})


        elif signal == 'SELL':
            # For backtesting, a 'SELL' signal usually means closing a long position or opening a short position
            # Let's assume 'SELL' means closing a long position if one exists for simplicity in this example.
            # For a shorting strategy, you'd need different logic.
            if instrument_key in self.positions and self.positions[instrument_key]['side'] == 'BUY':
                # Use the dedicated close method
                self.close_position_on_trigger(instrument_key, timestamp, price, 'Signal_SELL', data_point)

            else:
                # No matching long position to close, or maybe a shorting signal
                # For this simple backtester, we'll just log and skip if no long position
                logger.debug(f"Skipping SELL signal for {instrument_key} at {timestamp}. No matching long position to close.")
                self.debug_log.append({'type': 'SKIP_SELL_NO_LONG', 'trade_id': trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': price, 'strategy': strategy_name, 'reason': 'No Long Position'})


    def close_position_on_trigger(self, instrument_key: str, timestamp: datetime, closing_price: float, reason: str, exit_data_point: pd.Series):
        """
        Closes an open position for a specific instrument due to a trigger (SL, TP, etc.).

        Args:
            instrument_key: The instrument key for the trade.
            timestamp: The timestamp of the closure (bar close time).
            closing_price: The price at which the position is closed.
            reason: The reason for closing ('Stop Loss', 'Take Profit', 'Signal_SELL', 'Forced_Close', 'Trailing Stop').
            exit_data_point: The pandas Series representing the data row for this bar at exit.
        """
        if instrument_key not in self.positions:
            logger.warning(f"Attempted to close non-existent position for {instrument_key} at {timestamp} (Reason: {reason}).")
            return

        position = self.positions[instrument_key]

        quantity_to_sell = position['quantity']
        entry_price = position['entry_price']
        entry_time = position['entry_time']
        strategy_opened = position['strategy']
        open_trade_id = position['trade_id']
        side = position['side']

        # Ensure side is 'BUY' for long position closure logic
        if side != 'BUY':
             logger.warning(f"Attempted to close non-long position for {instrument_key} at {timestamp} (Side: {side}). Skipping.")
             return

        revenue = quantity_to_sell * closing_price
        self.current_capital += revenue

        # Calculate Profit/Loss (Gross PnL) - For long positions
        gross_pnl = (closing_price - entry_price) * quantity_to_sell

        # Calculate exit costs (slippage and commission on exit)
        exit_slippage = revenue * self.slippage_pct
        exit_commission = self.commission_per_trade # Fixed commission per trade

        self.current_capital -= (exit_slippage + exit_commission) # Deduct costs from capital

        # Calculate Net PnL
        total_slippage = position.get('Slippage_Entry', 0) + exit_slippage
        total_commission = position.get('Commission_Fees_Entry', 0) + exit_commission
        pnl_trade = gross_pnl - total_slippage - total_commission

        # Calculate Trade Duration
        trade_duration = (timestamp - entry_time).total_seconds() if pd.notnull(timestamp) and pd.notnull(entry_time) else None

        # Generate a unique trade ID for the closing trade
        close_trade_id = self.generate_trade_id(timestamp)

        # Record completed trade - Populate all desired columns
        trade_record = {
            'open_trade_id': open_trade_id,
            'close_trade_id': close_trade_id,
            'instrument_key': instrument_key,
            'instrument_type': position.get('instrument_type'),
            'side': side,
            'quantity': quantity_to_sell,
            'entry_price': entry_price,
            'entry_time': entry_time,
            'exit_price': closing_price,
            'exit_time': timestamp,
            'pnl': gross_pnl, # Keep gross PnL for reference
            'strategy_opened': strategy_opened,
            'strategy_closed': 'Backtester_Trigger' if reason not in ['Signal_SELL', 'Forced_Close'] else reason, # Indicate trigger closure or signal close
            'Position_type': position.get('Position_type'),
            'Entry_order_type': position.get('Entry_order_type'),
            'Entry_timestamp': position.get('Entry_timestamp'),
            'Entry_price_trigger': position.get('Entry_price_trigger'),
            'Entry_price_execution': position.get('Entry_price_execution'),
            'Entry_shares': position.get('Entry_shares'),
            'Entry_cost': position.get('Entry_cost'),
            'Entry_signal_type': position.get('Entry_signal_type'),
            'Entry_Trend': position.get('Entry_Trend'),
            'Entry_SMA20': position.get('Entry_SMA20'),
            'Entry_RSI': position.get('Entry_RSI'),
            'Entry_RSI_MA': position.get('Entry_RSI_MA'),
            'Entry_ATR': position.get('Entry_ATR'),
            'Entry_ADX': position.get('Entry_ADX'),
            'Entry_Volatility': position.get('Entry_Volatility'),
            'Entry_Breakout_Detected': position.get('Entry_Breakout_Detected'),
            'Entry_Breakdown_Detected': position.get('Breakdown_Detected'),
            'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'),
            'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'),
            'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'),
            'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'),
            'Instrument_Type': position.get('Instrument_Type'),
            'Currency': position.get('Currency'),
            'Slippage_Entry': position.get('Slippage_Entry'),
            'Commission_Fees_Entry': position.get('Commission_Fees_Entry'),
            'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
            'Risk_Amount': position.get('Risk_Amount'),
            'Reward_Amount': position.get('Reward_Amount'),
            'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
            'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),

            # --- Exit-Specific Columns (Populated from exit_data_point) ---
            'Exit_Trend': exit_data_point.get('Trend'),
            'Exit_signal_type': reason, # The reason is the "signal" for exit
            'Exit_SMA20': exit_data_point.get('SMA20'),
            'Exit_RSI': exit_data_point.get('RSI'),
            'Exit_RSI_MA': exit_data_point.get('RSI_MA'),
            'Exit_ATR': exit_data_point.get('ATR'),
            'Exit_ADX': exit_data_point.get('ADX'),
            'Exit_Volatility': exit_data_point.get('Volatility'),
            'Exit_Breakout_Detected': exit_data_point.get('Breakout_Detected'),
            'Exit_Breakdown_Detected': exit_data_point.get('Breakdown_Detected'),
            'Exit_Bullish_Candlestick_Name': exit_data_point.get('Bullish_Candlestick_Detected'),
            'Exit_Bearish_Candlestick_Name': exit_data_point.get('Bearish_Candlestick_Detected'),
            'Exit_Bullish_Chart_Pattern_Detected': exit_data_point.get('Detected_Bullish_Chart_Pattern_Name'),
            'Exit_Bearish_Chart_Pattern_Detected': exit_data_point.get('Detected_Bearish_Chart_Pattern_Name'),
            'Exit_shares': quantity_to_sell,
            'Exit_cost': 0, # Assuming no cost to exit a long position
            'Exit_revenue': revenue,
            'PnL_trade': pnl_trade,
            'Trade_type': f'Long Close ({reason})', # Indicate the closure type
            'Profit_loss': pnl_trade,
            'Exit_reason': reason,
            'Exit_Order_Type': 'Market' if reason != 'Forced_Close_Error: No_Last_Price' else None, # Assume market order unless error

            # Placeholder for other exit-specific details
            'Current_trailing_stop': position.get('Current_trailing_stop'), # Capture the stop level that was active
            'Trailing_stop_method': position.get('Trailing_stop_method'),
            'Trailing_stop_value': position.get('Trailing_stop_value'), # The value used by the method (e.g., percentage)
            'Stop_loss_price': position.get('Stop_loss_price'), # Initial Stop Loss

            'Slippage': total_slippage,
            'Commission_Fees': total_commission,
            'Trade_Duration': trade_duration,
        }
        self.completed_trades.append(trade_record)

        # Remove position
        del self.positions[instrument_key]

        logger.info(f"Closed position {open_trade_id} for {instrument_key} at {timestamp} @ {closing_price} (Reason: {reason}). Net PnL: {pnl_trade:.2f}. Capital left: {self.current_capital:.2f}")
        self.debug_log.append({'type': 'CLOSE_TRIGGER', 'open_trade_id': open_trade_id, 'close_trade_id': close_trade_id, 'instrument': instrument_key, 'time': timestamp, 'price': closing_price, 'quantity': quantity_to_sell, 'net_pnl': pnl_trade, 'reason': reason})


    def run_backtest(self):
        """
        Runs the backtesting simulation bar by bar through the data.
        """
        logger.info("Starting backtest simulation...")

        # Group data by timestamp first, then iterate through timestamps
        grouped_by_time = self.data.groupby('timestamp')

        for timestamp, time_slice_df in grouped_by_time:
            logger.debug(f"Processing timestamp: {timestamp}")

            # --- Check for Stop Loss, Take Profit, and Trailing Stop triggers BEFORE signal generation ---
            # Iterate over a copy of the positions dictionary
            positions_to_check = list(self.positions.keys())
            for instrument_key in positions_to_check:
                # Ensure the position still exists in case it was closed by another trigger
                if instrument_key not in self.positions:
                    continue # Position already closed

                position = self.positions[instrument_key]
                entry_price = position.get('entry_price')
                stop_loss_price = position.get('Stop_loss_price') # Get SL from position details
                reward_amount = position.get('Reward_Amount') # Get Reward from position details
                position_side = position.get('side') # Get side from position details
                trailing_stop_method = position.get('Trailing_stop_method')
                current_trailing_stop = position.get('Current_trailing_stop')
                trailing_stop_value = position.get('Trailing_stop_value') # The value used by the method (e.g., percentage)


                # Find the current bar data for this instrument in the current time slice
                current_bar_data = time_slice_df[time_slice_df['instrument_key'] == instrument_key]

                if current_bar_data.empty:
                    logger.warning(f"No data found for instrument {instrument_key} at timestamp {timestamp}. Cannot check triggers.")
                    continue # Skip trigger check for this instrument at this timestamp

                # Get high, low, and close for trigger checks
                current_high = current_bar_data['high'].iloc[0]
                current_low = current_bar_data['low'].iloc[0]
                current_close = current_bar_data['close'].iloc[0] # Use close for TP/SL check consistency


                if pd.isna(current_high) or pd.isna(current_low) or pd.isna(current_close):
                    logger.debug(f"Invalid bar data (high, low, or close) for {instrument_key} at {timestamp}. Cannot check triggers.")
                    continue # Cannot check triggers with invalid bar data

                # Also get the full data point Series for passing to close method
                current_data_point = current_bar_data.iloc[0]


                # --- Check Stop Loss (using Low price for long positions for more realistic check) ---
                if position_side == 'BUY' and pd.notna(stop_loss_price) and current_low <= stop_loss_price:
                    logger.info(f"Stop Loss triggered for {instrument_key} at {timestamp}. Low: {current_low:.4f} <= SL: {stop_loss_price:.4f}. Closing @ {current_close:.4f}")
                    # Close at the trigger price (stop_loss_price) or the bar's low? Using close for simplicity now.
                    # A more realistic model would use the stop_loss_price itself or low, adjusted for slippage.
                    self.close_position_on_trigger(instrument_key, timestamp, current_close, 'Stop Loss', current_data_point)
                    continue # Position closed, move to the next position check


                # --- Check Take Profit (using High price for long positions for more realistic check) ---
                take_profit_price = None
                if pd.notna(entry_price) and pd.notna(reward_amount):
                     # Assuming Reward_Amount is an absolute value add-on to entry price for long positions
                    take_profit_price = entry_price + reward_amount

                if position_side == 'BUY' and pd.notna(take_profit_price) and current_high >= take_profit_price:
                    logger.info(f"Take Profit triggered for {instrument_key} at {timestamp}. High: {current_high:.4f} >= TP: {take_profit_price:.4f}. Closing @ {current_close:.4f}")
                     # Close at the trigger price (take_profit_price) or the bar's high/close? Using close for simplicity now.
                     # A more realistic model would use the take_profit_price itself or high, adjusted for slippage.
                    self.close_position_on_trigger(instrument_key, timestamp, current_close, 'Take Profit', current_data_point)
                    continue # Position closed, move to the next position check

                # --- Check and Update Trailing Stop for Long Positions ---
                if position_side == 'BUY' and pd.notna(trailing_stop_method) and pd.notna(trailing_stop_value):
                     # Ensure Current_trailing_stop is initialized if not already
                    if pd.isna(current_trailing_stop):
                        # Initialize with the initial stop loss price or entry price minus value, depending on method
                        # For simplicity, initialize with entry price - trailing_stop_value (assuming value is an offset)
                        # This needs to be aligned with how 'Trailing_stop_value' is defined in the strategy
                        current_trailing_stop = entry_price - trailing_stop_value # Example initialization
                        self.positions[instrument_key]['Current_trailing_stop'] = current_trailing_stop
                        logger.debug(f"Initialized Current_trailing_stop for {instrument_key} at {timestamp} to {current_trailing_stop:.4f} based on Trailing_stop_method: {trailing_stop_method}")


                    new_trailing_stop = None
                    # Example trailing stop logic (needs implementation based on method)
                    if trailing_stop_method == 'Percentage':
                        # Trailing stop value is a percentage below the high
                        new_trailing_stop = current_high * (1 - trailing_stop_value / 100.0)
                    elif trailing_stop_method == 'ATR':
                         # Trailing stop value is an ATR multiplier
                         # You need the current ATR value available in data_point or calculated
                         current_atr = data_point.get('ATR') # Get ATR from data_point
                         if pd.notna(current_atr):
                              new_trailing_stop = current_high - (trailing_stop_value * current_atr)
                         else:
                              logger.warning(f"ATR value not found for {instrument_key} at {timestamp}. Cannot calculate ATR trailing stop.")


                    # Update trailing stop only if it moves favorably (up for long positions)
                    if pd.notna(new_trailing_stop) and pd.notna(current_trailing_stop) and new_trailing_stop > current_trailing_stop:
                        self.positions[instrument_key]['Current_trailing_stop'] = new_trailing_stop
                        logger.debug(f"Updated Trailing Stop for {instrument_key} at {timestamp} to {new_trailing_stop:.4f}")
                    elif pd.isna(current_trailing_stop) and pd.notna(new_trailing_stop):
                         # Initialize if it was NaN previously and a new stop can be calculated
                         self.positions[instrument_key]['Current_trailing_stop'] = new_trailing_stop
                         logger.debug(f"Initialized Trailing Stop for {instrument_key} at {timestamp} to {new_trailing_stop:.4f} based on first valid calculation.")


                    # Check if the bar's low breaches the current trailing stop
                    updated_trailing_stop = self.positions[instrument_key].get('Current_trailing_stop') # Get the potentially updated stop
                    if pd.notna(updated_trailing_stop) and current_low <= updated_trailing_stop:
                        logger.info(f"Trailing Stop triggered for {instrument_key} at {timestamp}. Low: {current_low:.4f} <= TS: {updated_trailing_stop:.4f}. Closing @ {current_close:.4f}")
                        # Close at the trailing stop price (updated_trailing_stop) or the bar's low/close? Using close for simplicity.
                        # A more realistic model would use the stop price itself or low, adjusted for slippage.
                        self.close_position_on_trigger(instrument_key, timestamp, current_close, 'Trailing Stop', current_data_point)
                        continue # Position closed, move to the next position check


            # --- Process data for all instruments available at this timestamp for signals ---
            # Filter out instruments for which positions were just closed by triggers
            instruments_for_signal = time_slice_df[~time_slice_df['instrument_key'].isin(self.positions.keys())].copy()
            # Add back instruments for which positions are still open (though they won't generate new signals of the same type usually)
            # This might not be strictly necessary if your strategy logic correctly handles existing positions,
            # but ensures all instrument data points are iterated through for potential trailing stop updates, MFE/MAE calculation, etc.
            instruments_for_signal = pd.concat([instruments_for_signal, time_slice_df[time_slice_df['instrument_key'].isin(self.positions.keys())].copy()])
            instruments_for_signal = instruments_for_signal.drop_duplicates(subset=['instrument_key']).reset_index(drop=True) # Remove duplicates if any


            for index, data_point in instruments_for_signal.iterrows():
                 instrument_key = data_point['instrument_key']
                 current_price = data_point['close'] # Assume close price for execution

                 # Debug capture (already done above, can be kept or moved)
                 # self._debug_timestamps.append(timestamp)
                 # self._debug_close_values.append(current_price)
                 # self._debug_validity.append(pd.notna(current_price))


                 # Ensure current_price is valid for trading (already checked above, but good redundancy)
                 if pd.isna(current_price):
                     logger.debug(f"Skipping signal generation for {instrument_key} at {timestamp} due to invalid close price ({current_price}).")
                     self.debug_log.append({'type': 'SKIP_SIGNAL_PRICE_NAN', 'instrument': instrument_key, 'time': timestamp, 'reason': 'Invalid Price'})
                     continue # Skip this data point if price is invalid


                 # Check for signals from all active strategies for this data point
                 for strategy_name, strategy_instance in self.active_strategies_instances.items():
                     try:
                         # Pass the single data_point (as a Series converted to DataFrame) to the strategy
                         signal = strategy_instance.generate_signal(pd.DataFrame([data_point]))
                         # Ensure signal is a string, handle potential None returns gracefully
                         signal = str(signal).upper() if signal is not None else 'HOLD'

                         if signal in ['BUY', 'SELL']:
                             # Generate a unique trade ID for this potential trade using the bar's timestamp
                             trade_id = self.generate_trade_id(timestamp) # Pass the historical timestamp
                             # Pass the original data_point Series to execute_trade
                             self.execute_trade(trade_id, instrument_key, timestamp, signal, strategy_name, current_price, data_point)

                     except Exception as e:
                         logger.error(f"Error generating signal for {instrument_key} at {timestamp} using strategy '{strategy_name}': {e}", exc_info=True)
                         self.debug_log.append({'type': 'STRATEGY_ERROR', 'instrument': instrument_key, 'time': timestamp, 'strategy': strategy_name, 'error': str(e)})


        # After iterating through all data, close any remaining open positions
        self.close_all_positions(self.data['timestamp'].max()) # Use the timestamp of the last data point as exit time

        logger.info("Backtest simulation completed.")
        logger.info(f"Final Capital: {self.current_capital:.2f}")
        logger.info(f"Number of completed trades: {len(self.completed_trades)}")
        logger.info(f"Number of open positions remaining: {len(self.positions)}")

        # Return completed trades as a DataFrame for analysis
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

            # Ensure numeric columns are numeric
            numeric_cols = [
                'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
            ]
            for col in numeric_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')


            return completed_trades_df
        else:
            logger.warning("No completed trades recorded. Returning empty DataFrame.")
            return pd.DataFrame() # Return empty DataFrame if no trades


    def close_all_positions(self, exit_timestamp: datetime):
        """
        Closes all remaining open positions at the specified exit timestamp.
        Assumes closing at the price of the last available bar for each instrument.
        """
        logger.info(f"Closing all remaining {len(self.positions)} open positions at {exit_timestamp}...")

        # Get the last known price and data point for each instrument with an open position
        last_data_points = self.data.groupby('instrument_key').tail(1).set_index('instrument_key')
        last_prices = last_data_points['close'].to_dict()


        positions_to_close = list(self.positions.keys()) # Iterate over a copy

        for instrument_key in positions_to_close:
            # Check if position still exists (wasn't closed by a trigger just before the end)
            if instrument_key in self.positions:
                position = self.positions[instrument_key]
                closing_price = last_prices.get(instrument_key, np.nan) # Get last price, default to NaN if instrument not found

                # Get the last data point for the instrument to capture exit conditions
                last_data_point = last_data_points.get(instrument_key, pd.Series({})) # Use empty Series if no data found


                if pd.notna(closing_price):
                    # Use the dedicated close method for forced closure
                    self.close_position_on_trigger(instrument_key, exit_timestamp, closing_price, 'Forced_Close', last_data_point)

                else:
                    logger.warning(f"Could not find last price for {instrument_key}. Cannot close position {position['trade_id']}. Logging as unresolved.")
                    # Log as an unresolved position or assume zero PnL

                    # Calculate Trade Duration even if closing price is NaN
                    trade_duration = (exit_timestamp - position.get('entry_time')).total_seconds() if pd.notnull(exit_timestamp) and pd.notnull(position.get('entry_time')) else None

                    # Transfer known costs
                    entry_slippage = position.get('Slippage_Entry', 0)
                    entry_commission = position.get('Commission_Fees_Entry', 0)

                    unresolved_trade_record = {
                        'open_trade_id': position.get('trade_id'),
                        'close_trade_id': None, # No closing trade ID
                        'instrument_key': instrument_key,
                        'instrument_type': position.get('instrument_type'),
                        'side': position.get('side'),
                        'quantity': position.get('quantity'),
                        'entry_price': position.get('entry_price'),
                        'entry_time': position.get('entry_time'),
                        'exit_price': None, # No exit price
                        'exit_time': exit_timestamp, # Use the requested exit timestamp
                        'pnl': -(entry_slippage + entry_commission), # Only entry costs are known
                        'strategy_opened': position.get('strategy'),
                        'strategy_closed': 'Backtester_Forced_Close_Error', # Indicate error

                        # --- Transfer Entry Details from Position ---
                        'Strategy_name': position.get('Strategy_name'),
                        'instrument_key': position.get('instrument_key'),
                        'name': position.get('name'),
                        'interval': position.get('interval'),
                        'Position_type': position.get('Position_type'),
                        'Entry_order_type': position.get('Entry_order_type'),
                        'Entry_timestamp': position.get('Entry_timestamp'),
                        'Entry_price_trigger': position.get('Entry_price_trigger'),
                        'Entry_price_execution': position.get('Entry_price_execution'),
                        'Entry_shares': position.get('Entry_shares'),
                        'Entry_cost': position.get('Entry_cost'),
                        'Entry_signal_type': position.get('Entry_signal_type'),
                        'Entry_Trend': position.get('Entry_Trend'), # Get Trend at Entry from Position
                        'Entry_SMA20': position.get('Entry_SMA20'), # Get SMA20 at Entry from Position
                        'Entry_RSI': position.get('Entry_RSI'), # Get RSI at Entry from Position
                        'Entry_RSI_MA': position.get('RSI_MA'), # Get RSI_MA at Entry from Position
                        'Entry_ATR': position.get('Entry_ATR'), # Get ATR at Entry from Position
                        'Entry_ADX': position.get('ADX'), # Get ADX at Entry from Position
                        'Entry_Volatility': position.get('Entry_Volatility'), # Get Volatility at Entry from Position
                        'Entry_Breakout_Detected': position.get('Breakout_Detected'), # Get Breakout_Detected at Entry from Position
                        'Entry_Breakdown_Detected': position.get('Breakdown_Detected'), # Get Breakdown_Detected at Entry from Position
                        'Entry_Bullish_Candlestick_Name': position.get('Entry_Bullish_Candlestick_Name'), # Get Bullish Candlestick Name/Flag at Entry from Position
                        'Entry_Bearish_Candlestick_Name': position.get('Entry_Bearish_Candlestick_Name'), # Get Bearish Candlestick Name/Flag at Entry from Position
                        'Entry_Bullish_Chart_Pattern_Name': position.get('Entry_Bullish_Chart_Pattern_Name'), # Get Bullish Chart Pattern Name at Entry from Position
                        'Entry_Bearish_Chart_Pattern_Name': position.get('Entry_Bearish_Chart_Pattern_Name'), # Get Bearish Chart Pattern Name at Entry from Position
                        'Instrument_Type': position.get('Instrument_Type'), # Get Instrument Type at Entry from Position
                        'Currency': position.get('Currency'), # Get Currency at Entry from Position
                        'Slippage_Entry': entry_slippage, # Transfer entry slippage
                        'Commission_Fees_Entry': entry_commission, # Transfer entry commission
                        'Initial_Stop_Loss_Distance (%)': position.get('Initial_Stop_Loss_Distance (%)'),
                        'Risk_Amount': position.get('Risk_Amount'),
                        'Reward_Amount': position.get('Reward_Amount'),
                        'Max_Favorable_Excursion_MFE': position.get('Max_Favorable_Excursion_MFE'),
                        'Max_Adverse_Excursion_MAE': position.get('Max_Adverse_Excursion_MAE'),

                        # --- Exit-Specific Columns (Populated from last_data_point or defaults) ---
                        'Exit_Trend': last_data_point.get('Trend'), # Capture Trend at Forced Exit
                        'Exit_signal_type': 'Forced_Close_Error', # Indicate forced close error
                        'Exit_SMA20': last_data_point.get('SMA20'), # Capture SMA20 at Forced Exit
                        'Exit_RSI': last_data_point.get('RSI'), # Capture RSI at Forced Exit
                        'Exit_RSI_MA': last_data_point.get('RSI_MA'), # Capture RSI_MA at Forced Exit
                        'Exit_ATR': last_data_point.get('ATR'), # Capture ATR at Forced Exit
                        'Exit_ADX': last_data_point.get('ADX'), # Capture ADX at Forced Exit
                        'Exit_Volatility': last_data_point.get('Volatility'), # Capture Volatility at Forced Exit
                        'Exit_Breakout_Detected': last_data_point.get('Breakout_Detected'), # Capture Breakout_Detected at Forced Exit
                        'Exit_Breakdown_Detected': last_data_point.get('Breakdown_Detected'), # Capture Breakdown_Detected at Forced Exit
                        'Exit_Bullish_Candlestick_Name': last_data_point.get('Bullish_Candlestick_Detected'), # Capture Bullish Candlestick Name/Flag at Forced Exit
                        'Exit_Bearish_Candlestick_Name': last_data_point.get('Bearish_Candlestick_Detected'), # Capture Bearish Candlestick Name/Flag at Forced Exit
                        'Exit_Bullish_Chart_Pattern_Detected': last_data_point.get('Detected_Bullish_Chart_Pattern_Name'), # Capture Bullish Chart Pattern Name at Forced Exit
                        'Exit_Bearish_Chart_Pattern_Detected': last_data_point.get('Detected_Bearish_Chart_Pattern_Name'), # Capture Bearish Chart Pattern Name at Forced Exit
                        'Exit_shares': position.get('quantity'), # Shares that were supposed to be exited
                        'Exit_cost': 0,
                        'Exit_revenue': 0, # Assuming zero revenue if cannot close
                        'PnL_trade': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Profit_loss': -(entry_slippage + entry_commission), # Only entry costs are known
                        'Exit_reason': 'Backtester_Forced_Close_Error: No_Last_Price', # Reason for exit
                        'Exit_Order_Type': None, # Could not execute exit order

                         # Placeholder for other exit-specific details that might be calculated by strategy (e.g., trailing stop)
                        'Current_trailing_stop': position.get('Current_trailing_stop'), # Capture the stop level that was active
                        'Trailing_stop_method': position.get('Trailing_stop_method'),
                        'Trailing_stop_value': position.get('Trailing_stop_value'), # The value used by the method (e.g., percentage)
                        'Stop_loss_price': position.get('Stop_loss_price'), # Initial Stop Loss


                        'Slippage': entry_slippage, # Only entry slippage is known
                        'Commission_Fees': entry_commission, # Only entry commission is known
                        'Trade_Duration': trade_duration,


                    }
                    self.completed_trades.append(unresolved_trade_record)

                    # Remove position even if it couldn't be closed properly to prevent it from being processed again
                    del self.positions[instrument_key]

                    self.debug_log.append({'type': 'FORCED_CLOSE_ERROR', 'open_trade_id': position['trade_id'], 'instrument': instrument_key, 'time': exit_timestamp, 'reason': 'Last price not available'})


        logger.info("All remaining positions closed.")


    def analyze_backtest_results(self):
        """
        Analyzes the completed trades and provides performance metrics.
        Returns a DataFrame summarizing the analysis.
        """
        logger.info("Analyzing backtest results...")

        if not self.completed_trades:
            logger.warning("No completed trades to analyze.")
            return pd.DataFrame({'Message': ['No completed trades to analyze.']})

        # 1. Access the self.completed_trades list and Create a pandas DataFrame
        trades_df = pd.DataFrame(self.completed_trades)

        # 3. Ensure that relevant columns are converted to appropriate numeric types
        numeric_cols = [
            'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
            'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
            'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
            'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
            'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
            'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
            'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
            'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
            'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Include other potentially numeric cols
        ]
        for col in numeric_cols:
            if col in trades_df.columns:
                trades_df[col] = pd.to_numeric(trades_df[col], errors='coerce')

        # Handle potential NaN values during conversion - drop rows where PnL_trade (or pnl) is NaN
        pnl_col_for_analysis = 'PnL_trade' if 'PnL_trade' in trades_df.columns else 'pnl'
        if pnl_col_for_analysis in trades_df.columns:
            # Only consider trades with a valid PnL for core analysis metrics
            trades_df_analysis = trades_df.dropna(subset=[pnl_col_for_analysis]).copy() # Use .copy() to avoid SettingWithCopyWarning
        else:
            logger.warning("Neither 'PnL_trade' nor 'pnl' column found for analysis.")
            return pd.DataFrame({'Message': ['No PnL column found for analysis.']})


        if trades_df_analysis.empty:
            logger.warning("No valid trades after numeric conversion/dropna for analysis. Analysis stopped.")
            return pd.DataFrame({'Message': ['No valid trades after numeric conversion/dropna for analysis.']})


        # 4. Update the calculation of basic performance metrics using 'PnL_trade'
        total_trades = len(trades_df_analysis)
        total_pnl = trades_df_analysis[pnl_col_for_analysis].sum()

        winning_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] > 0]
        losing_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] < 0]
        breakeven_trades = trades_df_analysis[trades_df_analysis[pnl_col_for_analysis] == 0]

        num_winning = len(winning_trades)
        num_losing = len(losing_trades)
        num_breakeven = len(breakeven_trades)

        win_rate = (num_winning / total_trades) * 100 if total_trades > 0 else 0
        avg_win = winning_trades[pnl_col_for_analysis].mean() if num_winning > 0 else 0
        avg_loss = losing_trades[pnl_col_for_analysis].mean() if num_losing > 0 else 0
        expectancy = (win_rate / 100) * avg_win + ((100 - win_rate) / 100) * avg_loss if total_trades > 0 else 0

        # 5. Update Max Drawdown calculation to use 'PnL_trade' and sort by exit time
        # Calculate cumulative PnL and then cumulative capital
        trades_df_analysis = trades_df_analysis.sort_values(by='exit_time') # Sort by exit time for cumulative calculation

        trades_df_analysis['cumulative_pnl'] = trades_df_analysis[pnl_col_for_analysis].cumsum()

        # Add initial capital to cumulative PnL
        trades_df_analysis['cumulative_capital'] = self.initial_capital + trades_df_analysis['cumulative_pnl']

        # Calculate peak capital up to each point
        trades_df_analysis['peak_capital'] = trades_df_analysis['cumulative_capital'].cummax()

        # Calculate drawdown at each point
        trades_df_analysis['drawdown'] = trades_df_analysis['peak_capital'] - trades_df_analysis['cumulative_capital']

        # Calculate percentage drawdown
        # Avoid division by zero if peak_capital is 0 or None
        trades_df_analysis['pct_drawdown'] = trades_df_analysis.apply(
            lambda row: (row['drawdown'] / row['peak_capital']) * 100 if row['peak_capital'] > 0 and pd.notna(row['peak_capital']) else 0,
            axis=1
        )


        max_drawdown_amount = trades_df_analysis['drawdown'].max() if not trades_df_analysis.empty else 0
        max_drawdown_pct = trades_df_analysis['pct_drawdown'].max() if not trades_df_analysis.empty else 0


        # 6. Update analysis summary metric names
        analysis_summary = {
            'Metric': [
                'Initial Capital',
                'Final Capital',
                'Total PnL (Net)', # Indicate Net PnL
                'Total Trades',
                'Winning Trades (Net)', # Indicate Net PnL
                'Losing Trades (Net)', # Indicate Net PnL
                'Breakeven Trades (Net)', # Indicate Net PnL
                'Win Rate (%) (Net PnL)', # Indicate Net PnL
                'Average Win (Net)', # Indicate Net PnL
                'Average Loss (Net)', # Indicate Net PnL
                'Expectancy per Trade (Net)', # Indicate Net PnL
                'Max Drawdown (Amount)',
                'Max Drawdown (%)',
            ],
            'Value': [
                self.initial_capital,
                self.current_capital,
                round(total_pnl, 2), # Format to 2 decimal places
                total_trades,
                num_winning,
                num_losing,
                num_breakeven,
                round(win_rate, 2), # Format to 2 decimal places
                round(avg_win, 2),
                round(avg_loss, 2),
                round(expectancy, 2),
                round(max_drawdown_amount, 2),
                round(max_drawdown_pct, 2),
            ]
        }

        analysis_df = pd.DataFrame(analysis_summary)

        logger.info("Backtest analysis completed.")
        # You can print the analysis_df here or return it
        # print("\n--- Backtest Analysis Summary ---")
        # display(analysis_df) # Use display for notebooks

        # 8. Ensure the method returns the updated analysis summary DataFrame
        return analysis_df

    def get_completed_trades(self):
        """Returns a DataFrame of completed trades."""
        if self.completed_trades:
            completed_trades_df = pd.DataFrame(self.completed_trades)
            # Ensure timestamp columns are datetime objects
            timestamp_cols = ['entry_time', 'exit_time', 'Entry_timestamp'] # Add new timestamp columns
            for col in timestamp_cols:
                if col in completed_trades_df.columns:
                    completed_trades_df[col] = pd.to_datetime(completed_trades_df[col], errors='coerce', utc=True)

                    # Ensure numeric columns are numeric
                    numeric_cols = [
                        'Entry_price_trigger', 'Entry_price_execution', 'Entry_shares', 'Entry_cost',
                        'Entry_Trend', 'Entry_SMA20', 'Entry_RSI', 'Entry_RSI_MA', 'Entry_ATR',
                        'Entry_ADX', 'Entry_Volatility', 'Exit_Trend', 'Exit_SMA20', 'Exit_RSI',
                        'Exit_RSI_MA', 'Exit_ATR', 'Exit_ADX', 'Exit_Volatility', 'Exit_shares',
                        'Exit_cost', 'Exit_revenue', 'PnL_trade', 'Profit_loss', 'Initial_Stop_Loss_Distance (%)',
                        'Risk_Amount', 'Reward_Amount', 'Max_Favorable_Excursion_MFE',
                        'Max_Adverse_Excursion_MAE', 'Slippage', 'Commission_Fees', 'Trade_Duration',
                        'Current_trailing_stop', 'Trailing_stop_method', 'Trailing_stop_value', 'Stop_loss_price',
                        'Slippage_Entry', 'Commission_Fees_Entry', 'pnl', 'quantity', 'entry_price', 'exit_price' # Added original pnl and price/qty
                    ]
                    for col in numeric_cols:
                        if col in completed_trades_df.columns:
                            completed_trades_df[col] = pd.to_numeric(completed_trades_df[col], errors='coerce')

                    return completed_trades_df
                else:
                    return pd.DataFrame() # Return empty DataFrame if no trades

Logging level set to DEBUG for test.
